<a href="https://colab.research.google.com/github/dsporcaro/Anythink-Market-73vwk/blob/main/CRDC_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Installs


In [ ]:
!pip install neo4j

## Credentials

In [ ]:
# David's dev env
NEO4J_URI = "neo4j+s://2a5877d3.databases.neo4j.io"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "1WdDxFnlqo8l-EjYez4FJVFqenJAR6G4b21A0HNlbFQ"

# Production database
#NEO4J_URI = "neo4j://db.learningdesignalliance.org:7687"
#NEO4J_USER = "neo4j"
#NEO4J_PASSWORD = "F8x5s3R6t7TdW7c"

import os
os.chdir("/content/drive/MyDrive/CRDC")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dtype_dict = {
    'NCESSCH': str,  # Institution ID
    'COMBOKEY': str, # Institution ID
    'LEAID': str,     # LEA ID
    'UNION': str
}


## Workspace

###Check for individual records

In [ ]:
import pandas as pd

def existing_check(df, year, nces_ids):
    # Filter the dataframe to only include rows with NCES IDs in the specified list
    df_filtered = df[df['COMBOKEY'].isin(nces_ids)]

    if df_filtered.empty:
        print(f"No matching NCES IDs found for the specified list in the year {year}.")
        return []

    # Identify columns that are not of string type and exclude 'SCHID'
    numeric_columns = df_filtered.select_dtypes(include=[int, float]).columns
    numeric_columns = numeric_columns.drop('SCHID')  # Exclude 'SCHID' from the columns to check

    # Check if any of these numeric columns have values greater than 0
    if not (df_filtered[numeric_columns] > 0).any().any():  # `.any().any()` to check across all columns and then across all rows
        print("Matching NCES IDs found, but no valid positive data entries present.")
        return []
    else:
        print("Yup, they're there with valid positive data entries.")
        return df_filtered[(df_filtered[numeric_columns] > 0).any(axis=1)]

# Example usage
year = 2020
check = existing_check(internet_df, year, nces_id_list)

### Transform specific records

In [ ]:
from collections import defaultdict

def transform_internet_data(df, year, nces_ids):
    df_filtered = df[df['COMBOKEY'].isin(nces_ids)]

    transformed_data = []
    for _, row in df_filtered.iterrows():
        descriptions = [str(year)]  # Start with the year in the descriptions list

        # Check each condition and append the description if "Yes"
        if row["SCH_INTERNET_FIBER"] == "Yes":
            descriptions.append("Fiber-optic connection at school")
        if row["SCH_INTERNET_WIFI"] == "Yes":
            descriptions.append("Wifi in every classroom")
        if row["SCH_INTERNET_SCHDEV"] == "Yes":
            descriptions.append("Students have take-home device for Internet access")
        if row["SCH_INTERNET_STUDDEV"] == "Yes":
            descriptions.append("Students can use student-owned devices for Internet access")

        # Check if the number of Wi-Fi-enabled devices is greater than 0
        wifi_devices = int(row["SCH_INTERNET_WIFIENDEV"])
        if wifi_devices > 0:
            descriptions.append(f"{wifi_devices} student devices provided to students")

        # Join the descriptions into a single string, including the year at the beginning
        access_description = ", ".join(descriptions)

        # Construct the transformed data for the school
        school_data = {
            "institution_id": row["COMBOKEY"],
            "institution_name": row["SCH_NAME"],
            "jurisdiction_name": row["LEA_STATE_NAME"],
            "lea_id": row["LEAID"],
            "lea_name": row["LEA_NAME"],
            "Internet Access and Devices": access_description
        }

        transformed_data.append(school_data)

    return transformed_data

internet_df_subset = internet_df #.head(200)
transformed_internet_data = transform_internet_data(internet_df_subset, 2020, nces_id_list)
print(transformed_internet_data[:5])

### Load those specific records

In [ ]:
from neo4j import GraphDatabase

def insert_test_data(session, data):
    cypher_query = """
    UNWIND $data AS schoolData
    // Create or find Institution
    MERGE (institution:Institution {ncesID: schoolData.institution_id})
        ON CREATE SET institution.name = schoolData.institution_name,
                      institution.internetAccessDevices = schoolData.`Internet Access and Devices`
        ON MATCH SET institution.internetAccessDevices = schoolData.`Internet Access and Devices`
    """
    session.run(cypher_query, {"data": data})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Execute the insertion
with driver.session() as session:
    insert_test_data(session, transformed_internet_data)

driver.close()

## EdFacts Absenteeism

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
chronic_absenteeism_path = './ID 814 SCH - Chronic Absenteeism.csv'
chronic_absenteeism_df = pd.read_csv(chronic_absenteeism_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Chronic Absenteeism Data Preview:")
print(chronic_absenteeism_df.head(), "\n")


Chronic Absenteeism Data Preview:
  LEA_STATE LEA_STATE_NAME    LEAID          LEA_NAME  SCHID  \
0        AL        ALABAMA  0100005  Albertville City    870   
1        AL        ALABAMA  0100005  Albertville City    871   
2        AL        ALABAMA  0100005  Albertville City    879   
3        AL        ALABAMA  0100005  Albertville City    889   
4        AL        ALABAMA  0100005  Albertville City   1616   

                         SCHOOL_NAME       NCESSCH  SCH_ABSENT_AM_M  \
0          Albertville Middle School  010000500870               -8   
1            Albertville High School  010000500871               -8   
2    Albertville Intermediate School  010000500879               -8   
3      Albertville Elementary School  010000500889               -8   
4  Albertville Kindergarten and PreK  010000501616               -8   

   SCH_ABSENT_AS_M  SCH_ABSENT_BL_M  ...  SCH_ABSENT_WH_F  SCH_ABSENT_EUT  \
0               -8                2  ...               34             107   


### Transform

In [ ]:
# Mapping for absenteeism data columns to descriptive node names
column_to_description_mapping = {
    'SCH_ABSENT_AM_M': "American Indian/Alaskan Native Male",
    'SCH_ABSENT_AS_M': "Asian Male",
    'SCH_ABSENT_BL_M': "Black Male",
    'SCH_ABSENT_HI_M': "Hispanic Male",
    'SCH_ABSENT_MU_M': "Two or more races Male",
    'SCH_ABSENT_PI_M': "Native Hawaiian or other Pacific Islander Male",
    'SCH_ABSENT_WH_M': "White Male",
    'SCH_ABSENT_AM_F': "American Indian/Alaskan Native Female",
    'SCH_ABSENT_AS_F': "Asian Female",
    'SCH_ABSENT_BL_F': "Black Female",
    'SCH_ABSENT_HI_F': "Hispanic Female",
    'SCH_ABSENT_MU_F': "Two or more races Female",
    'SCH_ABSENT_PI_F': "Native Hawaiian or other Pacific Islander Female",
    'SCH_ABSENT_WH_F': "White Female",
    'SCH_ABSENT_IDEA_M': "Student with disability (IDEA) Male",
    'SCH_ABSENT_IDEA_F': "Student with disability (IDEA) Female",
    'SCH_ABSENT_504_M': "Student with disability (Section 504) Male",
    'SCH_ABSENT_504_F': "Student with disability (Section 504) Female",
    'SCH_ABSENT_LEP_M': "English Learner (EL) Student Male",
    'SCH_ABSENT_LEP_F': "English Learner (EL) Student Female",
    'SCH_ABSENT_HMENRL_M': "Homeless enrolled students Male",
    'SCH_ABSENT_HMENRL_F': "Homeless enrolled students Female"
}

def prepare_absentee_data(df, year):
    transformed_absentee_data = []

    for index, row in df.iterrows():
        school_data = {
            "school_id": row['NCESSCH'],
            "school_name": row['SCHOOL_NAME'],
            "lea_id": row['LEAID'],
            "lea_name": row['LEA_NAME'],
            "jurisdiction_name": row['LEA_STATE_NAME'].title(),
            "year": year,
            "subgroups": {},
            "total_students": {"Male": 0, "Female": 0, "Total": 0}
        }

        for column, description in column_to_description_mapping.items():
            value = row[column]
            if pd.notnull(value) and value > 0:
                # Split the description to get subgroup and gender
                subgroup, gender = description.rsplit(' ', 1)
                if subgroup not in school_data["subgroups"]:
                    school_data["subgroups"][subgroup] = {"Male": 0, "Female": 0, "Total": 0}
                school_data["subgroups"][subgroup][gender] += value
                school_data["subgroups"][subgroup]["Total"] += value
                school_data["total_students"][gender] += value
                school_data["total_students"]["Total"] += value

        if school_data["subgroups"]:
            transformed_absentee_data.append(school_data)

    return transformed_absentee_data

year = 2020
transformed_absentee_data = prepare_absentee_data(chronic_absenteeism_df, year)

print(transformed_absentee_data[:5])  # Print details of the first 5 schools for inspection


[{'school_id': '010000500870', 'school_name': 'Albertville Middle School', 'lea_id': '0100005', 'lea_name': 'Albertville City', 'jurisdiction_name': 'Alabama', 'year': 2020, 'subgroups': {'Black': {'Male': 2, 'Female': 1, 'Total': 3}, 'Hispanic': {'Male': 25, 'Female': 15, 'Total': 40}, 'Two or more races': {'Male': 7, 'Female': 1, 'Total': 8}, 'White': {'Male': 22, 'Female': 34, 'Total': 56}, 'Student with disability (IDEA)': {'Male': 8, 'Female': 4, 'Total': 12}, 'Student with disability (Section 504)': {'Male': 2, 'Female': 0, 'Total': 2}, 'English Learner (EL) Student': {'Male': 11, 'Female': 6, 'Total': 17}, 'Homeless enrolled students': {'Male': 3, 'Female': 0, 'Total': 3}}, 'total_students': {'Male': 80, 'Female': 61, 'Total': 141}}, {'school_id': '010000500871', 'school_name': 'Albertville High School', 'lea_id': '0100005', 'lea_name': 'Albertville City', 'jurisdiction_name': 'Alabama', 'year': 2020, 'subgroups': {'Black': {'Male': 11, 'Female': 5, 'Total': 16}, 'Hispanic': {'M

In [ ]:
if transformed_absentee_data:  # Check if the list is not empty
    first_record = transformed_absentee_data[0]
    print(first_record)
else:
    print("transformed_data is empty.")

transformed_data is empty.


###Load

In [ ]:
test_data = transformed_absentee_data
from tqdm import tqdm

from neo4j import GraphDatabase

def insert_absenteeism_data(session, data):
    cypher_query = """
    UNWIND $data AS schoolData
    // Create or update Jurisdiction, LEA, and Institution nodes
    MERGE (j:Jurisdiction {name: schoolData.jurisdiction_name})
    MERGE (lea:LEA {ncesID: schoolData.lea_id, name: schoolData.lea_name})
    MERGE (j)-[:INCLUDES]->(lea)
    MERGE (institution:Institution {ncesID: schoolData.school_id, name: schoolData.school_name})
    MERGE (lea)-[:INCLUDES]->(institution)

    // Handle Subgroups and Chronic Absenteeism Outcome for all, including "Total Students"
    WITH schoolData, institution
    UNWIND keys(schoolData.subgroups) AS subgroupKey
    MERGE (subgroup:Subgroup {name: CASE WHEN subgroupKey = 'total_students' THEN 'Total Students' ELSE subgroupKey END})
    WITH institution, schoolData, subgroup, subgroupKey, schoolData.subgroups[subgroupKey] AS subgroupData
    MERGE (outcome:Outcome {name: 'Chronic Absenteeism', date: schoolData.year})
    MERGE (institution)-[reports:REPORTS {date: schoolData.year, subgroup: CASE WHEN subgroupKey = 'total_students' THEN 'Total Students' ELSE subgroupKey END}]->(outcome)
    ON CREATE SET reports.male = subgroupData.Male, reports.female = subgroupData.Female, reports.total = subgroupData.Total
    ON MATCH SET reports.male = subgroupData.Male, reports.female = subgroupData.Female, reports.total = subgroupData.Total
    """

    session.run(cypher_query, {"data": data})

# Initialize Neo4j driver
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Execute insertion
with driver.session() as session:
  for i in tqdm(range(0, len(test_data) - 1, 1000)):
    sub_data = test_data [i:i+1000]
    insert_absenteeism_data(session, sub_data)

driver.close()


100%|██████████| 93/93 [1:41:31<00:00, 65.50s/it]


##Edfacts Race by Placement

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
race_by_placement_path = './ID 74 SCH - Race by Placement.csv'  # Update with the actual file path
race_by_placement_df = pd.read_csv(race_by_placement_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Race by Placement Preview:")
print(race_by_placement_df.head(), "\n")

Race by Placement Preview:
  LEA_STATE LEA_STATE_NAME    LEAID         LEA_NAME  SCHID  \
0        AL        ALABAMA  0103580   Winston County   1357   
1        AL        ALABAMA  0103030    Shelby County   1588   
2        AL        ALABAMA  0103030    Shelby County   1588   
3        AL        ALABAMA  0103580   Winston County   1357   
4        AL        ALABAMA  0101800  Huntsville City   2379   

                    SCHOOL_NAME       NCESSCH EDUCATIONAL_ENVIRONMENT  \
0    Winston County High School  010358001357                    RC80   
1  Elvin Hill Elementary School  010303001588                    RC80   
2  Elvin Hill Elementary School  010303001588                RC79TO40   
3    Winston County High School  010358001357                RC79TO40   
4              Ronald McNair 78  010180002379                    RC80   

   AM_RACE_7  AS_RACE_7  BL_RACE_7  HI_RACE_7  MU_RACE_7  PI_RACE_7  \
0          0          0          0          1          0          0   
1          0 

### Transform

In [ ]:
from collections import defaultdict
import pandas as pd

educational_environment_mapping = {
    "RC80": "Inside regular class 80% or more of the day",
    "RC79TO40": "Inside regular class 40% through 79% of the day",
    "RC39": "Inside regular class less than 40% of the day",
    "SS": "Separate School",
    "RF": "Residential Facility",
    "HH": "Homebound/Hospital",
    "CF": "Correctional Facility",
    "PPPS": "Parentally Placed in Private Schools"
}

race_mapping = {
    "AM_RACE_7": "American Indian/Alaskan Native with Disability",
    "AS_RACE_7": "Asian with Disability",
    "BL_RACE_7": "Black with Disability",
    "HI_RACE_7": "Hispanic with Disability",
    "MU_RACE_7": "Two or more races with Disability",
    "PI_RACE_7": "Native Hawaiian or other Pacific Islander with Disability",
    "WH_RACE_7": "White with Disability",
    "TOTAL_STUDENTS_REPORTED": "Students with Disability"
}

def prepare_race_by_disability(df, year):
    school_data = {}

    for index, row in df.iterrows():
        school_id = row['NCESSCH']

        if school_id not in school_data:
            school_data[school_id] = {
                "institution_id": school_id,
                "institution_name": row['SCHOOL_NAME'],
                "jurisdiction_name": row['LEA_STATE_NAME'].title(),
                "lea_id": row['LEAID'],
                "lea_name": row['LEA_NAME'],
                "year": year,
                "placements": [],
                "aggregated_data": {}

            }

        race_placement = {race_mapping[key]: row[key] for key in race_mapping if pd.notnull(row[key]) and row[key] > 0}

        placement = {
            "educational_environment_code": row['EDUCATIONAL_ENVIRONMENT'],
            "educational_environment_name": educational_environment_mapping.get(row['EDUCATIONAL_ENVIRONMENT'], "Unknown Environment"),
            "race_placement": {race_mapping[key]: row[key] for key in race_mapping.keys() if pd.notnull(row[key]) and row[key] > 0}
        }

        school_data[school_id]["placements"].append(placement)

    transformed_disability_race_data = list(school_data.values())

    for school in transformed_disability_race_data:
        for placement in school['placements']:
            for subgroup, total in placement['race_placement'].items():
                if subgroup not in school['aggregated_data']:
                    school['aggregated_data'][subgroup] = 0
                school['aggregated_data'][subgroup] += total

    return transformed_disability_race_data

year = 2020
transformed_disability_race_data = prepare_race_by_disability(race_by_placement_df, year)

print(transformed_absentee_data[:5])  # Print details of the first 5 schools for inspection


No matching NCES IDs found for the specified list in the year 2020.
[]


In [ ]:
if transformed_disability_race_data:  # Check if the list is not empty
    first_record = transformed_disability_race_data[0:5]
    print(first_record)
else:
    print("transformed_disability_race_data is empty.")

[{'institution_id': 10358001357, 'institution_name': 'Winston County High School', 'jurisdiction_name': 'ALABAMA', 'lea_id': 103580, 'lea_name': 'Winston County', 'year': 2020, 'placements': [{'educational_environment_code': 'RC80', 'educational_environment_name': 'Inside regular class 80% or more of the day', 'race_placement': {'Hispanic with Disability': 1, 'White with Disability': 30, 'Students with Disability': 31}}, {'educational_environment_code': 'RC79TO40', 'educational_environment_name': 'Inside regular class 40% through 79% of the day', 'race_placement': {'White with Disability': 30, 'Students with Disability': 30}}], 'aggregated_data': {'Hispanic with Disability': 1, 'White with Disability': 60, 'Students with Disability': 61}}, {'institution_id': 10303001588, 'institution_name': 'Elvin Hill Elementary School', 'jurisdiction_name': 'ALABAMA', 'lea_id': 103030, 'lea_name': 'Shelby County', 'year': 2020, 'placements': [{'educational_environment_code': 'RC80', 'educational_envi

###Load

In [ ]:
test_data = transformed_disability_race_data
from tqdm import tqdm

from neo4j import GraphDatabase

# Function to insert data into Neo4j using the Cypher query
def insert_test_data(session, data):
    cypher_query = """
    UNWIND $data AS school
    MERGE (jurisdiction:Jurisdiction {name: school.jurisdiction_name})
    MERGE (lea:LEA {ncesID: school.lea_id})
        ON CREATE SET lea.name = school.lea_name
    MERGE (jurisdiction)-[:INCLUDES]->(lea)
    MERGE (institution:Institution {ncesID: school.institution_id})
        ON CREATE SET institution.name = school.institution_name
    MERGE (lea)-[:INCLUDES]->(institution)

    WITH school, institution
    UNWIND school.placements AS placement
    MERGE (intervention:Intervention {code: placement.educational_environment_code})
        ON CREATE SET intervention.name = placement.educational_environment_name

    WITH institution, intervention, placement, school
    UNWIND keys(placement.race_placement) AS subgroupKey
    MERGE (subgroup:Subgroup {name: subgroupKey})
    MERGE (institution)-[rAssigns:ASSIGNS {date: school.year, subgroup: subgroup.name}]->(intervention)
        ON CREATE SET rAssigns.total = placement.race_placement[subgroupKey], rAssigns.subgroup = subgroup.name
        ON MATCH SET rAssigns.total = placement.race_placement[subgroupKey], rAssigns.subgroup = subgroup.name
        """
    session.run(cypher_query, {"data": data})

# Assuming you have already initialized the Neo4j driver
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Execute the insertion for the test data
with driver.session() as session:
  for i in tqdm(range(0, len(test_data) - 1, 1000)):
    sub_data = test_data [i:i+1000]
    insert_test_data(session, sub_data)

driver.close()



100%|██████████| 90/90 [2:05:33<00:00, 83.70s/it]


## Edfacts Educational Environment by Gender by Disability

### Extract

In [ ]:
import pandas as pd

# Replace the path with the actual location of your CSV file
file_path = './ID 74 SCH - Educational Environment by Gender by Disability.csv'
disability_df = pd.read_csv(file_path, dtype=dtype_dict)

# Display the first few rows to verify the data
print(disability_df.head())

  LEA_STATE LEA_STATE_NAME    LEAID        LEA_NAME  SCHID  \
0        AL        ALABAMA  0103580  Winston County   1357   
1        AL        ALABAMA  0103580  Winston County   1357   
2        AL        ALABAMA  0103580  Winston County   1357   
3        AL        ALABAMA  0103030   Shelby County   1588   
4        AL        ALABAMA  0103030   Shelby County   1588   

                    SCHOOL_NAME       NCESSCH DISABILITY_CATEGORY  CF_M  HH_M  \
0    Winston County High School  010358001357                 AUT     0     0   
1    Winston County High School  010358001357                  ID     0     0   
2    Winston County High School  010358001357                 OHI     0     0   
3  Elvin Hill Elementary School  010303001588                 AUT     0     0   
4  Elvin Hill Elementary School  010303001588                  DD     0     0   

   ...  SS_M  CF_F  HH_F  PPPS_F  RC39_F  RC79TO40_F  RC80_F  RF_F  SS_F  \
0  ...     0     0     0       0       0           0      12    

### Transform

In [ ]:
from collections import defaultdict
import pandas as pd
from tqdm import tqdm

disability_categories_mapping = {
    "AUT": "Autism",
    "DB": "Deaf-blindness",
    "DD": "Developmental delay",
    "EMN": "Emotional disturbance",
    "HI": "Hearing impairment",
    "ID": "Intellectual disability",
    "MD": "Multiple disabilities",
    "OI": "Orthopedic impairment",
    "SLD": "Specific learning disability",
    "SLI": "Speech or language impairment",
    "TBI": "Traumatic brain injury",
    "VI": "Visual impairment",
    "OHI": "Other health impairment",
    "MISSING": "Missing data"
}

educational_environment_categories = {
    "RC80": "Inside regular class 80% or more of the day",
    "RC79TO40": "Inside regular class 40% through 79% of the day",
    "RC39": "Inside regular class less than 40% of the day",
    "SS": "Separate School",
    "RF": "Residential Facility",
    "HH": "Homebound/Hospital",
    "CF": "Correctional Facility",
    "PPPS": "Parentally Placed in Private Schools"
}



def transform_disability_gender_data(df, year):
    transformed_data = []

    for _, row in df.iterrows():
        school_id = row["NCESSCH"]

        school_data = next((item for item in transformed_data if item["institution_id"] == school_id), None)
        if school_data is None:
            school_data = {
                "institution_id": school_id,
                "institution_name": row['SCHOOL_NAME'],
                "jurisdiction_name": row['LEA_STATE_NAME'].title(),
                "lea_id": row['LEAID'],
                "lea_name": row['LEA_NAME'],
                "year": year,
                "placements": {},
                "total": {"Male": 0, "Female": 0, "Total": 0},
                "disabilities": defaultdict(lambda: {"Male": 0, "Female": 0, "Total": 0})
            }
            transformed_data.append(school_data)

         # Process counts for each educational environment and gender
        for env_code, env_name in educational_environment_categories.items():
            male_count = int(row.get(f"{env_code}_M", 0))
            female_count = int(row.get(f"{env_code}_F", 0))
            total_count = male_count + female_count

            if total_count > 0:
                disability = disability_categories_mapping.get(row['DISABILITY_CATEGORY'], "Unknown")

                # Initialize placement if it doesn't exist
                if env_code not in school_data['placements']:
                    school_data['placements'][env_code] = {
                        "name": env_name,
                        "disabilities": defaultdict(lambda: {"Male": 0, "Female": 0, "Total": 0}),
                        "aggregated": {"Male": 0, "Female": 0, "Total": 0}
                    }

                placement = school_data['placements'][env_code]
                placement['disabilities'][disability]['Male'] += male_count
                placement['disabilities'][disability]['Female'] += female_count
                placement['disabilities'][disability]['Total'] += total_count

                # Aggregate totals for this environment
                placement['aggregated']['Male'] += male_count
                placement['aggregated']['Female'] += female_count
                placement['aggregated']['Total'] += total_count

                # Aggregate totals for disabilities across all environments
                school_data['disabilities'][disability]['Male'] += male_count
                school_data['disabilities'][disability]['Female'] += female_count
                school_data['disabilities'][disability]['Total'] += total_count

    # Convert defaultdicts to regular dicts for serialization or further processing
    for school in transformed_data:
        school['placements'] = {env_code: {**env_data, "disabilities": dict(env_data['disabilities'])} for env_code, env_data in school['placements'].items()}
        school['disabilities'] = dict(school['disabilities'])

    return transformed_data


# Initialize the list where transformed data will be stored
transformed_disability_gender_data = []

# Process the dataframe in batches
for i in tqdm(range(0, len(disability_df), 1000)):
    # Select a subset of the dataframe for the current batch
    sub_data = disability_df[i:i+1000]

    # Transform the current batch and retrieve the results
    batch_results = transform_disability_gender_data(sub_data, 2020)

    # Append the results of the current batch to the main list
    transformed_disability_gender_data.extend(batch_results)

#df_subset = disability_df.head(200)

# Now, use this subset to transform the data
#transformed_disability_gender_data = transform_disability_gender_data(df_subset, 2020)

# Optionally, print or inspect part of the transformed data to check its structure
print(transformed_disability_gender_data[:5])  # Print details of the first 5 schools for inspection

100%|██████████| 577/577 [01:35<00:00,  6.07it/s]

[{'institution_id': '010358001357', 'institution_name': 'Winston County High School', 'jurisdiction_name': 'Alabama', 'lea_id': '0103580', 'lea_name': 'Winston County', 'year': 2020, 'placements': {'RC80': {'name': 'Inside regular class 80% or more of the day', 'disabilities': {'Autism': {'Male': 15, 'Female': 12, 'Total': 27}, 'Intellectual disability': {'Male': 0, 'Female': 12, 'Total': 12}, 'Other health impairment': {'Male': 15, 'Female': 0, 'Total': 15}, 'Hearing impairment': {'Male': 15, 'Female': 12, 'Total': 27}, 'Multiple disabilities': {'Male': 15, 'Female': 0, 'Total': 15}, 'Specific learning disability': {'Male': 15, 'Female': 12, 'Total': 27}}, 'aggregated': {'Male': 75, 'Female': 48, 'Total': 123}}, 'RC79TO40': {'name': 'Inside regular class 40% through 79% of the day', 'disabilities': {'Autism': {'Male': 3, 'Female': 0, 'Total': 3}, 'Intellectual disability': {'Male': 3, 'Female': 0, 'Total': 3}, 'Other health impairment': {'Male': 0, 'Female': 1, 'Total': 1}}, 'aggregat

### Load

In [ ]:
test_data = transformed_disability_gender_data

from neo4j import GraphDatabase

# Function to insert data into Neo4j using the Cypher query
def insert_test_data(session, data):
    cypher_query = """
    UNWIND $data AS schoolData
    // Create or update Jurisdiction, LEA, and Institution nodes
    MERGE (jurisdiction:Jurisdiction {name: schoolData.jurisdiction_name})
    MERGE (lea:LEA {ncesID: schoolData.lea_id})
        ON CREATE SET lea.name = schoolData.lea_name
    MERGE (jurisdiction)-[:INCLUDES]->(lea)
    MERGE (institution:Institution {ncesID: schoolData.institution_id})
        ON CREATE SET institution.name = schoolData.institution_name
    MERGE (lea)-[:INCLUDES]->(institution)

    // Aggregate totals for subgroups across placements
    WITH schoolData, institution
    UNWIND keys(schoolData.disabilities) AS disability
    MERGE (subgroup:Subgroup {name: disability})
    MERGE (institution)-[hasSubgroup:HAS_SUBGROUP]->(subgroup)
        ON CREATE SET hasSubgroup.total = schoolData.disabilities[disability].Total,
                      hasSubgroup.male = schoolData.disabilities[disability].Male,
                      hasSubgroup.female = schoolData.disabilities[disability].Female,
                      hasSubgroup.date = schoolData.year
        ON MATCH SET  hasSubgroup.total = schoolData.disabilities[disability].Total,
                      hasSubgroup.male = schoolData.disabilities[disability].Male,
                      hasSubgroup.female = schoolData.disabilities[disability].Female,
                      hasSubgroup.date = schoolData.year

    // Assigning intervention based on placement
    WITH schoolData, institution
    UNWIND keys(schoolData.placements) AS placementCode
    MERGE (intervention:Intervention {code: placementCode, name: schoolData.placements[placementCode].name})
    WITH schoolData, institution, intervention, placementCode
    UNWIND keys(schoolData.placements[placementCode].disabilities) AS disability
    MERGE (subgroup:Subgroup {name: disability})
    WITH institution, intervention, subgroup, schoolData.placements[placementCode].disabilities[disability] AS disabilityData, schoolData.year AS year
    MERGE (institution)-[assigns:ASSIGNS {date: year, subgroup: subgroup.name}]->(intervention)
        ON CREATE SET assigns.total = disabilityData.Total,
                      assigns.male = disabilityData.Male,
                      assigns.female = disabilityData.Female
        ON MATCH SET  assigns.total = disabilityData.Total,
                      assigns.male = disabilityData.Male,
                      assigns.female = disabilityData.Female
        """
    session.run(cypher_query, {"data": data})

# Assuming you have already initialized the Neo4j driver
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Execute the insertion for the test data
with driver.session() as session:
  for i in tqdm(range(88000, len(test_data) - 1, 1000)):
    sub_data = test_data [i:i+1000]
    insert_test_data(session, sub_data)
driver.close()

100%|██████████| 3/3 [01:54<00:00, 38.23s/it]


## CRDC Advanced Math

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
advanced_math_path = './Advanced Mathematics.csv'  # Update with the actual file path
advanced_math_df = pd.read_csv(advanced_math_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Advanced math Preview:")
print(advanced_math_df.head(), "\n")

Advanced math Preview:
  LEA_STATE LEA_STATE_NAME    LEAID                LEA_NAME  SCHID  \
0        AL        Alabama  0100002  Alabama Youth Services   1705   
1        AL        Alabama  0100002  Alabama Youth Services   1706   
2        AL        Alabama  0100002  Alabama Youth Services  99995   
3        AL        Alabama  0100005        Albertville City    870   
4        AL        Alabama  0100005        Albertville City    871   

                        SCH_NAME      COMBOKEY   JJ  SCH_MATHCLASSES_ADVM  \
0  Wallace Sch - Mt Meigs Campus  010000201705  Yes                     0   
1      Mcneel Sch - Vacca Campus  010000201706  Yes                     0   
2                 Autauga Campus  010000299995  Yes                     0   
3      Albertville Middle School  010000500870   No                    -9   
4        Albertville High School  010000500871   No                    10   

   SCH_MATHCERT_ADVM  ...  SCH_MATHENR_ADVM_WH_M  SCH_MATHENR_ADVM_WH_F  \
0                 

### Transform

In [ ]:
def existing_check(df, year, nces_ids):
    # Filter the dataframe to only include rows with NCES IDs in the specified list
    df_filtered = df[df['COMBOKEY'].isin(nces_ids)]

    if df_filtered.empty:
        print(f"No matching NCES IDs found for the specified list in the year {year}.")
        return []

    else:
        print("yup they're there")

year = 2020
check = existing_check(advanced_math_df, year, nces_id_list)


yup they're there


In [ ]:
from collections import defaultdict
import pandas as pd

def transform_advanced_math_data(df, year):
    race_categories = {
        "HI": "Hispanic",
        "AM": "American Indian/Alaska Native",
        "AS": "Asian",
        "HP": "Native Hawaiian/Pacific Islander",
        "BL": "Black",
        "WH": "White",
        "TR": "Two or More Races",
        "LEP": "Limited English Proficiency",
        "IDEA": "Student With Disability (IDEA)"
    }

    transformed_data = []
    for _, row in df.iterrows():
        enrollments = {}
        total_students = {}  # Initialize without default values

        total_students["Male"] = int(row.get("TOT_MATHENR_ADVM_M", 0))
        total_students["Female"] = int(row.get("TOT_MATHENR_ADVM_F", 0))
        total_students["Total"] = total_students["Male"] + total_students["Female"]

        for key, value in row.items():
            if 'MATHENR_ADVM' not in key or value in (-9, '-9', '') or 'TOT_' in key:
                continue  # Skip columns not related to advanced math enrollments or invalid data

            parts = key.split("_")
            if len(parts) < 5:  # Ensure there are enough parts to get race code and gender
                continue

            race_code = parts[3]
            race_name = race_categories.get(race_code, "Unknown")
            gender = "Male" if parts[-1] == "M" else "Female"
            increment = int(value)

            # Skip if value is -9 or increment is 0
            if value == -9 or increment == 0:
                continue

            subgroup = f"{race_name}"
            if subgroup not in enrollments:
                enrollments[subgroup] = {"Male": 0, "Female": 0, "Total": 0}

            enrollments[subgroup][gender] = increment
            # Total per subgroup is calculated as the sum of Male and Female to ensure accuracy
            enrollments[subgroup]["Total"] = enrollments[subgroup]["Male"] + enrollments[subgroup]["Female"]

        if total_students["Total"] > 0:  # Ensure there's at least one student before adding to transformed data
            # Adding "Total Students" subgroup with calculated totals
            enrollments["Total Students"] = total_students

            school_data = {
                "institution_id": row["COMBOKEY"],
                "institution_name": row["SCH_NAME"],
                "jurisdiction_name": row["LEA_STATE_NAME"],
                "lea_id": row["LEAID"],
                "lea_name": row["LEA_NAME"],
                "year": year,
                "enrollments": enrollments
            }
            # Calculating course certification ratio if data is valid
            math_classes = row.get("SCH_MATHCLASSES_ADVM", 0)
            math_cert = row.get("SCH_MATHCERT_ADVM", 0)
            if math_classes > 0 and math_cert >= 0:
                school_data["course_certification"] = math_cert / math_classes if math_classes else None

            transformed_data.append(school_data)

    return transformed_data

#transformed_advanced_math_data = transform_advanced_math_data(advanced_math_df, 2020)

advanced_math_df_subset = advanced_math_df.head(200)

# Now, use this subset to transform the data
transformed_advanced_math_data = transform_advanced_math_data(advanced_math_df_subset, 2020)

# Optionally, print or inspect part of the transformed data to check its structure
print(transformed_advanced_math_data[:5])  # Print details of the first 5 schools for inspection


      LEA_STATE LEA_STATE_NAME    LEAID                           LEA_NAME  \
45545        MN      Minnesota  2700460  Metro Tech Academy Charter School   
45546        MN      Minnesota  2700464      Innovation Sci & Tech Academy   

       SCHID                       SCH_NAME      COMBOKEY  JJ  \
45545   5321  Metro Tech Academy Elementary  270046005321  No   
45546   5322  Innovation Sci & Tech Academy  270046405322  No   

       SCH_MATHCLASSES_ADVM  SCH_MATHCERT_ADVM  ...  SCH_MATHENR_ADVM_WH_M  \
45545                    -9                 -9  ...                     -9   
45546                    -9                 -9  ...                     -9   

       SCH_MATHENR_ADVM_WH_F  SCH_MATHENR_ADVM_TR_M  SCH_MATHENR_ADVM_TR_F  \
45545                     -9                     -9                     -9   
45546                     -9                     -9                     -9   

       TOT_MATHENR_ADVM_M  TOT_MATHENR_ADVM_F  SCH_MATHENR_ADVM_LEP_M  \
45545                  -9 

### Load

In [ ]:
from neo4j import GraphDatabase

def insert_advanced_math_data(session, data):
    cypher_query = """
    UNWIND $data AS schoolData
    // Create or find Jurisdiction
    MERGE (jurisdiction:Jurisdiction {name: schoolData.jurisdiction_name})
    // Create or find LEA
    MERGE (lea:LEA {ncesID: schoolData.lea_id, name: schoolData.lea_name})
    MERGE (jurisdiction)-[:INCLUDES]->(lea)
    // Create or find Institution
    MERGE (institution:Institution {ncesID: schoolData.institution_id})
        ON CREATE SET institution.name = schoolData.institution_name
    MERGE (lea)-[:INCLUDES]->(institution)
    MERGE (course:Course {name: "Advanced Mathematics", date: schoolData.year})

    WITH institution, course, schoolData
    UNWIND keys(schoolData.enrollments) AS subgroupName
    WITH institution, course, schoolData, subgroupName, schoolData.enrollments[subgroupName] AS enrollmentData
    MERGE (institution)-[offers:OFFERS {date: schoolData.year, subgroup: subgroupName}]->(course)
        ON CREATE SET offers.male = enrollmentData.Male,
                      offers.female = enrollmentData.Female,
                      offers.total = enrollmentData.Total
    WITH institution, offers, schoolData, CASE WHEN subgroupName = 'Total Students' THEN schoolData.course_certification ELSE NULL END AS certificationRate
    SET offers.certificationRate = certificationRate
    """
    session.run(cypher_query, {"data": data})


driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Here we take a subset for demonstration
test_data_advanced_math = transformed_advanced_math_data[:10]

# Execute the insertion for the advanced math data
with driver.session() as session:
    insert_advanced_math_data(session, test_data_advanced_math)

driver.close()

## CRDC Algebra 1

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
algebrai_path = './Algebra I.csv'  # Update with the actual file path
algebrai_df = pd.read_csv(algebrai_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Algebra I Preview:")
print(algebrai_df.head(), "\n")

Algebra I Preview:
  LEA_STATE LEA_STATE_NAME    LEAID                LEA_NAME  SCHID  \
0        AL        Alabama  0100002  Alabama Youth Services   1705   
1        AL        Alabama  0100002  Alabama Youth Services   1706   
2        AL        Alabama  0100002  Alabama Youth Services  99995   
3        AL        Alabama  0100005        Albertville City    870   
4        AL        Alabama  0100005        Albertville City    871   

                        SCH_NAME      COMBOKEY   JJ  SCH_ALGCLASSES_GS0708  \
0  Wallace Sch - Mt Meigs Campus  010000201705  Yes                      0   
1      Mcneel Sch - Vacca Campus  010000201706  Yes                      0   
2                 Autauga Campus  010000299995  Yes                      6   
3      Albertville Middle School  010000500870   No                      6   
4        Albertville High School  010000500871   No                     -9   

   SCH_ALGCERT_GS0708  ... SCH_ALGPASS_GS1112_WH_M  SCH_ALGPASS_GS1112_WH_F  \
0           

### Transform

In [ ]:
from collections import defaultdict
import pandas as pd

def transform_algebrai_data(df, year):
    transformed_data = []
    race_categories = {
        "HI": "Hispanic",
        "AM": "American Indian/Alaska Native",
        "AS": "Asian",
        "HP": "Native Hawaiian/Pacific Islander",
        "BL": "Black",
        "WH": "White",
        "TR": "Two or More Races",
        "LEP": "Limited English Proficiency",
        "IDEA": "Student with Disability (IDEA)"
    }

    grade_band_mapping = {
        "G07": "7",
        "G08": "8",
        "GS0910": "9-10",
        "GS1112": "11-12"
    }

    transformed_data = []
    for _, row in df.iterrows():
        grade_band_enrollments = {}
        for key, value in row.items():
            if 'ALGENR' in key or 'ALGPASS' in key:
                if 'IND' in key or value in ['-9', '', None]:
                    continue  # Exclude indicators and non-numeric values

                parts = key.split('_')
                grade_band_code = parts[2]
                grade_band = grade_band_mapping.get(grade_band_code, None)
                race_code = parts[-2]
                race = race_categories.get(race_code, "Total")
                gender = "Male" if "M" in parts[-1] else "Female" if "F" in parts[-1] else "Total"

                enrollment_value = int(value) if value not in ['-9', ''] else 0

                if grade_band not in grade_band_enrollments:
                    grade_band_enrollments[grade_band] = {}
                if race not in grade_band_enrollments[grade_band]:
                    grade_band_enrollments[grade_band][race] = {"Male": {"enroll": 0, "pass": 0}, "Female": {"enroll": 0, "pass": 0}, "Total": {"enroll": 0, "pass": 0}}

                if 'ALGENR' in key and enrollment_value > 0:
                    grade_band_enrollments[grade_band][race][gender]['enroll'] += enrollment_value
                    grade_band_enrollments[grade_band][race]["Total"]['enroll'] += enrollment_value
                elif 'ALGPASS' in key and enrollment_value > 0:
                    if grade_band_enrollments[grade_band][race][gender]['enroll'] > 0:
                        grade_band_enrollments[grade_band][race][gender]['pass'] = round(enrollment_value / grade_band_enrollments[grade_band][race][gender]['enroll'], 2)

        for grade_band, races in grade_band_enrollments.items():
            for race, data in races.items():
                total_enrolled = data['Male']['enroll'] + data['Female']['enroll']
                if total_enrolled > 0:
                    # Calculate the total number of students who passed using the individual pass rates
                    male_pass = (data['Male']['pass'] * data['Male']['enroll']) if data['Male']['enroll'] > 0 else 0
                    female_pass = (data['Female']['pass'] * data['Female']['enroll']) if data['Female']['enroll'] > 0 else 0
                    total_pass = male_pass + female_pass

                    # Calculate the overall pass rate
                    data['Total']['pass'] = round(total_pass / total_enrolled, 2)

                    # Ensure the total enrollments are accurately reflected
                    data['Total']['enroll'] = total_enrolled
                else:
                    # If there are no enrollments, set the pass rate to 0
                    data['Total']['pass'] = 0

        # Corrected filtering step
        for grade_band, races in list(grade_band_enrollments.items()):
            for race, data in list(races.items()):
                for gender, stats in list(data.items()):
                    if stats['enroll'] == 0:
                        del data[gender]
                if not data:  # If the race dictionary is empty after deletion
                    del grade_band_enrollments[grade_band][race]
            if not grade_band_enrollments[grade_band]:  # If the grade band is empty after deletion
                del grade_band_enrollments[grade_band]

        if grade_band_enrollments:  # Append only if there's data
            transformed_data.append({
                "institution_id": row["COMBOKEY"],
                "institution_name": row["SCH_NAME"],
                "jurisdiction_name": row["LEA_STATE_NAME"],
                "lea_id": row["LEAID"],
                "lea_name": row["LEA_NAME"],
                "year": year,
                "grade_band_enrollments": grade_band_enrollments
            })

    return transformed_data



# Assuming algebrai_df is your DataFrame for Algebra I data and the transform function is named `transform_algebra_data`
algebrai_df_subset = algebrai_df #.head(200)

# Now, use this subset to transform the data for Algebra I
transformed_algebrai_data = transform_algebrai_data(algebrai_df_subset, 2020)

# Optionally, print or inspect part of the transformed data to check its structure
print(transformed_algebrai_data[:5])  # Print details of the first 5 schools for inspection

[{'institution_id': '010000201705', 'institution_name': 'Wallace Sch - Mt Meigs Campus', 'jurisdiction_name': 'Alabama', 'lea_id': '0100002', 'lea_name': 'Alabama Youth Services', 'year': 2020, 'grade_band_enrollments': {'9-10': {'Hispanic': {'Male': {'enroll': 1, 'pass': 1.0}, 'Total': {'enroll': 1, 'pass': 1.0}}, 'Black': {'Male': {'enroll': 10, 'pass': 1.0}, 'Total': {'enroll': 10, 'pass': 1.0}}, 'White': {'Male': {'enroll': 15, 'pass': 1.0}, 'Total': {'enroll': 15, 'pass': 1.0}}, 'Total': {'Male': {'enroll': 26, 'pass': 1.0}, 'Total': {'enroll': 26, 'pass': 1.0}}}, '11-12': {'White': {'Male': {'enroll': 1, 'pass': 1.0}, 'Total': {'enroll': 1, 'pass': 1.0}}, 'Total': {'Male': {'enroll': 1, 'pass': 1.0}, 'Total': {'enroll': 1, 'pass': 1.0}}}}}, {'institution_id': '010000299995', 'institution_name': 'Autauga Campus', 'jurisdiction_name': 'Alabama', 'lea_id': '0100002', 'lea_name': 'Alabama Youth Services', 'year': 2020, 'grade_band_enrollments': {'9-10': {'Hispanic': {'Male': {'enroll

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

def insert_test_data(session, data):
    cypher_query = """
    UNWIND $data as schoolData

    // Create or update Jurisdiction and LEA nodes, and their relationship
    MERGE (jurisdiction:Jurisdiction {name: schoolData.jurisdiction_name})
    MERGE (lea:LEA {ncesID: schoolData.lea_id, name: schoolData.lea_name})
        ON CREATE SET lea.name = schoolData.lea_name
    MERGE (jurisdiction)-[:INCLUDES]->(lea)

    // Ensure the Institution node is present and linked to its LEA
    MERGE (institution:Institution {ncesID: schoolData.institution_id})
        ON CREATE SET institution.name = schoolData.institution_name
    MERGE (lea)-[:INCLUDES]->(institution)

    // Define the Course node for Algebra 1
    MERGE (course:Course {name: "Algebra 1"})

    // Process each grade band's enrollment
    FOREACH (gradeBand IN keys(schoolData.grade_band_enrollments) |
        // Create a unique CourseOffering node for each grade at each institution
        MERGE (courseOffering:CourseOffering {id: schoolData.institution_id + "_" + gradeBand + "_" + toString(schoolData.year)})
        ON CREATE SET courseOffering.name = "Algebra 1, Grade: " + gradeBand, courseOffering.grade = gradeBand, courseOffering.date = schoolData.year

        // Relate the CourseOffering to the Course via IS_INSTANCE relationship
        MERGE (courseOffering)-[:INSTANCE_OF]->(course)

        // Relate the Institution to the CourseOffering via the OFFERS relationship
        MERGE (institution)-[:OFFERS]->(courseOffering)

        // Iterate over each subgroup's enrollment for this gradeBand
        FOREACH (subgroupKey IN keys(schoolData.grade_band_enrollments[gradeBand]) |
            MERGE (sg:Subgroup {name: subgroupKey})

            // Relate Subgroup to CourseOffering with enrollment data
            MERGE (sg)-[enrolled:ENROLLS_IN {date: schoolData.year}]->(courseOffering)
                ON CREATE SET enrolled.male = schoolData.grade_band_enrollments[gradeBand][subgroupKey]["Male"].enroll,
                              enrolled.female = schoolData.grade_band_enrollments[gradeBand][subgroupKey]["Female"].enroll,
                              enrolled.total = schoolData.grade_band_enrollments[gradeBand][subgroupKey]["Total"].enroll

            // Create the Outcome node for passing Algebra 1 and relate it to the CourseOffering
            MERGE (outcome:Outcome {name: "Pass Algebra 1", date: schoolData.year})

            // Create LEADS_TO relationship with pass rates for each subgroup
            MERGE (courseOffering)-[leadsTo:LEADS_TO {subgroup: subgroupKey}]->(outcome)
                ON CREATE SET leadsTo.malePass = schoolData.grade_band_enrollments[gradeBand][subgroupKey]["Male"].pass,
                              leadsTo.femalePass = schoolData.grade_band_enrollments[gradeBand][subgroupKey]["Female"].pass,
                              leadsTo.totalPass = schoolData.grade_band_enrollments[gradeBand][subgroupKey]["Total"].pass
        )
    )
    """
    session.run(cypher_query, {"data": data})


driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Here we take a subset for demonstration
test_data= transformed_algebrai_data

# Execute the insertion for the advanced math data
with driver.session() as session:
  for i in tqdm(range(36000, len(test_data) - 1, 1000)):
    sub_data = test_data [i:i+1000]
    insert_test_data(session, sub_data)

driver.close()

100%|██████████| 2/2 [11:03<00:00, 331.53s/it]


## CRDC Geometry

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
geometry_path = './Geometry.csv'  # Update with the actual file path
geometry_df = pd.read_csv(geometry_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Geometry Preview:")
print(geometry_df.head(), "\n")

Geometry Preview:
  LEA_STATE LEA_STATE_NAME    LEAID                LEA_NAME  SCHID  \
0        AL        Alabama  0100002  Alabama Youth Services   1705   
1        AL        Alabama  0100002  Alabama Youth Services   1706   
2        AL        Alabama  0100002  Alabama Youth Services  99995   
3        AL        Alabama  0100005        Albertville City    870   
4        AL        Alabama  0100005        Albertville City    871   

                        SCH_NAME      COMBOKEY   JJ SCH_GEOMENR_G08_IND  \
0  Wallace Sch - Mt Meigs Campus  010000201705  Yes                  No   
1      Mcneel Sch - Vacca Campus  010000201706  Yes                  No   
2                 Autauga Campus  010000299995  Yes                  No   
3      Albertville Middle School  010000500870   No                  No   
4        Albertville High School  010000500871   No                  -9   

   SCH_GEOMENR_G08  ...  SCH_MATHENR_GEOM_WH_M  SCH_MATHENR_GEOM_WH_F  \
0               -9  ...              

### Transform

In [ ]:
from collections import defaultdict
import pandas as pd

def transform_geometry_data(df, year, nces_ids):

    df_filtered = df[df['COMBOKEY'].isin(nces_ids)]
    print(df_filtered)

    race_categories = {
        "HI": "Hispanic",
        "AM": "American Indian/Alaska Native",
        "AS": "Asian",
        "HP": "Native Hawaiian/Pacific Islander",
        "BL": "Black",
        "WH": "White",
        "TR": "Two or More Races",
        "LEP": "Limited English Proficiency",
        "IDEA": "Student With Disability (IDEA)"
    }

    transformed_data = []
    for _, row in df_filtered.iterrows():
        enrollments = {}
        total_students = {}  # Initialize without default values

        # Process total enrollments first
        total_students["Male"] = int(row.get("TOT_MATHENR_GEOM_M", 0))
        total_students["Female"] = int(row.get("TOT_MATHENR_GEOM_F", 0))
        total_students["Total"] = total_students["Male"] + total_students["Female"]

        for key, value in row.items():
            # Skip indicators, non-numeric values, and keys not related to Geometry enrollments
            if 'GEOM' not in key or value in (-9, '-9', '') or 'IND' in key or 'TOT_' in key:
                continue

            parts = key.split("_")
            # Ensure there are enough parts to get race code and gender
            if len(parts) < 4:
                continue

            race_code = parts[3]  # Adjusted for Geometry's column naming
            race_name = race_categories.get(race_code, "Unknown")
            gender = "Male" if parts[-1] == "M" else "Female"
            increment = int(value) if value not in ['-9', ''] else 0

            if increment == 0:
                continue

            subgroup = f"{race_name}"
            if subgroup not in enrollments:
                enrollments[subgroup] = {"Male": 0, "Female": 0, "Total": 0}

            # Instead of incrementing, directly assign the total values for Male and Female
            enrollments[subgroup][gender] = increment
            # Total per subgroup is calculated as the sum of Male and Female to ensure accuracy
            enrollments[subgroup]["Total"] = enrollments[subgroup]["Male"] + enrollments[subgroup]["Female"]

        if total_students["Total"] > 0:  # Ensure there's at least one student before adding to transformed data
            # Adding "Total Students" subgroup with calculated totals
            enrollments["Total Students"] = total_students

            # Package the school data, including course certification if applicable
            school_data = {
                "institution_id": row["COMBOKEY"],
                "institution_name": row["SCH_NAME"],
                "jurisdiction_name": row["LEA_STATE_NAME"],
                "lea_id": row["LEAID"],
                "lea_name": row["LEA_NAME"],
                "year": year,
                "enrollments": enrollments,
                "course_certification": row.get("SCH_MATHCERT_GEOM", 0) / row.get("SCH_MATHCLASSES_GEOM", 1) if row.get("SCH_MATHCLASSES_GEOM", 1) > 0 else None
            }

            transformed_data.append(school_data)

    return transformed_data

geometry_df_subset = geometry_df.head(200)

# Now, use this subset to transform the data for Geometry
transformed_geometry_data = transform_geometry_data(geometry_df, 2020, nces_id_list)

# Optionally, print or inspect part of the transformed data to check its structure
print(transformed_geometry_data[:5])  # Print details of the first 5 schools for inspection

      LEA_STATE LEA_STATE_NAME    LEAID                           LEA_NAME  \
45545        MN      Minnesota  2700460  Metro Tech Academy Charter School   
45546        MN      Minnesota  2700464      Innovation Sci & Tech Academy   

       SCHID                       SCH_NAME      COMBOKEY  JJ  \
45545   5321  Metro Tech Academy Elementary  270046005321  No   
45546   5322  Innovation Sci & Tech Academy  270046405322  No   

      SCH_GEOMENR_G08_IND  SCH_GEOMENR_G08  ...  SCH_MATHENR_GEOM_WH_M  \
45545                  -9               -9  ...                     -9   
45546                  -9               -9  ...                     -9   

       SCH_MATHENR_GEOM_WH_F  SCH_MATHENR_GEOM_TR_M  SCH_MATHENR_GEOM_TR_F  \
45545                     -9                     -9                     -9   
45546                     -9                     -9                     -9   

       TOT_MATHENR_GEOM_M  TOT_MATHENR_GEOM_F  SCH_MATHENR_GEOM_LEP_M  \
45545                  -9             

### Load

In [ ]:
from neo4j import GraphDatabase

def insert_geometry_data(session, data):
    cypher_query = """
    UNWIND $data AS schoolData
    // Create or find Jurisdiction
    MERGE (jurisdiction:Jurisdiction {name: schoolData.jurisdiction_name})
    // Create or find LEA
    MERGE (lea:LEA {ncesID: schoolData.lea_id, name: schoolData.lea_name})
    MERGE (jurisdiction)-[:INCLUDES]->(lea)
    // Create or find Institution
    MERGE (institution:Institution {ncesID: schoolData.institution_id})
        ON CREATE SET institution.name = schoolData.institution_name
    MERGE (lea)-[:INCLUDES]->(institution)

    MERGE (course:Course {name: "Geometry", date: schoolData.year})
    WITH institution, course, schoolData
    UNWIND keys(schoolData.enrollments) AS subgroupName
    WITH institution, course, schoolData, subgroupName, schoolData.enrollments[subgroupName] AS enrollmentData
    MERGE (institution)-[offers:OFFERS {date: schoolData.year, subgroup: subgroupName}]->(course)
        ON CREATE SET offers.male = enrollmentData.Male,
                      offers.female = enrollmentData.Female,
                      offers.total = enrollmentData.Total
    WITH institution, offers, schoolData, CASE WHEN subgroupName = 'Total Students' THEN schoolData.course_certification ELSE NULL END AS certificationRate
    SET offers.certificationRate = certificationRate

    """
    session.run(cypher_query, {"data": data})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Assuming your transformed geometry data is stored in a variable named transformed_geometry_data
# Here we take a subset for demonstration
test_data_geometry = transformed_geometry_data[:10]  # Replace with your actual transformed geometry data

# Execute the insertion for the geometry data
with driver.session() as session:
    insert_geometry_data(session, test_data_geometry)

driver.close()


## CRDC Algebra 2

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
algebraii_path = './Algebra II.csv'  # Update with the actual file path
algebraii_df = pd.read_csv(algebraii_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Algebra II Preview:")
print(algebraii_df.head(), "\n")

Algebra II Preview:
  LEA_STATE LEA_STATE_NAME    LEAID                LEA_NAME  SCHID  \
0        AL        Alabama  0100002  Alabama Youth Services   1705   
1        AL        Alabama  0100002  Alabama Youth Services   1706   
2        AL        Alabama  0100002  Alabama Youth Services  99995   
3        AL        Alabama  0100005        Albertville City    870   
4        AL        Alabama  0100005        Albertville City    871   

                        SCH_NAME      COMBOKEY   JJ  SCH_MATHCLASSES_ALG2  \
0  Wallace Sch - Mt Meigs Campus  010000201705  Yes                     4   
1      Mcneel Sch - Vacca Campus  010000201706  Yes                     0   
2                 Autauga Campus  010000299995  Yes                     0   
3      Albertville Middle School  010000500870   No                    -9   
4        Albertville High School  010000500871   No                     9   

   SCH_MATHCERT_ALG2  ...  SCH_MATHENR_ALG2_WH_M  SCH_MATHENR_ALG2_WH_F  \
0                  4 

### Transform

In [ ]:
from collections import defaultdict

def transform_algebraii_data(df, year):
    race_categories = {
        "HI": "Hispanic",
        "AM": "American Indian/Alaska Native",
        "AS": "Asian",
        "HP": "Native Hawaiian/Pacific Islander",
        "BL": "Black",
        "WH": "White",
        "TR": "Two or More Races",
        "LEP": "Limited English Proficiency",
        "IDEA": "Student With Disability (IDEA)"
    }

    transformed_data = []
    for _, row in df.iterrows():
        enrollments = {}
        total_students = {}

        # Process total enrollments first
        total_students["Male"] = int(row.get("TOT_MATHENR_ALG2_M", 0))
        total_students["Female"] = int(row.get("TOT_MATHENR_ALG2_F", 0))
        total_students["Total"] = total_students["Male"] + total_students["Female"]

        for key, value in row.items():
            if 'ALG2' not in key or value in (-9, '-9', '') or 'IND' in key or 'TOT_' in key:
                continue

            parts = key.split("_")
            if len(parts) < 4:
                continue

            race_code = parts[3]  # Adjusted for Algebra II's column naming
            race_name = race_categories.get(race_code, "Unknown")
            gender = "Male" if "M" in parts[-1] else "Female"
            increment = int(value) if value not in ['-9', ''] else 0

            if increment == 0:
                continue

            subgroup = f"{race_name}"
            if subgroup not in enrollments:
                enrollments[subgroup] = {"Male": 0, "Female": 0, "Total": 0}

            # Instead of incrementing, directly assign the total values for Male and Female
            enrollments[subgroup][gender] = increment
            # Total per subgroup is calculated as the sum of Male and Female to ensure accuracy
            enrollments[subgroup]["Total"] = enrollments[subgroup]["Male"] + enrollments[subgroup]["Female"]

        if total_students["Total"] > 0:  # Ensure there's at least one student before adding to transformed data
            # Adding "Total Students" subgroup with calculated totals
            enrollments["Total Students"] = total_students

            school_data = {
                "institution_id": row["COMBOKEY"],
                "institution_name": row["SCH_NAME"],
                "jurisdiction_name": row["LEA_STATE_NAME"],
                "lea_id": row["LEAID"],
                "lea_name": row["LEA_NAME"],
                "year": year,
                "enrollments": enrollments,
                "course_certification": row.get("SCH_MATHCERT_ALG2", 0) / row.get("SCH_MATHCLASSES_ALG2", 1) if row.get("SCH_MATHCLASSES_ALG2", 1) > 0 else None
            }

            transformed_data.append(school_data)

    return transformed_data

# Assuming algebra2_df is your DataFrame containing the Algebra II data
algebraii_df_subset = algebraii_df.head(200)
transformed_algebraii_data = transform_algebraii_data(algebraii_df_subset, 2020)
print(transformed_algebraii_data[:5])

[{'institution_id': 10000201705, 'institution_name': 'Wallace Sch - Mt Meigs Campus', 'jurisdiction_name': 'Alabama', 'lea_id': 100002, 'lea_name': 'Alabama Youth Services', 'year': 2020, 'enrollments': {'Black': {'Male': 2, 'Female': 0, 'Total': 2}, 'White': {'Male': 3, 'Female': 0, 'Total': 3}, 'Total Students': {'Male': 5, 'Female': 0, 'Total': 5}}, 'course_certification': 1.0}, {'institution_id': 10000500871, 'institution_name': 'Albertville High School', 'jurisdiction_name': 'Alabama', 'lea_id': 100005, 'lea_name': 'Albertville City', 'year': 2020, 'enrollments': {'Hispanic': {'Male': 41, 'Female': 53, 'Total': 94}, 'American Indian/Alaska Native': {'Male': 0, 'Female': 1, 'Total': 1}, 'Asian': {'Male': 1, 'Female': 2, 'Total': 3}, 'Black': {'Male': 3, 'Female': 7, 'Total': 10}, 'White': {'Male': 55, 'Female': 76, 'Total': 131}, 'Two or More Races': {'Male': 0, 'Female': 1, 'Total': 1}, 'Limited English Proficiency': {'Male': 5, 'Female': 4, 'Total': 9}, 'Student With Disability (

### Load

In [ ]:
from neo4j import GraphDatabase

def insert_algebraii_data(session, data):
    cypher_query = """
    UNWIND $data AS schoolData
    // Create or find Jurisdiction
    MERGE (jurisdiction:Jurisdiction {name: schoolData.jurisdiction_name})
    // Create or find LEA
    MERGE (lea:LEA {ncesID: schoolData.lea_id, name: schoolData.lea_name})
    MERGE (jurisdiction)-[:INCLUDES]->(lea)
    // Create or find Institution
    MERGE (institution:Institution {ncesID: schoolData.institution_id})
        ON CREATE SET institution.name = schoolData.institution_name
    MERGE (lea)-[:INCLUDES]->(institution)

    // Adjust the course name to "Algebra 2" and ensure the year is set correctly
    MERGE (course:Course {name: "Algebra 2", date: schoolData.year})
    WITH institution, course, schoolData
    UNWIND keys(schoolData.enrollments) AS subgroupName
    WITH institution, course, schoolData, subgroupName, schoolData.enrollments[subgroupName] AS enrollmentData
    MERGE (institution)-[offers:OFFERS {date: schoolData.year, subgroup: subgroupName}]->(course)
        ON CREATE SET offers.male = enrollmentData.Male,
                      offers.female = enrollmentData.Female,
                      offers.total = enrollmentData.Total

    // Handling course certification rate for the "Total Students" subgroup
    WITH institution, offers, schoolData, CASE WHEN subgroupName = 'Total Students' THEN schoolData.course_certification ELSE NULL END AS certificationRate
    SET offers.certificationRate = certificationRate
    """
    session.run(cypher_query, {"data": data})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Assuming your transformed geometry data is stored in a variable named transformed_geometry_data
# Here we take a subset for demonstration
test_data_algebraii = transformed_algebraii_data[:10]  # Replace with your actual transformed geometry data

# Execute the insertion for the geometry data
with driver.session() as session:
    insert_algebraii_data(session, test_data_algebraii)

driver.close()

## CRDC Calculus

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
calculus_path = './Calculus.csv'  # Update with the actual file path
calculus_df = pd.read_csv(calculus_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Calculus Preview:")
print(calculus_df.head(), "\n")

Calculus Preview:
  LEA_STATE LEA_STATE_NAME    LEAID                LEA_NAME  SCHID  \
0        AL        Alabama  0100002  Alabama Youth Services   1705   
1        AL        Alabama  0100002  Alabama Youth Services   1706   
2        AL        Alabama  0100002  Alabama Youth Services  99995   
3        AL        Alabama  0100005        Albertville City    870   
4        AL        Alabama  0100005        Albertville City    871   

                        SCH_NAME      COMBOKEY   JJ  SCH_MATHCLASSES_CALC  \
0  Wallace Sch - Mt Meigs Campus  010000201705  Yes                     0   
1      Mcneel Sch - Vacca Campus  010000201706  Yes                     0   
2                 Autauga Campus  010000299995  Yes                     0   
3      Albertville Middle School  010000500870   No                    -9   
4        Albertville High School  010000500871   No                     3   

   SCH_MATHCERT_CALC  ...  SCH_MATHENR_CALC_WH_M  SCH_MATHENR_CALC_WH_F  \
0                 -9  .

### Transform

In [ ]:
from collections import defaultdict

def transform_calculus_data(df, year):
    race_categories = {
        "HI": "Hispanic",
        "AM": "American Indian/Alaska Native",
        "AS": "Asian",
        "HP": "Native Hawaiian/Pacific Islander",
        "BL": "Black",
        "WH": "White",
        "TR": "Two or More Races",
        "LEP": "Limited English Proficiency",
        "IDEA": "Student With Disability (IDEA)"
    }

    transformed_data = []
    for _, row in df.iterrows():
        enrollments = {}
        total_students = {}  # Initialize without default values

        for key, value in row.items():
            # Skip indicators, non-numeric values, and keys not related to Calculus enrollments
            if 'CALC' not in key or value in (-9, '-9', '') or 'IND' in key:
                continue

            parts = key.split("_")
            # Ensure there are enough parts to get race code and gender
            if len(parts) < 4:
                continue

            race_code = parts[3]  # Adjusted for Calculus's column naming
            race_name = race_categories.get(race_code, "Unknown")
            gender = "Male" if parts[-1] == "M" else "Female"
            increment = int(value) if value not in ['-9', ''] else 0

            if increment == 0:
                continue

            subgroup = f"{race_name}"
            if subgroup not in enrollments:
                enrollments[subgroup] = {"Male": 0, "Female": 0, "Total": 0}

            enrollments[subgroup][gender] += increment
            enrollments[subgroup]["Total"] += increment
            total_students[gender] = total_students.get(gender, 0) + increment
            total_students["Total"] = total_students.get("Total", 0) + increment

        if total_students:
            # Filter out subgroups with zero total enrollments
            enrollments = {k: v for k, v in enrollments.items() if v["Total"] > 0}

            # Add the "Total Students" category if there are total students
            if total_students["Total"] > 0:
                enrollments["Total Students"] = total_students

            # Package the school data, including course certification if applicable
            school_data = {
                "institution_id": row["COMBOKEY"],
                "institution_name": row["SCH_NAME"],
                "jurisdiction_name": row["LEA_STATE_NAME"],
                "lea_id": row["LEAID"],
                "lea_name": row["LEA_NAME"],
                "year": year,
                "enrollments": enrollments,
                "course_certification": row.get("SCH_MATHCERT_CALC", 0) / row.get("SCH_MATHCLASSES_CALC", 1) if row.get("SCH_MATHCLASSES_CALC", 1) > 0 else None
            }

            transformed_data.append(school_data)

    return transformed_data

calculus_df_subset = calculus_df #.head(200)
transformed_calculus_data = transform_calculus_data(calculus_df_subset, 2020)
print(transformed_calculus_data[:5])

[{'institution_id': 10000500871, 'institution_name': 'Albertville High School', 'jurisdiction_name': 'Alabama', 'lea_id': 100005, 'lea_name': 'Albertville City', 'year': 2020, 'enrollments': {'Hispanic': {'Male': 5, 'Female': 5, 'Total': 10}, 'White': {'Male': 10, 'Female': 12, 'Total': 22}, 'Two or More Races': {'Male': 1, 'Female': 0, 'Total': 1}, 'Unknown': {'Male': 16, 'Female': 17, 'Total': 33}, 'Student With Disability (IDEA)': {'Male': 1, 'Female': 0, 'Total': 1}, 'Total Students': {'Male': 33, 'Total': 67, 'Female': 34}}, 'course_certification': 1.0}, {'institution_id': 10000600878, 'institution_name': 'Douglas High School', 'jurisdiction_name': 'Alabama', 'lea_id': 100006, 'lea_name': 'Marshall County', 'year': 2020, 'enrollments': {'Hispanic': {'Male': 4, 'Female': 0, 'Total': 4}, 'American Indian/Alaska Native': {'Male': 0, 'Female': 1, 'Total': 1}, 'Asian': {'Male': 0, 'Female': 1, 'Total': 1}, 'White': {'Male': 4, 'Female': 4, 'Total': 8}, 'Unknown': {'Male': 8, 'Female': 

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

def insert_test_data(session, data):
    cypher_query = """
    UNWIND $data AS schoolData
    // Create or find Jurisdiction
    MERGE (jurisdiction:Jurisdiction {name: schoolData.jurisdiction_name})
    // Create or find LEA
    MERGE (lea:LEA {ncesID: schoolData.lea_id, name: schoolData.lea_name})
    MERGE (jurisdiction)-[:INCLUDES]->(lea)
    // Create or find Institution
    MERGE (institution:Institution {ncesID: schoolData.institution_id})
        ON CREATE SET institution.name = schoolData.institution_name
    MERGE (lea)-[:INCLUDES]->(institution)

    // Adjust the course name to "Calculus" and ensure the year is set correctly
    MERGE (course:Course {name: "Calculus", date: schoolData.year})
    WITH institution, course, schoolData
    UNWIND keys(schoolData.enrollments) AS subgroupName
    WITH institution, course, schoolData, subgroupName, schoolData.enrollments[subgroupName] AS enrollmentData
    MERGE (institution)-[offers:OFFERS {date: schoolData.year, subgroup: subgroupName}]->(course)
        ON CREATE SET offers.male = enrollmentData.Male,
                      offers.female = enrollmentData.Female,
                      offers.total = enrollmentData.Total

    // Handling course certification rate for the "Total Students" subgroup
    WITH institution, offers, schoolData, CASE WHEN subgroupName = 'Total Students' THEN schoolData.course_certification ELSE NULL END AS certificationRate
    SET offers.certificationRate = certificationRate
    """
    session.run(cypher_query, {"data": data})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Assuming your transformed geometry data is stored in a variable named transformed_geometry_data

test_data = transformed_calculus_data

# Execute the insertion for the geometry data
with driver.session() as session:
  for i in tqdm(range(8000, len(test_data) - 1, 1000)):
    sub_data = test_data [i:i+1000]
    insert_test_data(session, sub_data)

driver.close()

100%|██████████| 5/5 [16:12<00:00, 194.55s/it]


## CRDC Biology

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
biology_path = './Biology.csv'  # Update with the actual file path
biology_df = pd.read_csv(biology_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Biology Preview:")
print(biology_df.head(), "\n")

Biology Preview:
  LEA_STATE LEA_STATE_NAME    LEAID                LEA_NAME  SCHID  \
0        AL        Alabama  0100002  Alabama Youth Services   1705   
1        AL        Alabama  0100002  Alabama Youth Services   1706   
2        AL        Alabama  0100002  Alabama Youth Services  99995   
3        AL        Alabama  0100005        Albertville City    870   
4        AL        Alabama  0100005        Albertville City    871   

                        SCH_NAME      COMBOKEY   JJ  SCH_SCICLASSES_BIOL  \
0  Wallace Sch - Mt Meigs Campus  010000201705  Yes                    5   
1      Mcneel Sch - Vacca Campus  010000201706  Yes                    0   
2                 Autauga Campus  010000299995  Yes                    6   
3      Albertville Middle School  010000500870   No                   -9   
4        Albertville High School  010000500871   No                   16   

   SCH_SCICCERT_BIOL  ...  SCH_SCIENR_BIOL_WH_M  SCH_SCIENR_BIOL_WH_F  \
0                  5  ...       

### Transform

In [ ]:
from collections import defaultdict

def transform_biology_data(df, year):
    race_categories = {
        "HI": "Hispanic",
        "AM": "American Indian/Alaska Native",
        "AS": "Asian",
        "HP": "Native Hawaiian/Pacific Islander",
        "BL": "Black",
        "WH": "White",
        "TR": "Two or More Races",
        "LEP": "Limited English Proficiency",
        "IDEA": "Student With Disability (IDEA)"
    }

    transformed_data = []
    for _, row in df.iterrows():
        enrollments = {}
        total_students = {}  # Initialize without default values

        # Process total enrollments first
        total_students["Male"] = int(row.get("TOT_SCIENR_BIOL_M", 0))
        total_students["Female"] = int(row.get("TOT_SCIENR_BIOL_F", 0))
        total_students["Total"] = total_students["Male"] + total_students["Female"]

        for key, value in row.items():
            # Skip indicators, non-numeric values, and keys not related to Biology enrollments
            if 'BIOL' not in key or value in (-9, '-9', '') or 'IND' in key or 'TOT_' in key:
                continue

            parts = key.split("_")
            # Ensure there are enough parts to get race code and gender
            if len(parts) < 4:
                continue

            race_code = parts[3]  # Adjusted for Biology's column naming
            race_name = race_categories.get(race_code, "Unknown")
            gender = "Male" if parts[-1] == "M" else "Female"
            increment = int(value) if value not in ['-9', ''] else 0

            if increment == 0:
                continue

            subgroup = f"{race_name}"
            if subgroup not in enrollments:
                enrollments[subgroup] = {"Male": 0, "Female": 0, "Total": 0}

            # Instead of incrementing, directly assign the total values for Male and Female
            enrollments[subgroup][gender] = increment
            # Total per subgroup is calculated as the sum of Male and Female to ensure accuracy
            enrollments[subgroup]["Total"] = enrollments[subgroup]["Male"] + enrollments[subgroup]["Female"]

        if total_students["Total"] > 0:  # Ensure there's at least one student before adding to transformed data
            # Adding "Total Students" subgroup with calculated totals
            enrollments["Total Students"] = total_students

            school_data = {
                "institution_id": row["COMBOKEY"],
                "institution_name": row["SCH_NAME"],
                "jurisdiction_name": row["LEA_STATE_NAME"],
                "lea_id": row["LEAID"],
                "lea_name": row["LEA_NAME"],
                "year": year,
                "enrollments": enrollments,
                "course_certification": row.get("SCH_SCICCERT_BIOL", 0) / row.get("SCH_SCICLASSES_BIOL", 1) if row.get("SCH_SCICLASSES_BIOL", 1) > 0 else None
            }

            transformed_data.append(school_data)

    return transformed_data

biology_df_subset = biology_df.head(200)
transformed_biology_data = transform_biology_data(biology_df_subset, 2020)
print(transformed_biology_data[:5])

[{'institution_id': 1705, 'institution_name': 'Wallace Sch - Mt Meigs Campus', 'jurisdiction_name': 'Alabama', 'lea_id': 100002, 'lea_name': 'Alabama Youth Services', 'year': 2020, 'enrollments': {'Black': {'Male': 18, 'Female': 0, 'Total': 18}, 'White': {'Male': 7, 'Female': 0, 'Total': 7}, 'Total Students': {'Male': 25, 'Female': 0, 'Total': 25}}, 'course_certification': 1.0}, {'institution_id': 99995, 'institution_name': 'Autauga Campus', 'jurisdiction_name': 'Alabama', 'lea_id': 100002, 'lea_name': 'Alabama Youth Services', 'year': 2020, 'enrollments': {'Hispanic': {'Male': 1, 'Female': 0, 'Total': 1}, 'Black': {'Male': 37, 'Female': 0, 'Total': 37}, 'White': {'Male': 10, 'Female': 0, 'Total': 10}, 'Total Students': {'Male': 48, 'Female': 0, 'Total': 48}}, 'course_certification': 1.0}, {'institution_id': 871, 'institution_name': 'Albertville High School', 'jurisdiction_name': 'Alabama', 'lea_id': 100005, 'lea_name': 'Albertville City', 'year': 2020, 'enrollments': {'Hispanic': {'Ma

### Load

In [ ]:
from neo4j import GraphDatabase

def insert_biology_data(session, data):
    cypher_query = """
    UNWIND $data AS schoolData
    // Create or find Jurisdiction
    MERGE (jurisdiction:Jurisdiction {name: schoolData.jurisdiction_name})
    // Create or find LEA
    MERGE (lea:LEA {ncesID: schoolData.lea_id, name: schoolData.lea_name})
    MERGE (jurisdiction)-[:INCLUDES]->(lea)
    // Create or find Institution
    MERGE (institution:Institution {ncesID: schoolData.institution_id})
        ON CREATE SET institution.name = schoolData.institution_name
    MERGE (lea)-[:INCLUDES]->(institution)

    // Adjust the course name to "Biology" and ensure the year is set correctly
    MERGE (course:Course {name: "Biology", date: schoolData.year})
    WITH institution, course, schoolData
    UNWIND keys(schoolData.enrollments) AS subgroupName
    WITH institution, course, schoolData, subgroupName, schoolData.enrollments[subgroupName] AS enrollmentData
    MERGE (institution)-[offers:OFFERS {date: schoolData.year, subgroup: subgroupName}]->(course)
        ON CREATE SET offers.male = enrollmentData.Male,
                      offers.female = enrollmentData.Female,
                      offers.total = enrollmentData.Total

    // Handling course certification rate for the "Total Students" subgroup
    WITH institution, offers, schoolData, CASE WHEN subgroupName = 'Total Students' THEN schoolData.course_certification ELSE NULL END AS certificationRate
    SET offers.certificationRate = certificationRate
    """
    session.run(cypher_query, {"data": data})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Assuming your transformed geometry data is stored in a variable named transformed_geometry_data
# Here we take a subset for demonstration
test_data_biology = transformed_biology_data[:10]  # Replace with your actual transformed geometry data

# Execute the insertion for the geometry data
with driver.session() as session:
    insert_biology_data(session, test_data_biology)

driver.close()

## CRDC Chemistry

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
chemistry_path = './Chemistry.csv'  # Update with the actual file path
chemistry_df = pd.read_csv(chemistry_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Chemistry Preview:")
print(chemistry_df.head(), "\n")

Chemistry Preview:
  LEA_STATE LEA_STATE_NAME    LEAID                LEA_NAME  SCHID  \
0        AL        Alabama  0100002  Alabama Youth Services   1705   
1        AL        Alabama  0100002  Alabama Youth Services   1706   
2        AL        Alabama  0100002  Alabama Youth Services  99995   
3        AL        Alabama  0100005        Albertville City    870   
4        AL        Alabama  0100005        Albertville City    871   

                        SCH_NAME      COMBOKEY   JJ  SCH_SCICLASSES_CHEM  \
0  Wallace Sch - Mt Meigs Campus  010000201705  Yes                    0   
1      Mcneel Sch - Vacca Campus  010000201706  Yes                    0   
2                 Autauga Campus  010000299995  Yes                    0   
3      Albertville Middle School  010000500870   No                   -9   
4        Albertville High School  010000500871   No                   14   

   SCH_SCICCERT_CHEM  ...  SCH_SCIENR_CHEM_WH_M  SCH_SCIENR_CHEM_WH_F  \
0                 -9  ...     

### Transform

In [ ]:
from collections import defaultdict

def transform_chemistry_data(df, year):
    race_categories = {
        "HI": "Hispanic",
        "AM": "American Indian/Alaska Native",
        "AS": "Asian",
        "HP": "Native Hawaiian/Pacific Islander",
        "BL": "Black",
        "WH": "White",
        "TR": "Two or More Races",
        "LEP": "Limited English Proficiency",
        "IDEA": "Student With Disability (IDEA)"
    }

    transformed_data = []
    for _, row in df.iterrows():
        enrollments = {}
        total_students = {}  # Initialize without default values

        # Process total enrollments first
        total_students["Male"] = int(row.get("TOT_SCIENR_CHEM_M", 0))
        total_students["Female"] = int(row.get("TOT_SCIENR_CHEM_F", 0))
        total_students["Total"] = total_students["Male"] + total_students["Female"]

        for key, value in row.items():
            # Skip indicators, non-numeric values, and keys not related to Chem enrollments
            if 'CHEM' not in key or value in (-9, '-9', '') or 'IND' in key or 'TOT_' in key:
                continue

            parts = key.split("_")
            # Ensure there are enough parts to get race code and gender
            if len(parts) < 4:
                continue

            race_code = parts[3]
            race_name = race_categories.get(race_code, "Unknown")
            gender = "Male" if parts[-1] == "M" else "Female"
            increment = int(value) if value not in ['-9', ''] else 0

            if increment == 0:
                continue

            subgroup = f"{race_name}"
            if subgroup not in enrollments:
                enrollments[subgroup] = {"Male": 0, "Female": 0, "Total": 0}

            # Instead of incrementing, directly assign the total values for Male and Female
            enrollments[subgroup][gender] = increment
            # Total per subgroup is calculated as the sum of Male and Female to ensure accuracy
            enrollments[subgroup]["Total"] = enrollments[subgroup]["Male"] + enrollments[subgroup]["Female"]

        if total_students["Total"] > 0:  # Ensure there's at least one student before adding to transformed data
            # Adding "Total Students" subgroup with calculated totals
            enrollments["Total Students"] = total_students

            school_data = {
                "institution_id": row["COMBOKEY"],
                "institution_name": row["SCH_NAME"],
                "jurisdiction_name": row["LEA_STATE_NAME"],
                "lea_id": row["LEAID"],
                "lea_name": row["LEA_NAME"],
                "year": year,
                "enrollments": enrollments,
                "course_certification": row.get("SCH_SCICCERT_CHEM", 0) / row.get("SCH_SCICLASSES_CHEM", 1) if row.get("SCH_SCICLASSES_CHEM", 1) > 0 else None
            }

            transformed_data.append(school_data)

    return transformed_data

chemistry_df_subset = chemistry_df.head(200)
transformed_chemistry_data = transform_chemistry_data(chemistry_df_subset, 2020)
print(transformed_chemistry_data[:5])

[{'institution_id': 871, 'institution_name': 'Albertville High School', 'jurisdiction_name': 'Alabama', 'lea_id': 100005, 'lea_name': 'Albertville City', 'year': 2020, 'enrollments': {'Hispanic': {'Male': 101, 'Female': 99, 'Total': 200}, 'Asian': {'Male': 2, 'Female': 0, 'Total': 2}, 'Black': {'Male': 3, 'Female': 12, 'Total': 15}, 'White': {'Male': 90, 'Female': 99, 'Total': 189}, 'Two or More Races': {'Male': 1, 'Female': 1, 'Total': 2}, 'Limited English Proficiency': {'Male': 18, 'Female': 13, 'Total': 31}, 'Student With Disability (IDEA)': {'Male': 19, 'Female': 17, 'Total': 36}, 'Total Students': {'Male': 197, 'Female': 211, 'Total': 408}}, 'course_certification': 1.0}, {'institution_id': 872, 'institution_name': 'Asbury High School', 'jurisdiction_name': 'Alabama', 'lea_id': 100006, 'lea_name': 'Marshall County', 'year': 2020, 'enrollments': {'Hispanic': {'Male': 5, 'Female': 4, 'Total': 9}, 'American Indian/Alaska Native': {'Male': 0, 'Female': 1, 'Total': 1}, 'White': {'Male':

### Load

In [ ]:
from neo4j import GraphDatabase

def insert_chemistry_data(session, data):
    cypher_query = """
    UNWIND $data AS schoolData
    // Create or find Jurisdiction
    MERGE (jurisdiction:Jurisdiction {name: schoolData.jurisdiction_name})
    // Create or find LEA
    MERGE (lea:LEA {ncesID: schoolData.lea_id, name: schoolData.lea_name})
    MERGE (jurisdiction)-[:INCLUDES]->(lea)
    // Create or find Institution
    MERGE (institution:Institution {ncesID: schoolData.institution_id})
        ON CREATE SET institution.name = schoolData.institution_name
    MERGE (lea)-[:INCLUDES]->(institution)

    // Adjust the course name to "Chemistry" and ensure the year is set correctly
    MERGE (course:Course {name: "Chemistry", date: schoolData.year})
    WITH institution, course, schoolData
    UNWIND keys(schoolData.enrollments) AS subgroupName
    WITH institution, course, schoolData, subgroupName, schoolData.enrollments[subgroupName] AS enrollmentData
    MERGE (institution)-[offers:OFFERS {date: schoolData.year, subgroup: subgroupName}]->(course)
        ON CREATE SET offers.male = enrollmentData.Male,
                      offers.female = enrollmentData.Female,
                      offers.total = enrollmentData.Total

    // Handling course certification rate for the "Total Students" subgroup
    WITH institution, offers, schoolData, CASE WHEN subgroupName = 'Total Students' THEN schoolData.course_certification ELSE NULL END AS certificationRate
    SET offers.certificationRate = certificationRate
    """
    session.run(cypher_query, {"data": data})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Here we take a subset for demonstration
test_data_chemistry = transformed_chemistry_data[:10]

# Execute the insertion for the chemistry data
with driver.session() as session:
    insert_chemistry_data(session, test_data_chemistry)

driver.close()

 80%|████████  | 16/20 [29:59<07:39, 114.84s/it]

## CRDC Physics

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
physics_path = './Physics.csv'  # Update with the actual file path
physics_df = pd.read_csv(physics_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Physics Preview:")
print(physics_df.head(), "\n")

Physics Preview:
  LEA_STATE LEA_STATE_NAME    LEAID                LEA_NAME  SCHID  \
0        AL        Alabama  0100002  Alabama Youth Services   1705   
1        AL        Alabama  0100002  Alabama Youth Services   1706   
2        AL        Alabama  0100002  Alabama Youth Services  99995   
3        AL        Alabama  0100005        Albertville City    870   
4        AL        Alabama  0100005        Albertville City    871   

                        SCH_NAME      COMBOKEY   JJ  SCH_SCICLASSES_PHYS  \
0  Wallace Sch - Mt Meigs Campus  010000201705  Yes                    0   
1      Mcneel Sch - Vacca Campus  010000201706  Yes                    0   
2                 Autauga Campus  010000299995  Yes                    0   
3      Albertville Middle School  010000500870   No                   -9   
4        Albertville High School  010000500871   No                    2   

   SCH_SCICCERT_PHYS  ...  SCH_SCIENR_PHYS_WH_M  SCH_SCIENR_PHYS_WH_F  \
0                 -9  ...       

### Transform

In [ ]:
from collections import defaultdict

def transform_physics_data(df, year):
    race_categories = {
        "HI": "Hispanic",
        "AM": "American Indian/Alaska Native",
        "AS": "Asian",
        "HP": "Native Hawaiian/Pacific Islander",
        "BL": "Black",
        "WH": "White",
        "TR": "Two or More Races",
        "LEP": "Limited English Proficiency",
        "IDEA": "Student With Disability (IDEA)"
    }

    transformed_data = []
    for _, row in df.iterrows():
        enrollments = {}
        total_students = {}  # Initialize without default values

        # Process total enrollments first
        total_students["Male"] = int(row.get("TOT_SCIENR_PHYS_M", 0))
        total_students["Female"] = int(row.get("TOT_SCIENR_PHYS_F", 0))
        total_students["Total"] = total_students["Male"] + total_students["Female"]

        for key, value in row.items():
            # Skip indicators, non-numeric values, and keys not related to Physics enrollments
            if 'PHYS' not in key or value in (-9, '-9', '') or 'IND' in key or 'TOT_' in key:
                continue

            parts = key.split("_")
            # Ensure there are enough parts to get race code and gender
            if len(parts) < 4:
                continue

            race_code = parts[3]
            race_name = race_categories.get(race_code, "Unknown")
            gender = "Male" if parts[-1] == "M" else "Female"
            increment = int(value) if value not in ['-9', ''] else 0

            if increment == 0:
                continue

            subgroup = f"{race_name}"
            if subgroup not in enrollments:
                enrollments[subgroup] = {"Male": 0, "Female": 0, "Total": 0}

            # Instead of incrementing, directly assign the total values for Male and Female
            enrollments[subgroup][gender] = increment
            # Total per subgroup is calculated as the sum of Male and Female to ensure accuracy
            enrollments[subgroup]["Total"] = enrollments[subgroup]["Male"] + enrollments[subgroup]["Female"]

        if total_students["Total"] > 0:  # Ensure there's at least one student before adding to transformed data
            # Adding "Total Students" subgroup with calculated totals
            enrollments["Total Students"] = total_students

            school_data = {
                "institution_id": row["COMBOKEY"],
                "institution_name": row["SCH_NAME"],
                "jurisdiction_name": row["LEA_STATE_NAME"],
                "lea_id": row["LEAID"],
                "lea_name": row["LEA_NAME"],
                "year": year,
                "enrollments": enrollments,
                "course_certification": row.get("SCH_SCICCERT_PHYS", 0) / row.get("SCH_SCICLASSES_PHYS", 1) if row.get("SCH_SCICLASSES_PHYS", 1) > 0 else None
            }

            transformed_data.append(school_data)

    return transformed_data

physics_df_subset = physics_df #.head(200)
transformed_physics_data = transform_physics_data(physics_df_subset, 2020)
print(transformed_physics_data[:5])

[{'institution_id': 10000500871, 'institution_name': 'Albertville High School', 'jurisdiction_name': 'Alabama', 'lea_id': 100005, 'lea_name': 'Albertville City', 'year': 2020, 'enrollments': {'Hispanic': {'Male': 5, 'Female': 4, 'Total': 9}, 'Black': {'Male': 1, 'Female': 0, 'Total': 1}, 'White': {'Male': 10, 'Female': 6, 'Total': 16}, 'Two or More Races': {'Male': 2, 'Female': 0, 'Total': 2}, 'Student With Disability (IDEA)': {'Male': 0, 'Female': 1, 'Total': 1}, 'Total Students': {'Male': 18, 'Female': 10, 'Total': 28}}, 'course_certification': 1.0}, {'institution_id': 10000600872, 'institution_name': 'Asbury High School', 'jurisdiction_name': 'Alabama', 'lea_id': 100006, 'lea_name': 'Marshall County', 'year': 2020, 'enrollments': {'Hispanic': {'Male': 1, 'Female': 6, 'Total': 7}, 'White': {'Male': 2, 'Female': 8, 'Total': 10}, 'Total Students': {'Male': 3, 'Female': 14, 'Total': 17}}, 'course_certification': 1.0}, {'institution_id': 10000601585, 'institution_name': 'Brindlee Mountai

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

def insert_test_data(session, data):
    cypher_query = """
    UNWIND $data AS schoolData
    // Create or find Jurisdiction
    MERGE (jurisdiction:Jurisdiction {name: schoolData.jurisdiction_name})
    // Create or find LEA
    MERGE (lea:LEA {ncesID: schoolData.lea_id, name: schoolData.lea_name})
    MERGE (jurisdiction)-[:INCLUDES]->(lea)
    // Create or find Institution
    MERGE (institution:Institution {ncesID: schoolData.institution_id})
        ON CREATE SET institution.name = schoolData.institution_name
    MERGE (lea)-[:INCLUDES]->(institution)

    // Adjust the course name to "Physics" and ensure the year is set correctly
    MERGE (course:Course {name: "Physics", date: schoolData.year})
    WITH institution, course, schoolData
    UNWIND keys(schoolData.enrollments) AS subgroupName
    WITH institution, course, schoolData, subgroupName, schoolData.enrollments[subgroupName] AS enrollmentData
    MERGE (subgroup:Subgroup {name: subgroupName})
    MERGE (institution)-[offers:OFFERS {date: schoolData.year, subgroup: subgroupName}]->(course)
        ON CREATE SET offers.male = enrollmentData.Male,
                      offers.female = enrollmentData.Female,
                      offers.total = enrollmentData.Total

    // Handling course certification rate for the "Total Students" subgroup
    WITH institution, offers, schoolData, CASE WHEN subgroupName = 'Total Students' THEN schoolData.course_certification ELSE NULL END AS certificationRate
    SET offers.certificationRate = certificationRate
    """
    session.run(cypher_query, {"data": data})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Here we take a subset for demonstration
test_data = transformed_physics_data

# Execute the insertion for the chemistry data
with driver.session() as session:
  for i in tqdm(range(5000, len(test_data) - 1, 1000)):
    sub_data = test_data [i:i+1000]
    insert_test_data(session, sub_data)

driver.close()

100%|██████████| 12/12 [34:41<00:00, 173.49s/it]


## CRDC Computer Science

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
compsci_path = './Computer Science.csv'  # Update with the actual file path
compsci_df = pd.read_csv(compsci_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Computer Science Preview:")
print(compsci_df.head(), "\n")

Computer Science Preview:
  LEA_STATE LEA_STATE_NAME    LEAID                LEA_NAME  SCHID  \
0        AL        Alabama  0100002  Alabama Youth Services   1705   
1        AL        Alabama  0100002  Alabama Youth Services   1706   
2        AL        Alabama  0100002  Alabama Youth Services  99995   
3        AL        Alabama  0100005        Albertville City    870   
4        AL        Alabama  0100005        Albertville City    871   

                        SCH_NAME      COMBOKEY   JJ  SCH_COMPCLASSES_CSCI  \
0  Wallace Sch - Mt Meigs Campus  010000201705  Yes                     0   
1      Mcneel Sch - Vacca Campus  010000201706  Yes                     0   
2                 Autauga Campus  010000299995  Yes                     0   
3      Albertville Middle School  010000500870   No                    -9   
4        Albertville High School  010000500871   No                     0   

   SCH_COMPCCERT_CSCI  ...  SCH_COMPENR_CSCI_WH_M  SCH_COMPENR_CSCI_WH_F  \
0             

### Transform

In [ ]:
from collections import defaultdict

def transform_compsci_data(df, year):
    race_categories = {
        "HI": "Hispanic",
        "AM": "American Indian/Alaska Native",
        "AS": "Asian",
        "HP": "Native Hawaiian/Pacific Islander",
        "BL": "Black",
        "WH": "White",
        "TR": "Two or More Races",
        "LEP": "Limited English Proficiency",
        "IDEA": "Student With Disability (IDEA)"
    }

    transformed_data = []
    for _, row in df.iterrows():
        enrollments = {}
        total_students = {}  # Initialize without default values

        # Process total enrollments first
        total_students["Male"] = int(row.get("TOT_COMPENR_CSCI_M", 0))
        total_students["Female"] = int(row.get("TOT_COMPENR_CSCI_F", 0))
        total_students["Total"] = total_students["Male"] + total_students["Female"]

        for key, value in row.items():
            # Skip indicators, non-numeric values, and keys not related to CS enrollments
            if 'CSCI' not in key or value in (-9, '-9', '') or 'IND' in key or 'TOT_' in key:
                continue

            parts = key.split("_")
            # Ensure there are enough parts to get race code and gender
            if len(parts) < 4:
                continue

            race_code = parts[3]
            race_name = race_categories.get(race_code, "Unknown")
            gender = "Male" if parts[-1] == "M" else "Female"
            increment = int(value) if value not in ['-9', ''] else 0

            if increment == 0:
                continue

            subgroup = f"{race_name}"
            if subgroup not in enrollments:
                enrollments[subgroup] = {"Male": 0, "Female": 0, "Total": 0}

            # Instead of incrementing, directly assign the total values for Male and Female
            enrollments[subgroup][gender] = increment
            # Total per subgroup is calculated as the sum of Male and Female to ensure accuracy
            enrollments[subgroup]["Total"] = enrollments[subgroup]["Male"] + enrollments[subgroup]["Female"]

        if total_students["Total"] > 0:  # Ensure there's at least one student before adding to transformed data
            # Adding "Total Students" subgroup with calculated totals
            enrollments["Total Students"] = total_students

            school_data = {
                "institution_id": row["COMBOKEY"],
                "institution_name": row["SCH_NAME"],
                "jurisdiction_name": row["LEA_STATE_NAME"],
                "lea_id": row["LEAID"],
                "lea_name": row["LEA_NAME"],
                "year": year,
                "enrollments": enrollments,
                "course_certification": row.get("SCH_COMPCCERT_CSCI", 0) / row.get("SCH_COMPCLASSES_CSCI", 1) if row.get("SCH_COMPCLASSES_CSCI", 1) > 0 else None
            }

            transformed_data.append(school_data)

    return transformed_data

compsci_df_subset = compsci_df #.head(200)
transformed_compsci_data = transform_compsci_data(compsci_df_subset, 2020)
print(transformed_compsci_data[:5])

[{'institution_id': 10000600872, 'institution_name': 'Asbury High School', 'jurisdiction_name': 'Alabama', 'lea_id': 100006, 'lea_name': 'Marshall County', 'year': 2020, 'enrollments': {'Hispanic': {'Male': 4, 'Female': 15, 'Total': 19}, 'White': {'Male': 15, 'Female': 13, 'Total': 28}, 'Limited English Proficiency': {'Male': 1, 'Female': 1, 'Total': 2}, 'Student With Disability (IDEA)': {'Male': 1, 'Female': 2, 'Total': 3}, 'Total Students': {'Male': 19, 'Female': 28, 'Total': 47}}, 'course_certification': 0.0}, {'institution_id': 10000600878, 'institution_name': 'Douglas High School', 'jurisdiction_name': 'Alabama', 'lea_id': 100006, 'lea_name': 'Marshall County', 'year': 2020, 'enrollments': {'Hispanic': {'Male': 0, 'Female': 1, 'Total': 1}, 'White': {'Male': 2, 'Female': 0, 'Total': 2}, 'Total Students': {'Male': 2, 'Female': 1, 'Total': 3}}, 'course_certification': 1.0}, {'institution_id': 10000600883, 'institution_name': 'Kate D Smith Dar High School', 'jurisdiction_name': 'Alaba

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

def insert_test_data(session, data):
    cypher_query = """
    UNWIND $data AS schoolData
    // Create or find Jurisdiction
    MERGE (jurisdiction:Jurisdiction {name: schoolData.jurisdiction_name})
    // Create or find LEA
    MERGE (lea:LEA {ncesID: schoolData.lea_id, name: schoolData.lea_name})
    MERGE (jurisdiction)-[:INCLUDES]->(lea)
    // Create or find Institution
    MERGE (institution:Institution {ncesID: schoolData.institution_id})
        ON CREATE SET institution.name = schoolData.institution_name
    MERGE (lea)-[:INCLUDES]->(institution)

    MERGE (course:Course {name: "Computer Science", date: schoolData.year})
    WITH institution, course, schoolData
    UNWIND keys(schoolData.enrollments) AS subgroupName
    WITH institution, course, schoolData, subgroupName, schoolData.enrollments[subgroupName] AS enrollmentData
    MERGE (institution)-[offers:OFFERS {date: schoolData.year, subgroup: subgroupName}]->(course)
        ON CREATE SET offers.male = enrollmentData.Male,
                      offers.female = enrollmentData.Female,
                      offers.total = enrollmentData.Total

    // Handling course certification rate for the "Total Students" subgroup
    WITH institution, offers, schoolData, CASE WHEN subgroupName = 'Total Students' THEN schoolData.course_certification ELSE NULL END AS certificationRate
    SET offers.certificationRate = certificationRate
    """
    session.run(cypher_query, {"data": data})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Here we take a subset for demonstration
test_data = transformed_compsci_data

# Execute the insertion for the chemistry data
with driver.session() as session:
  for i in tqdm(range(0, len(test_data) - 1, 1000)):
    sub_data = test_data [i:i+1000]
    insert_test_data(session, sub_data)

driver.close()

  0%|          | 0/13 [00:05<?, ?it/s]


KeyboardInterrupt: 

## CRDC Advanced Placement

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
advplacement_path = './Advanced Placement.csv'  # Update with the actual file path
advplacement_df = pd.read_csv(advplacement_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Advanced Placement Preview:")
print(advplacement_df.head(), "\n")

Advanced Placement Preview:
  LEA_STATE LEA_STATE_NAME    LEAID                LEA_NAME  SCHID  \
0        AL        Alabama  0100002  Alabama Youth Services   1705   
1        AL        Alabama  0100002  Alabama Youth Services   1706   
2        AL        Alabama  0100002  Alabama Youth Services  99995   
3        AL        Alabama  0100005        Albertville City    870   
4        AL        Alabama  0100005        Albertville City    871   

                        SCH_NAME      COMBOKEY   JJ SCH_APENR_IND  \
0  Wallace Sch - Mt Meigs Campus  010000201705  Yes            -9   
1      Mcneel Sch - Vacca Campus  010000201706  Yes            -9   
2                 Autauga Campus  010000299995  Yes            -9   
3      Albertville Middle School  010000500870   No            -9   
4        Albertville High School  010000500871   No           Yes   

   SCH_APCOURSES  ... SCH_APCOMPENR_WH_M  SCH_APCOMPENR_WH_F  \
0             -9  ...                 -9                  -9   
1       

### Transform

In [ ]:
from collections import defaultdict

def transform_advplacement_data(df, year):
    race_categories = {
        "HI": "Hispanic",
        "AM": "American Indian/Alaska Native",
        "AS": "Asian",
        "HP": "Native Hawaiian/Pacific Islander",
        "BL": "Black",
        "WH": "White",
        "TR": "Two or More Races",
        "LEP": "Limited English Proficiency",
        "IDEA": "Student with Disability (IDEA)"
    }
    ap_subjects = {
        "APMATHENR": "AP Math",
        "APSCIENR": "AP Science",
        "APCOMPENR": "AP Computer Science"
    }

    ap_course_types = {
        "APENR": "All Advanced Placement (AP)",
        "APMATHENR": "Advanced Placement (AP) Math",
        "APSCIENR": "Advanced Placement (AP) Science",
        "APCOMPENR": "Advanced Placement (AP) Computer Science"
    }

    transformed_data = []

    for _, row in df.iterrows():
        ap_enrollments = {course_name: {"Total": {"Male": 0, "Female": 0, "Total": 0}} for course_name in ap_course_types.values()}

        # Handle total enrollments first
        for key, value in row.items():
            if 'TOT_' in key and value not in [-9, '-9', '']:
                parts = key.split("_")
                course_prefix = next((prefix for prefix in ap_course_types.keys() if prefix in key), None)
                if not course_prefix:
                    continue

                course_name = ap_course_types[course_prefix]
                gender = "Male" if "M" in parts[-1] else "Female"
                increment = max(int(value), 0)

                ap_enrollments[course_name]["Total"][gender] += increment
                # No need to calculate the "Total" value here; it will be calculated after processing individual enrollments

        # Process individual race and gender enrollments
        for key, value in row.items():
            if value in (-9, '-9', '') or 'IND' in key or 'TOT_' in key:
                continue

            parts = key.split("_")
            course_prefix = next((prefix for prefix in ap_course_types.keys() if prefix in key), None)
            if not course_prefix:
                continue

            course_name = ap_course_types[course_prefix]
            race_code = parts[2]
            race_name = race_categories.get(race_code, "Unknown")
            gender = "Male" if "M" in parts[-1] else "Female"
            increment = max(int(value), 0)

            if increment == 0:
                continue

            subgroup = f"{race_name}"
            if subgroup not in ap_enrollments[course_name]:
                ap_enrollments[course_name][subgroup] = {"Male": 0, "Female": 0, "Total": 0}

            ap_enrollments[course_name][subgroup][gender] += increment
            # Update totals after each increment
            ap_enrollments[course_name][subgroup]["Total"] = ap_enrollments[course_name][subgroup]["Male"] + ap_enrollments[course_name][subgroup]["Female"]
            ap_enrollments[course_name]["Total"]["Total"] = ap_enrollments[course_name]["Total"]["Male"] + ap_enrollments[course_name]["Total"]["Female"]

        # Cleanup: Remove course types without enrollment data and "Total" category if it only contains zeros
        for course_name in list(ap_enrollments.keys()):
            if all(value == 0 for value in ap_enrollments[course_name]["Total"].values()):
                del ap_enrollments[course_name]

        if ap_enrollments:  # Only include if there's at least one enrollment
            school_data = {
                "institution_id": row["COMBOKEY"],
                "institution_name": row["SCH_NAME"],
                "jurisdiction_name": row["LEA_STATE_NAME"],
                "lea_id": row["LEAID"],
                "lea_name": row["LEA_NAME"],
                "year": year,
                "ap_courses": ap_enrollments
            }
            transformed_data.append(school_data)

    return transformed_data

advplacement_df_subset = advplacement_df #.head(200)
transformed_advplacement_data = transform_advplacement_data(advplacement_df_subset, 2020)
print(transformed_advplacement_data[:5])

[{'institution_id': 10000500871, 'institution_name': 'Albertville High School', 'jurisdiction_name': 'Alabama', 'lea_id': 100005, 'lea_name': 'Albertville City', 'year': 2020, 'ap_courses': {'All Advanced Placement (AP)': {'Total': {'Male': 96, 'Female': 131, 'Total': 227}, 'Hispanic': {'Male': 32, 'Female': 56, 'Total': 88}, 'American Indian/Alaska Native': {'Male': 1, 'Female': 0, 'Total': 1}, 'Asian': {'Male': 1, 'Female': 0, 'Total': 1}, 'Black': {'Male': 1, 'Female': 3, 'Total': 4}, 'White': {'Male': 58, 'Female': 72, 'Total': 130}, 'Two or More Races': {'Male': 3, 'Female': 0, 'Total': 3}, 'Limited English Proficiency': {'Male': 1, 'Female': 0, 'Total': 1}, 'Student with Disability (IDEA)': {'Male': 8, 'Female': 5, 'Total': 13}}}}, {'institution_id': 10000600872, 'institution_name': 'Asbury High School', 'jurisdiction_name': 'Alabama', 'lea_id': 100006, 'lea_name': 'Marshall County', 'year': 2020, 'ap_courses': {'All Advanced Placement (AP)': {'Total': {'Male': 22, 'Female': 39, 

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

def insert_test_data(session, data):
    cypher_query = """
    UNWIND $data AS schoolData
    // Create or find Jurisdiction
    MERGE (jurisdiction:Jurisdiction {name: schoolData.jurisdiction_name})
    // Create or find LEA
    MERGE (lea:LEA {ncesID: schoolData.lea_id, name: schoolData.lea_name})
    MERGE (jurisdiction)-[:INCLUDES]->(lea)
    // Create or find Institution
    MERGE (institution:Institution {ncesID: schoolData.institution_id})
        ON CREATE SET institution.name = schoolData.institution_name
    MERGE (lea)-[:INCLUDES]->(institution)

    WITH schoolData, institution
    UNWIND keys(schoolData.ap_courses) AS apCourseType
    WITH institution, apCourseType, schoolData.ap_courses[apCourseType] AS courseDetails, schoolData
    MERGE (intervention:Intervention {name: apCourseType, date: schoolData.year})

    // Now, for each course type, process each subgroup including Total
    WITH institution, intervention, courseDetails, schoolData
    UNWIND keys(courseDetails) AS subgroupKey
    WITH institution, intervention, subgroupKey, courseDetails[subgroupKey] AS subgroupDetails, schoolData
    MERGE (subgroup:Subgroup {name: subgroupKey})
    MERGE (institution)-[assigns:ASSIGNS {date: schoolData.year, subgroup: subgroupKey}]->(intervention)
      ON CREATE SET assigns.total = subgroupDetails.Total, assigns.male = subgroupDetails.Male, assigns.female = subgroupDetails.Female

    """
    session.run(cypher_query, {"data": data})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Here we take a subset for demonstration
test_data = transformed_advplacement_data

# Execute the insertion for the AP data
with driver.session() as session:
  for i in tqdm(range(6000, len(test_data) - 1, 1000)):
    sub_data = test_data [i:i+1000]
    insert_test_data(session, sub_data)
driver.close()

100%|██████████| 9/9 [23:35<00:00, 157.25s/it]



## CRDC International Baccelaureate

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
ib_path = './International Baccalaureate.csv'  # Update with the actual file path
ib_df = pd.read_csv(ib_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("International Baccalaureate Preview:")
print(ib_df.head(), "\n")

International Baccalaureate Preview:
  LEA_STATE LEA_STATE_NAME    LEAID                LEA_NAME  SCHID  \
0        AL        Alabama  0100002  Alabama Youth Services   1705   
1        AL        Alabama  0100002  Alabama Youth Services   1706   
2        AL        Alabama  0100002  Alabama Youth Services  99995   
3        AL        Alabama  0100005        Albertville City    870   
4        AL        Alabama  0100005        Albertville City    871   

                        SCH_NAME      COMBOKEY   JJ SCH_IBENR_IND  \
0  Wallace Sch - Mt Meigs Campus  010000201705  Yes            -9   
1      Mcneel Sch - Vacca Campus  010000201706  Yes            -9   
2                 Autauga Campus  010000299995  Yes            -9   
3      Albertville Middle School  010000500870   No            -9   
4        Albertville High School  010000500871   No            No   

   SCH_IBENR_HI_M  ...  SCH_IBENR_WH_M  SCH_IBENR_WH_F  SCH_IBENR_TR_M  \
0              -9  ...              -9              -

### Transform

In [ ]:
from collections import defaultdict

def transform_ib_data(df, year):
    race_categories = {
        "HI": "Hispanic",
        "AM": "American Indian/Alaska Native",
        "AS": "Asian",
        "HP": "Native Hawaiian/Pacific Islander",
        "BL": "Black",
        "WH": "White",
        "TR": "Two or More Races",
        "LEP": "Limited English Proficiency",
        "IDEA": "Student with Disability (IDEA)"
    }

    transformed_data = []
    for _, row in df.iterrows():
        enrollments = {}
        total_students = {
            "Male": int(row.get("TOT_IBENR_M", 0)),
            "Female": int(row.get("TOT_IBENR_F", 0)),
            "Total": int(row.get("TOT_IBENR_M", 0)) + int(row.get("TOT_IBENR_F", 0))
        }

        for key, value in row.items():
            # Skip non-enrollment columns and handle missing or negative data
            if 'IBENR' not in key or value in [-9, '-9', ''] or 'IND' in key or 'TOT_' in key:
                continue

            parts = key.split("_")
            if len(parts) < 3:
                continue

            race_code = parts[2]
            gender = "Male" if parts[-1] == "M" else "Female"
            race_name = race_categories.get(race_code, "Unknown")
            increment = int(value) if value not in ['-9', ''] else 0

            if increment == 0:
                continue

            subgroup = f"{race_name}"
            if subgroup not in enrollments:
                enrollments[subgroup] = {"Male": 0, "Female": 0, "Total": 0}

            enrollments[subgroup][gender] = increment
            enrollments[subgroup]["Total"] += increment

        if total_students["Total"] > 0:
            enrollments["Total Students"] = total_students

            school_data = {
                "institution_id": row["COMBOKEY"],
                "institution_name": row["SCH_NAME"],
                "jurisdiction_name": row["LEA_STATE_NAME"],
                "lea_id": row["LEAID"],
                "lea_name": row["LEA_NAME"],
                "year": year,
                "enrollments": enrollments
            }

            transformed_data.append(school_data)

    return transformed_data

ib_df_subset = ib_df.head(200)
transformed_ib_data = transform_ib_data(ib_df_subset, 2020)
print(transformed_ib_data[:5])

[{'institution_id': 10000700251, 'institution_name': 'Hoover High School', 'jurisdiction_name': 'Alabama', 'lea_id': 100007, 'lea_name': 'Hoover City', 'year': 2020, 'enrollments': {'Hispanic': {'Male': 3, 'Female': 2, 'Total': 5}, 'Asian': {'Male': 19, 'Female': 17, 'Total': 36}, 'Black': {'Male': 0, 'Female': 10, 'Total': 10}, 'White': {'Male': 21, 'Female': 29, 'Total': 50}, 'Two or More Races': {'Male': 4, 'Female': 1, 'Total': 5}, 'Limited English Proficiency': {'Male': 12, 'Female': 10, 'Total': 22}, 'Student with Disability (IDEA)': {'Male': 4, 'Female': 3, 'Total': 7}, 'Total Students': {'Male': 47, 'Female': 59, 'Total': 106}}}, {'institution_id': 10021000036, 'institution_name': 'Auburn High School', 'jurisdiction_name': 'Alabama', 'lea_id': 100210, 'lea_name': 'Auburn City', 'year': 2020, 'enrollments': {'American Indian/Alaska Native': {'Male': 0, 'Female': 1, 'Total': 1}, 'Asian': {'Male': 16, 'Female': 19, 'Total': 35}, 'White': {'Male': 9, 'Female': 8, 'Total': 17}, 'Lim

### Load

In [ ]:
from neo4j import GraphDatabase

def insert_ib_data(session, data):
    cypher_query = """
    UNWIND $data AS schoolData
    // Create or find Jurisdiction
    MERGE (jurisdiction:Jurisdiction {name: schoolData.jurisdiction_name})
    // Create or find LEA
    MERGE (lea:LEA {ncesID: schoolData.lea_id, name: schoolData.lea_name})
    MERGE (jurisdiction)-[:INCLUDES]->(lea)
    // Create or find Institution
    MERGE (institution:Institution {ncesID: schoolData.institution_id})
        ON CREATE SET institution.name = schoolData.institution_name
    MERGE (lea)-[:INCLUDES]->(institution)

    // Create or find Intervention for IB
    MERGE (intervention:Intervention {name: "International Baccalaureate (IB)", date: schoolData.year})
    WITH institution, intervention, schoolData
    UNWIND keys(schoolData.enrollments) AS subgroupName
    WITH institution, intervention, schoolData, subgroupName, schoolData.enrollments[subgroupName] AS enrollmentData
    // Create ASSIGN relationship between Institution and Intervention
    MERGE (institution)-[assigns:ASSIGNS {date: schoolData.year, subgroup: subgroupName}]->(intervention)
        ON CREATE SET assigns.male = enrollmentData.Male,
                      assigns.female = enrollmentData.Female,
                      assigns.total = enrollmentData.Total
    """
    session.run(cypher_query, {"data": data})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Here we take a subset for demonstration
test_data_ib = transformed_ib_data[:10]

# Execute the insertion for the AP data
with driver.session() as session:
    insert_ib_data(session, test_data_ib)

driver.close()

## CRDC Dual Enrollment

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
dualenr_path = './Dual Enrollment.csv'  # Update with the actual file path
dualenr_df = pd.read_csv(dualenr_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Dual Enrollment Preview:")
print(dualenr_df.head(), "\n")

Dual Enrollment Preview:
  LEA_STATE LEA_STATE_NAME    LEAID                LEA_NAME  SCHID  \
0        AL        Alabama  0100002  Alabama Youth Services   1705   
1        AL        Alabama  0100002  Alabama Youth Services   1706   
2        AL        Alabama  0100002  Alabama Youth Services  99995   
3        AL        Alabama  0100005        Albertville City    870   
4        AL        Alabama  0100005        Albertville City    871   

                        SCH_NAME      COMBOKEY   JJ SCH_DUAL_IND  \
0  Wallace Sch - Mt Meigs Campus  010000201705  Yes           No   
1      Mcneel Sch - Vacca Campus  010000201706  Yes           No   
2                 Autauga Campus  010000299995  Yes           No   
3      Albertville Middle School  010000500870   No           -9   
4        Albertville High School  010000500871   No          Yes   

   SCH_DUALENR_HI_M  ...  SCH_DUALENR_WH_M  SCH_DUALENR_WH_F  \
0                -9  ...                -9                -9   
1                

### Transform

In [ ]:
from collections import defaultdict

def transform_dualenr_data(df, year):
    race_categories = {
        "HI": "Hispanic",
        "AM": "American Indian/Alaska Native",
        "AS": "Asian",
        "HP": "Native Hawaiian/Pacific Islander",
        "BL": "Black",
        "WH": "White",
        "TR": "Two or More Races",
        "LEP": "Limited English Proficiency",
        "IDEA": "Student with Disability (IDEA)"
    }

    transformed_data = []
    for _, row in df.iterrows():
        if row['SCH_DUAL_IND'] == 'No' or row['SCH_DUAL_IND'] == -9:  # Skip schools without Dual Enrollment data
            continue

        enrollments = {}
        total_students = {
            "Male": int(row.get("TOT_DUAL_M", 0)),
            "Female": int(row.get("TOT_DUAL_F", 0)),
            "Total": int(row.get("TOT_DUAL_M", 0)) + int(row.get("TOT_DUAL_F", 0))
        }

        for key, value in row.items():
            if key.startswith('SCH_DUALENR_') and value not in [-9, '-9', '']:
                parts = key.split('_')
                if len(parts) < 4:
                    continue

                race_code = parts[2]
                gender = "Male" if parts[-1] == "M" else "Female"
                race_name = race_categories.get(race_code, "Unknown")

                if race_name not in enrollments:
                    enrollments[race_name] = {"Male": 0, "Female": 0, "Total": 0}

                enrollments[race_name][gender] = int(value)
                enrollments[race_name]["Total"] += int(value)

        if total_students["Total"] > 0:  # Ensure there are enrolled students before adding to transformed data
            enrollments["Total Students"] = total_students

            school_data = {
                "institution_id": row["COMBOKEY"],
                "institution_name": row["SCH_NAME"],
                "jurisdiction_name": row["LEA_STATE_NAME"],
                "lea_id": row["LEAID"],
                "lea_name": row["LEA_NAME"],
                "year": year,
                "enrollments": enrollments,
            }

            transformed_data.append(school_data)

    return transformed_data

dualenr_df_subset = dualenr_df #.head(200)
transformed_dualenr_data = transform_dualenr_data(dualenr_df_subset, 2020)
print(transformed_dualenr_data[:5])

[{'institution_id': 10000500871, 'institution_name': 'Albertville High School', 'jurisdiction_name': 'Alabama', 'lea_id': 100005, 'lea_name': 'Albertville City', 'year': 2020, 'enrollments': {'Hispanic': {'Male': 5, 'Female': 9, 'Total': 14}, 'American Indian/Alaska Native': {'Male': 0, 'Female': 0, 'Total': 0}, 'Asian': {'Male': 0, 'Female': 0, 'Total': 0}, 'Native Hawaiian/Pacific Islander': {'Male': 0, 'Female': 0, 'Total': 0}, 'Black': {'Male': 1, 'Female': 1, 'Total': 2}, 'White': {'Male': 20, 'Female': 16, 'Total': 36}, 'Two or More Races': {'Male': 0, 'Female': 0, 'Total': 0}, 'Limited English Proficiency': {'Male': 2, 'Female': 0, 'Total': 2}, 'Student with Disability (IDEA)': {'Male': 7, 'Female': 4, 'Total': 11}, 'Total Students': {'Male': 26, 'Female': 26, 'Total': 52}}}, {'institution_id': 10000600872, 'institution_name': 'Asbury High School', 'jurisdiction_name': 'Alabama', 'lea_id': 100006, 'lea_name': 'Marshall County', 'year': 2020, 'enrollments': {'Hispanic': {'Male': 

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

def insert_test_data(session, data):
    cypher_query = """
    UNWIND $data AS schoolData
    // Create or find Jurisdiction
    MERGE (jurisdiction:Jurisdiction {name: schoolData.jurisdiction_name})
    // Create or find LEA
    MERGE (lea:LEA {ncesID: schoolData.lea_id, name: schoolData.lea_name})
    MERGE (jurisdiction)-[:INCLUDES]->(lea)
    // Create or find Institution
    MERGE (institution:Institution {ncesID: schoolData.institution_id})
        ON CREATE SET institution.name = schoolData.institution_name
    MERGE (lea)-[:INCLUDES]->(institution)

    MERGE (intervention:Intervention {name: "Dual Enrollment", date: schoolData.year})
    WITH institution, intervention, schoolData
    UNWIND keys(schoolData.enrollments) AS subgroupName
    WITH institution, intervention, schoolData, subgroupName, schoolData.enrollments[subgroupName] AS enrollmentData
    // Create ASSIGN relationship between Institution and Intervention
    MERGE (institution)-[assigns:ASSIGNS {date: schoolData.year, subgroup: subgroupName}]->(intervention)
        ON CREATE SET assigns.male = enrollmentData.Male,
                      assigns.female = enrollmentData.Female,
                      assigns.total = enrollmentData.Total
    """
    session.run(cypher_query, {"data": data})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Here we take a subset for demonstration
test_data = transformed_dualenr_data

# Execute the insertion for the Duel Enrollment data
with driver.session() as session:
  for i in tqdm(range(11000, len(test_data) - 1, 1000)):
    sub_data = test_data [i:i+1000]
    insert_test_data(session, sub_data)

driver.close()

100%|██████████| 6/6 [14:54<00:00, 149.13s/it]


## CRDC Gifted and Talented

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
gt_path = './Gifted and Talented.csv'  # Update with the actual file path
gt_df = pd.read_csv(gt_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Gifted and Talented Preview:")
print(gt_df.head(), "\n")

Gifted and Talented Preview:
  LEA_STATE LEA_STATE_NAME    LEAID                LEA_NAME  SCHID  \
0        AL        Alabama  0100002  Alabama Youth Services   1705   
1        AL        Alabama  0100002  Alabama Youth Services   1706   
2        AL        Alabama  0100002  Alabama Youth Services  99995   
3        AL        Alabama  0100005        Albertville City    870   
4        AL        Alabama  0100005        Albertville City    871   

                        SCH_NAME      COMBOKEY   JJ SCH_GT_IND  \
0  Wallace Sch - Mt Meigs Campus  010000201705  Yes         No   
1      Mcneel Sch - Vacca Campus  010000201706  Yes         No   
2                 Autauga Campus  010000299995  Yes         No   
3      Albertville Middle School  010000500870   No         No   
4        Albertville High School  010000500871   No         No   

   SCH_GTENR_HI_M  ...  SCH_GTENR_WH_M  SCH_GTENR_WH_F  SCH_GTENR_TR_M  \
0              -9  ...              -9              -9              -9   
1    

### Transform

In [ ]:
from collections import defaultdict

def transform_gt_data(df, year):
    race_categories = {
        "HI": "Hispanic",
        "AM": "American Indian/Alaska Native",
        "AS": "Asian",
        "HP": "Native Hawaiian/Pacific Islander",
        "BL": "Black",
        "WH": "White",
        "TR": "Two or More Races",
        "LEP": "Limited English Proficiency",
        "IDEA": "Student with Disability (IDEA)"
    }

    transformed_data = []
    for _, row in df.iterrows():
        enrollments = {}
        total_students = {}  # Initialize without default values

        # Process total enrollments first
        total_students["Male"] = int(row.get("TOT_GTENR_M", 0))
        total_students["Female"] = int(row.get("TOT_GTENR_F", 0))
        total_students["Total"] = total_students["Male"] + total_students["Female"]

        for key, value in row.items():
            # Skip indicators, non-numeric values, and keys not related to Geometry enrollments
            if 'GT' not in key or value in (-9, '-9', '') or 'IND' in key or 'TOT_' in key:
                continue

            parts = key.split("_")
            # Ensure there are enough parts to get race code and gender
            if len(parts) < 4:
                continue

            race_code = parts[2]
            race_name = race_categories.get(race_code, "Unknown")
            gender = "Male" if parts[-1] == "M" else "Female"
            increment = int(value) if value not in ['-9', ''] else 0

            if increment == 0:
                continue

            subgroup = f"{race_name}"
            if subgroup not in enrollments:
                enrollments[subgroup] = {"Male": 0, "Female": 0, "Total": 0}

            # Instead of incrementing, directly assign the total values for Male and Female
            enrollments[subgroup][gender] = increment
            # Total per subgroup is calculated as the sum of Male and Female to ensure accuracy
            enrollments[subgroup]["Total"] = enrollments[subgroup]["Male"] + enrollments[subgroup]["Female"]

        if total_students["Total"] > 0:  # Ensure there's at least one student before adding to transformed data
            # Adding "Total Students" subgroup with calculated totals
            enrollments["Total Students"] = total_students

            school_data = {
                "institution_id": row["COMBOKEY"],
                "institution_name": row["SCH_NAME"],
                "jurisdiction_name": row["LEA_STATE_NAME"],
                "lea_id": row["LEAID"],
                "lea_name": row["LEA_NAME"],
                "year": year,
                "enrollments": enrollments,
            }

            transformed_data.append(school_data)

    return transformed_data

gt_df_subset = gt_df #.head(200)
transformed_gt_data = transform_gt_data(gt_df_subset, 2020)
print(transformed_gt_data[:5])

[{'institution_id': 10000500879, 'institution_name': 'Albertville Intermediate School', 'jurisdiction_name': 'Alabama', 'lea_id': 100005, 'lea_name': 'Albertville City', 'year': 2020, 'enrollments': {'Hispanic': {'Male': 7, 'Female': 10, 'Total': 17}, 'American Indian/Alaska Native': {'Male': 3, 'Female': 1, 'Total': 4}, 'Native Hawaiian/Pacific Islander': {'Male': 0, 'Female': 1, 'Total': 1}, 'White': {'Male': 28, 'Female': 24, 'Total': 52}, 'Two or More Races': {'Male': 0, 'Female': 2, 'Total': 2}, 'Limited English Proficiency': {'Male': 4, 'Female': 0, 'Total': 4}, 'Total Students': {'Male': 38, 'Female': 38, 'Total': 76}}}, {'institution_id': 10000500889, 'institution_name': 'Albertville Elementary School', 'jurisdiction_name': 'Alabama', 'lea_id': 100005, 'lea_name': 'Albertville City', 'year': 2020, 'enrollments': {'Hispanic': {'Male': 5, 'Female': 5, 'Total': 10}, 'Asian': {'Male': 1, 'Female': 0, 'Total': 1}, 'Black': {'Male': 0, 'Female': 1, 'Total': 1}, 'White': {'Male': 15, 

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

def insert_test_data(session, data):
    cypher_query = """
    UNWIND $data AS schoolData
    // Create or find Jurisdiction
    MERGE (jurisdiction:Jurisdiction {name: schoolData.jurisdiction_name})
    // Create or find LEA
    MERGE (lea:LEA {ncesID: schoolData.lea_id, name: schoolData.lea_name})
    MERGE (jurisdiction)-[:INCLUDES]->(lea)
    // Create or find Institution
    MERGE (institution:Institution {ncesID: schoolData.institution_id})
        ON CREATE SET institution.name = schoolData.institution_name
    MERGE (lea)-[:INCLUDES]->(institution)

    MERGE (intervention:Intervention {name: "Gifted and Talented", date: schoolData.year})
    WITH institution, intervention, schoolData
    UNWIND keys(schoolData.enrollments) AS subgroupName
    WITH institution, intervention, schoolData, subgroupName, schoolData.enrollments[subgroupName] AS enrollmentData
    // Create ASSIGN relationship between Institution and Intervention
    MERGE (institution)-[assigns:ASSIGNS {date: schoolData.year, subgroup: subgroupName}]->(intervention)
        ON CREATE SET assigns.male = enrollmentData.Male,
                      assigns.female = enrollmentData.Female,
                      assigns.total = enrollmentData.Total
    """
    session.run(cypher_query, {"data": data})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Here we take a subset for demonstration
test_data = transformed_gt_data

# Execute the insertion for the GT data
with driver.session() as session:
  for i in tqdm(range(43000, len(test_data) - 1, 1000)):
    sub_data = test_data [i:i+1000]
    insert_test_data(session, sub_data)

driver.close()

100%|██████████| 12/12 [34:46<00:00, 173.86s/it]


## CRDC Single Sex Classes

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
singlesexclass_path = './Single sex Classes.csv'  # Update with the actual file path
singlesexclass_df = pd.read_csv(singlesexclass_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Single Sex Classes Preview:")
print(singlesexclass_df.head(), "\n")

Single Sex Classes Preview:
  LEA_STATE LEA_STATE_NAME    LEAID                LEA_NAME  SCHID  \
0        AL        Alabama  0100002  Alabama Youth Services   1705   
1        AL        Alabama  0100002  Alabama Youth Services   1706   
2        AL        Alabama  0100002  Alabama Youth Services  99995   
3        AL        Alabama  0100005        Albertville City    870   
4        AL        Alabama  0100005        Albertville City    871   

                        SCH_NAME      COMBOKEY   JJ SCH_SSCLASSES_IND  \
0  Wallace Sch - Mt Meigs Campus  010000201705  Yes                -9   
1      Mcneel Sch - Vacca Campus  010000201706  Yes                -9   
2                 Autauga Campus  010000299995  Yes                -9   
3      Albertville Middle School  010000500870   No                No   
4        Albertville High School  010000500871   No                No   

   SCH_SSCLASSES_ALGG_M  ...  TOT_SSCLASSES_OTHM  SCH_SSCLASSES_SCI_M  \
0                    -9  ...           

### Transform

In [ ]:
from collections import defaultdict

def transform_singlesexclass_data(df, year):
    # Define the types of single sex classes offered
    class_types = {
        "ALGG": "Algebra I, Geometry, Algebra II",
        "OTHM": "Other Mathematics",
        "SCI": "Science",
        "ENGL": "English, Reading, Language Arts",
        "OTHA": "Other Academic Subjects"
    }

    transformed_data = []

    for _, row in df.iterrows():
        single_sex_enrollments = {}

        # Iterate through each type of class to process enrollments
        for class_code, class_name in class_types.items():
            male_key = f"SCH_SSCLASSES_{class_code}_M"
            female_key = f"SCH_SSCLASSES_{class_code}_F"
            total_key = f"TOT_SSCLASSES_{class_code}"

            male_enrollment = row.get(male_key, 0)
            female_enrollment = row.get(female_key, 0)
            total_enrollment = row.get(total_key, 0)

            # Adjust for non-numeric or placeholder values
            male_enrollment = max(int(male_enrollment), 0) if male_enrollment not in [-9, '-9', ''] else 0
            female_enrollment = max(int(female_enrollment), 0) if female_enrollment not in [-9, '-9', ''] else 0
            total_enrollment = max(int(total_enrollment), 0) if total_enrollment not in [-9, '-9', ''] else 0

            # If there is enrollment data, add it to the dictionary
            if male_enrollment > 0 or female_enrollment > 0:
                single_sex_enrollments[class_name] = {
                    "Male": male_enrollment,
                    "Female": female_enrollment,
                    "Total": total_enrollment
                }

        # If there are any class enrollments, add the school data to the list
        if single_sex_enrollments:
            school_data = {
                "institution_id": row["COMBOKEY"],
                "institution_name": row["SCH_NAME"],
                "jurisdiction_name": row["LEA_STATE_NAME"],
                "lea_id": row["LEAID"],
                "lea_name": row["LEA_NAME"],
                "year": year,
                "single_sex_classes": single_sex_enrollments
            }
            transformed_data.append(school_data)

    return transformed_data

singlesexclass_df_subset = singlesexclass_df.head(200)
transformed_singlesexclass_data = transform_singlesexclass_data(singlesexclass_df_subset, 2020)
print(transformed_singlesexclass_data[:5])

[{'institution_id': 10003602451, 'institution_name': 'Lee Co Youth Detention Ctr', 'jurisdiction_name': 'Alabama', 'lea_id': 100036, 'lea_name': 'Lee County Youth Development Center', 'year': 2020, 'single_sex_classes': {'Algebra I, Geometry, Algebra II': {'Male': 1, 'Female': 1, 'Total': 2}, 'Other Mathematics': {'Male': 1, 'Female': 1, 'Total': 2}, 'Science': {'Male': 1, 'Female': 1, 'Total': 2}, 'English, Reading, Language Arts': {'Male': 1, 'Female': 1, 'Total': 2}, 'Other Academic Subjects': {'Male': 1, 'Female': 1, 'Total': 2}}}, {'institution_id': 10004201853, 'institution_name': 'Laurel Oaks Behavioral Health Center', 'jurisdiction_name': 'Alabama', 'lea_id': 100042, 'lea_name': 'Laurel Oaks Behavioral Health Center', 'year': 2020, 'single_sex_classes': {'Algebra I, Geometry, Algebra II': {'Male': 1, 'Female': 1, 'Total': 2}, 'Other Mathematics': {'Male': 1, 'Female': 1, 'Total': 2}, 'Science': {'Male': 1, 'Female': 1, 'Total': 2}, 'English, Reading, Language Arts': {'Male': 1,

### Load

In [ ]:
from neo4j import GraphDatabase

def insert_singlesexclass_data(session, data):
    cypher_query = """
    UNWIND $data AS schoolData
    // Create or find Jurisdiction
    MERGE (jurisdiction:Jurisdiction {name: schoolData.jurisdiction_name})
    // Create or find LEA
    MERGE (lea:LEA {ncesID: schoolData.lea_id, name: schoolData.lea_name})
    MERGE (jurisdiction)-[:INCLUDES]->(lea)
    // Create or find Institution
    MERGE (institution:Institution {ncesID: schoolData.institution_id})
        ON CREATE SET institution.name = schoolData.institution_name
    MERGE (lea)-[:INCLUDES]->(institution)

    // Create a single Intervention node for "Single Sex Classes"
    MERGE (intervention:Intervention {name: "Single Sex Classes", date: schoolData.year})

    WITH schoolData, institution, intervention

    // Iterate over each class type in the single_sex_classes object
    UNWIND keys(schoolData.single_sex_classes) AS classType
    WITH institution, intervention, classType, schoolData.single_sex_classes[classType] AS classDetails, schoolData.year AS year

    // Create an ASSIGNS relationship for each class type with properties for Male, Female, Total, and date
    MERGE (institution)-[assigns:ASSIGNS {date: year, type: classType}]->(intervention)
        ON CREATE SET assigns.male = classDetails.Male, assigns.female = classDetails.Female, assigns.total = classDetails.Total
    """
    session.run(cypher_query, {"data": data})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Here we take a subset for demonstration
test_data_singlesexclass = transformed_singlesexclass_data[:10]

# Execute the insertion for the AP data
with driver.session() as session:
    insert_singlesexclass_data(session, test_data_singlesexclass)

driver.close()

## CRDC Single Sex Athletics

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
singlesexath_path = './Single sex Athletics.csv'  # Update with the actual file path
singlesexath_df = pd.read_csv(singlesexath_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Single Sex Athletics Preview:")
print(singlesexath_df.head(), "\n")

Single Sex Athletics Preview:
  LEA_STATE LEA_STATE_NAME    LEAID                LEA_NAME  SCHID  \
0        AL        Alabama  0100002  Alabama Youth Services   1705   
1        AL        Alabama  0100002  Alabama Youth Services   1706   
2        AL        Alabama  0100002  Alabama Youth Services  99995   
3        AL        Alabama  0100005        Albertville City    870   
4        AL        Alabama  0100005        Albertville City    871   

                        SCH_NAME      COMBOKEY   JJ SCH_SSATHLETICS_IND  \
0  Wallace Sch - Mt Meigs Campus  010000201705  Yes                  -9   
1      Mcneel Sch - Vacca Campus  010000201706  Yes                  -9   
2                 Autauga Campus  010000299995  Yes                  -9   
3      Albertville Middle School  010000500870   No                  -9   
4        Albertville High School  010000500871   No                 Yes   

   SCH_SSSPORTS_M  SCH_SSSPORTS_F  TOT_SSSPORTS  SCH_SSTEAMS_M  SCH_SSTEAMS_F  \
0              -9

### Transform

In [ ]:
from collections import defaultdict

def transform_singlesexath_data(df, year):
    transformed_data = []
    for _, row in df.iterrows():
        # Define mappings for each category of athletics to its data columns
        athletics_mappings = {
            "Single Sex Sports": {"Male": "SCH_SSSPORTS_M", "Female": "SCH_SSSPORTS_F", "Total": "TOT_SSSPORTS"},
            "Single Sex Teams": {"Male": "SCH_SSTEAMS_M", "Female": "SCH_SSTEAMS_F", "Total": "TOT_SSTEAMS"},
            "Single Sex Participants": {"Male": "SCH_SSPART_M", "Female": "SCH_SSPART_F", "Total": "TOT_SSPART"}
        }

        athletics_data = {}

        # Process each category of single sex athletics
        for category, keys in athletics_mappings.items():
            male_enrollment = int(row.get(keys["Male"], 0))
            female_enrollment = int(row.get(keys["Female"], 0))
            total_enrollment = int(row.get(keys["Total"], 0))

            # Ensure we ignore -9, '-9', '', or any placeholder values
            male_enrollment = max(male_enrollment, 0) if male_enrollment != -9 else 0
            female_enrollment = max(female_enrollment, 0) if female_enrollment != -9 else 0
            total_enrollment = max(total_enrollment, 0) if total_enrollment != -9 else 0

            # Only include categories with meaningful data
            if male_enrollment > 0 or female_enrollment > 0 or total_enrollment > 0:
                athletics_data[category] = {
                    "Male": male_enrollment,
                    "Female": female_enrollment,
                    "Total": total_enrollment
                }

        # Add school data to the list if there is any athletics data
        if athletics_data:
            school_data = {
                "institution_id": row["COMBOKEY"],
                "institution_name": row["SCH_NAME"],
                "jurisdiction_name": row["LEA_STATE_NAME"],
                "lea_id": row["LEAID"],
                "lea_name": row["LEA_NAME"],
                "year": year,
                "athletics": athletics_data
            }
            transformed_data.append(school_data)

    return transformed_data

singlesexath_df_subset = singlesexath_df #.head(200)
transformed_singlesexath_data = transform_singlesexath_data(singlesexath_df_subset, 2020)
print(transformed_singlesexath_data[:5])

[{'institution_id': 10000500871, 'institution_name': 'Albertville High School', 'jurisdiction_name': 'Alabama', 'lea_id': 100005, 'lea_name': 'Albertville City', 'year': 2020, 'athletics': {'Single Sex Sports': {'Male': 2, 'Female': 2, 'Total': 4}, 'Single Sex Teams': {'Male': 5, 'Female': 4, 'Total': 9}, 'Single Sex Participants': {'Male': 70, 'Female': 60, 'Total': 130}}}, {'institution_id': 10000600872, 'institution_name': 'Asbury High School', 'jurisdiction_name': 'Alabama', 'lea_id': 100006, 'lea_name': 'Marshall County', 'year': 2020, 'athletics': {'Single Sex Sports': {'Male': 5, 'Female': 5, 'Total': 10}, 'Single Sex Teams': {'Male': 8, 'Female': 8, 'Total': 16}, 'Single Sex Participants': {'Male': 62, 'Female': 65, 'Total': 127}}}, {'institution_id': 10000600878, 'institution_name': 'Douglas High School', 'jurisdiction_name': 'Alabama', 'lea_id': 100006, 'lea_name': 'Marshall County', 'year': 2020, 'athletics': {'Single Sex Sports': {'Male': 9, 'Female': 10, 'Total': 19}, 'Sin

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

def insert_test_data(session, data):
    cypher_query = """
    UNWIND $data AS schoolData
    // Create or find Jurisdiction
    MERGE (jurisdiction:Jurisdiction {name: schoolData.jurisdiction_name})
    // Create or find LEA
    MERGE (lea:LEA {ncesID: schoolData.lea_id, name: schoolData.lea_name})
    MERGE (jurisdiction)-[:INCLUDES]->(lea)
    // Create or find Institution
    MERGE (institution:Institution {ncesID: schoolData.institution_id})
        ON CREATE SET institution.name = schoolData.institution_name
    MERGE (lea)-[:INCLUDES]->(institution)

    // Create a single Intervention node for "Single Sex Athletics"
    MERGE (intervention:Intervention {name: "Single Sex Athletics", date: schoolData.year})

    WITH schoolData, institution, intervention

    // Iterate over each athletics category
    UNWIND keys(schoolData.athletics) AS athleticsCategory
    WITH institution, intervention, athleticsCategory, schoolData.athletics[athleticsCategory] AS categoryDetails, schoolData.year AS year

    // Create an ASSIGNS relationship for each athletics type with properties for Male, Female, Total, and date
    MERGE (institution)-[assigns:ASSIGNS {date: year, type: athleticsCategory}]->(intervention)
        ON CREATE SET assigns.male = categoryDetails.Male, assigns.female = categoryDetails.Female, assigns.total = categoryDetails.Total
    """
    session.run(cypher_query, {"data": data})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Here we take a subset for demonstration
test_data = transformed_singlesexath_data

# Execute the insertion for the Singl Sex Athletics data
with driver.session() as session:
  for i in tqdm(range(6000, len(test_data) - 1, 1000)):
    sub_data = test_data [i:i+1000]
    insert_test_data(session, sub_data)

driver.close()

100%|██████████| 9/9 [26:19<00:00, 175.45s/it]


## CRDC ACT or SAT

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
satact_path = './SAT and ACT.csv'  # Update with the actual file path
satact_df = pd.read_csv(satact_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("SAT and ACT Preview:")
print(satact_df.head(), "\n")

SAT and ACT Preview:
  LEA_STATE LEA_STATE_NAME    LEAID                LEA_NAME  SCHID  \
0        AL        Alabama  0100002  Alabama Youth Services   1705   
1        AL        Alabama  0100002  Alabama Youth Services   1706   
2        AL        Alabama  0100002  Alabama Youth Services  99995   
3        AL        Alabama  0100005        Albertville City    870   
4        AL        Alabama  0100005        Albertville City    871   

                        SCH_NAME      COMBOKEY   JJ  SCH_SATACT_HI_M  \
0  Wallace Sch - Mt Meigs Campus  010000201705  Yes                0   
1      Mcneel Sch - Vacca Campus  010000201706  Yes                0   
2                 Autauga Campus  010000299995  Yes                0   
3      Albertville Middle School  010000500870   No               -9   
4        Albertville High School  010000500871   No               83   

   SCH_SATACT_HI_F  ...  SCH_SATACT_WH_M  SCH_SATACT_WH_F  SCH_SATACT_TR_M  \
0                0  ...                0       

### Transform

In [ ]:
from collections import defaultdict

def transform_satact_data(df, year):
    race_categories = {
        "HI": "Hispanic",
        "AM": "American Indian/Alaska Native",
        "AS": "Asian",
        "HP": "Native Hawaiian/Pacific Islander",
        "BL": "Black",
        "WH": "White",
        "TR": "Two or More Races",
        "LEP": "Limited English Proficiency",
        "IDEA": "Student With Disability (IDEA)"
    }

    transformed_data = []
    for _, row in df.iterrows():
        enrollments = {}
        total_students = {}  # Initialize without default values

        # Process total enrollments first
        total_students["Male"] = int(row.get("TOT_SATACT_M", 0))
        total_students["Female"] = int(row.get("TOT_SATACT_F", 0))
        total_students["Total"] = total_students["Male"] + total_students["Female"]

        for key, value in row.items():
            # Skip indicators, non-numeric values, and keys not related to CS enrollments
            if 'SATACT' not in key or value in (-9, '-9', '') or 'IND' in key or 'TOT_' in key:
                continue

            parts = key.split("_")
            # Ensure there are enough parts to get race code and gender
            if len(parts) < 4:
                continue

            race_code = parts[2]
            race_name = race_categories.get(race_code, "Unknown")
            gender = "Male" if parts[-1] == "M" else "Female"
            increment = int(value) if value not in ['-9', ''] else 0

            if increment == 0:
                continue

            subgroup = f"{race_name}"
            if subgroup not in enrollments:
                enrollments[subgroup] = {"Male": 0, "Female": 0, "Total": 0}

            # Instead of incrementing, directly assign the total values for Male and Female
            enrollments[subgroup][gender] = increment
            # Total per subgroup is calculated as the sum of Male and Female to ensure accuracy
            enrollments[subgroup]["Total"] = enrollments[subgroup]["Male"] + enrollments[subgroup]["Female"]

        if total_students["Total"] > 0:  # Ensure there's at least one student before adding to transformed data
            # Adding "Total Students" subgroup with calculated totals
            enrollments["Total Students"] = total_students

            school_data = {
                "institution_id": row["COMBOKEY"],
                "institution_name": row["SCH_NAME"],
                "jurisdiction_name": row["LEA_STATE_NAME"],
                "lea_id": row["LEAID"],
                "lea_name": row["LEA_NAME"],
                "year": year,
                "assessments": enrollments,
            }

            transformed_data.append(school_data)

    return transformed_data

satact_df_subset = satact_df.head(200)
transformed_satact_data = transform_satact_data(satact_df_subset, 2020)
print(transformed_satact_data[:5])

[{'institution_id': 10000299995, 'institution_name': 'Autauga Campus', 'jurisdiction_name': 'Alabama', 'lea_id': 100002, 'lea_name': 'Alabama Youth Services', 'year': 2020, 'assessments': {'White': {'Male': 2, 'Female': 0, 'Total': 2}, 'Total Students': {'Male': 2, 'Female': 0, 'Total': 2}}}, {'institution_id': 10000500871, 'institution_name': 'Albertville High School', 'jurisdiction_name': 'Alabama', 'lea_id': 100005, 'lea_name': 'Albertville City', 'year': 2020, 'assessments': {'Hispanic': {'Male': 83, 'Female': 99, 'Total': 182}, 'Asian': {'Male': 1, 'Female': 1, 'Total': 2}, 'Black': {'Male': 12, 'Female': 7, 'Total': 19}, 'White': {'Male': 91, 'Female': 82, 'Total': 173}, 'Two or More Races': {'Male': 2, 'Female': 6, 'Total': 8}, 'Limited English Proficiency': {'Male': 39, 'Female': 21, 'Total': 60}, 'Student With Disability (IDEA)': {'Male': 18, 'Female': 9, 'Total': 27}, 'Total Students': {'Male': 189, 'Female': 195, 'Total': 384}}}, {'institution_id': 10000600872, 'institution_

### Load

In [ ]:
from neo4j import GraphDatabase

def insert_satact_data(session, data):
    cypher_query = """
    UNWIND $data AS schoolData
    // Create or find Jurisdiction
    MERGE (jurisdiction:Jurisdiction {name: schoolData.jurisdiction_name})
    // Create or find LEA
    MERGE (lea:LEA {ncesID: schoolData.lea_id, name: schoolData.lea_name})
    MERGE (jurisdiction)-[:INCLUDES]->(lea)
    // Create or find Institution
    MERGE (institution:Institution {ncesID: schoolData.institution_id})
        ON CREATE SET institution.name = schoolData.institution_name
    MERGE (lea)-[:INCLUDES]->(institution)

    MERGE (assessment:Assessment {name: "SAT or ACT", date: schoolData.year})
    WITH institution, assessment, schoolData
    UNWIND keys(schoolData.assessments) AS subgroupName
    WITH institution, assessment, schoolData, subgroupName, schoolData.assessments[subgroupName] AS assessmentData
    MERGE (institution)-[assigns:ASSIGNS {date: schoolData.year, subgroup: subgroupName}]->(assessment)
        ON CREATE SET assigns.male = assessmentData.Male,
                      assigns.female = assessmentData.Female,
                      assigns.total = assessmentData.Total
    """
    session.run(cypher_query, {"data": data})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Here we take a subset for demonstration
test_data_satact = transformed_satact_data[:10]

# Execute the insertion for the chemistry data
with driver.session() as session:
    insert_satact_data(session, test_data_satact)

driver.close()

## CRDC Internet Access and Devices

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
internet_path = './Internet Access and Devices.csv'  # Update with the actual file path
internet_df = pd.read_csv(internet_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Internet Access and Devices Preview:")
print(internet_df.head(), "\n")

Internet Access and Devices Preview:
  LEA_STATE LEA_STATE_NAME    LEAID                LEA_NAME  SCHID  \
0        AL        Alabama  0100002  Alabama Youth Services   1705   
1        AL        Alabama  0100002  Alabama Youth Services   1706   
2        AL        Alabama  0100002  Alabama Youth Services  99995   
3        AL        Alabama  0100005        Albertville City    870   
4        AL        Alabama  0100005        Albertville City    871   

                        SCH_NAME      COMBOKEY   JJ SCH_INTERNET_FIBER  \
0  Wallace Sch - Mt Meigs Campus  010000201705  Yes                Yes   
1      Mcneel Sch - Vacca Campus  010000201706  Yes                Yes   
2                 Autauga Campus  010000299995  Yes                Yes   
3      Albertville Middle School  010000500870   No                Yes   
4        Albertville High School  010000500871   No                Yes   

  SCH_INTERNET_WIFI SCH_INTERNET_SCHDEV SCH_INTERNET_STUDDEV  \
0               Yes              

### Transform

In [ ]:
from collections import defaultdict

def transform_internet_data(df, year):
    transformed_data = []

    for _, row in df.iterrows():
        # Initialize variables
        fiber = 'Yes' if row["SCH_INTERNET_FIBER"] == "Yes" else 'No'
        wifi = 'Yes' if row["SCH_INTERNET_WIFI"] == "Yes" else 'No'
        school_device = 'Yes' if row["SCH_INTERNET_SCHDEV"] == "Yes" else 'No'
        student_device = 'Yes' if row["SCH_INTERNET_STUDDEV"] == "Yes" else 'No'
        wifi_devices = int(row["SCH_INTERNET_WIFIENDEV"]) if row["SCH_INTERNET_WIFIENDEV"] > 0 else 0

        school_data = {
            "institution_id": row["COMBOKEY"],
            "institution_name": row["SCH_NAME"],
            "jurisdiction_name": row["LEA_STATE_NAME"],
            "lea_id": row["LEAID"],
            "lea_name": row["LEA_NAME"],
            "fiber": fiber,
            "wifi": wifi,
            "school_device": school_device,
            "student_device": student_device,
            "num_wifi_devices": wifi_devices
        }

        transformed_data.append(school_data)

    return transformed_data

# Assuming internet_df is your DataFrame containing the internet data
transformed_internet_data = transform_internet_data(internet_df, 2020)
print(transformed_internet_data[:5])

[{'institution_id': '010000201705', 'institution_name': 'Wallace Sch - Mt Meigs Campus', 'jurisdiction_name': 'Alabama', 'lea_id': '0100002', 'lea_name': 'Alabama Youth Services', 'fiber': 'Yes', 'wifi': 'Yes', 'school_device': 'No', 'student_device': 'No', 'num_wifi_devices': 15}, {'institution_id': '010000201706', 'institution_name': 'Mcneel Sch - Vacca Campus', 'jurisdiction_name': 'Alabama', 'lea_id': '0100002', 'lea_name': 'Alabama Youth Services', 'fiber': 'Yes', 'wifi': 'Yes', 'school_device': 'No', 'student_device': 'No', 'num_wifi_devices': 30}, {'institution_id': '010000299995', 'institution_name': 'Autauga Campus', 'jurisdiction_name': 'Alabama', 'lea_id': '0100002', 'lea_name': 'Alabama Youth Services', 'fiber': 'Yes', 'wifi': 'Yes', 'school_device': 'No', 'student_device': 'No', 'num_wifi_devices': 31}, {'institution_id': '010000500870', 'institution_name': 'Albertville Middle School', 'jurisdiction_name': 'Alabama', 'lea_id': '0100005', 'lea_name': 'Albertville City', 'fi

In [ ]:
filtered_records = [record for record in transformed_internet_data if str(record['institution_id']) == "19SOP0200001" or "Broomfield High" in record['institution_name']]

for record in filtered_records:
    print(record)

{'institution_id': 80249000103, 'institution_name': 'Broomfield High School', 'jurisdiction_name': 'Colorado', 'lea_id': 802490, 'lea_name': 'Boulder Valley School District No. Re2', 'Internet Access and Devices': '2020, Fiber-optic connection at school, Wifi in every classroom, Students have take-home device for Internet access, 2359 student devices provided to students'}
{'institution_id': '19SOP0200001', 'institution_name': 'Iowa School For The Deaf', 'jurisdiction_name': 'Iowa', 'lea_id': '19SOP02', 'lea_name': 'Iowa School For The Deaf', 'Internet Access and Devices': '2020, Fiber-optic connection at school, Wifi in every classroom, Students have take-home device for Internet access, Students can use student-owned devices for Internet access, 165 student devices provided to students'}


### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

def insert_test_data(session, data):
    cypher_query = """
    UNWIND $data AS schoolData
    MERGE (institution:Institution {ncesID: schoolData.institution_id})
        ON MATCH SET
            institution.fiber = schoolData.fiber,
            institution.wifi = schoolData.wifi,
            institution.schoolDevice = schoolData.school_device,
            institution.studentDevice = schoolData.student_device,
            institution.numSchoolProvidedDevices = CASE WHEN schoolData.num_wifi_devices > 0 THEN schoolData.num_wifi_devices END
        ON CREATE SET institution.name = schoolData.institution_name,
            institution.fiber = schoolData.fiber,
            institution.wifi = schoolData.wifi,
            institution.schoolDevice = schoolData.school_device,
            institution.studentDevice = schoolData.student_device,
            institution.numSchoolProvidedDevices = CASE WHEN schoolData.num_wifi_devices > 0 THEN schoolData.num_wifi_devices END
    """
    session.run(cypher_query, {"data": data})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Here we take a subset for demonstration (add [:10])
test_data = transformed_internet_data[:10]

# Execute the insertion for the internet data
with driver.session() as session:
  for i in tqdm(range(0, len(test_data) - 1, 1000)):
    sub_data = test_data [i:i+1000]
    insert_test_data(session, sub_data)

driver.close()

100%|██████████| 1/1 [00:00<00:00,  4.44it/s]


## CRDC School Support

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
support_path = './School Support.csv'  # Update with the actual file path
support_df = pd.read_csv(support_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("School Support Preview:")
print(support_df.head(), "\n")

School Support Preview:
  LEA_STATE LEA_STATE_NAME    LEAID                LEA_NAME  SCHID  \
0        AL        Alabama  0100002  Alabama Youth Services   1705   
1        AL        Alabama  0100002  Alabama Youth Services   1706   
2        AL        Alabama  0100002  Alabama Youth Services  99995   
3        AL        Alabama  0100005        Albertville City    870   
4        AL        Alabama  0100005        Albertville City    871   

                        SCH_NAME      COMBOKEY   JJ  SCH_FTETEACH_TOT  \
0  Wallace Sch - Mt Meigs Campus  010000201705  Yes              24.0   
1      Mcneel Sch - Vacca Campus  010000201706  Yes              11.0   
2                 Autauga Campus  010000299995  Yes               3.0   
3      Albertville Middle School  010000500870   No              47.0   
4        Albertville High School  010000500871   No              86.0   

   SCH_FTETEACH_CERT  SCH_FTETEACH_NOTCERT  SCH_FTECOUNSELORS  \
0               24.0                   0.0         

### Transform

In [ ]:
from collections import defaultdict

def transform_support_data(df, year):
    transformed_data = []
    for index, row in df.iterrows():
        # Initialize the dictionary for the current row
        school_data = {
            "institution_id": row["COMBOKEY"],
            "institution_name": row["SCH_NAME"],
            "jurisdiction_name": row["LEA_STATE_NAME"],
            "lea_id": row["LEAID"],
            "lea_name": row["LEA_NAME"]
        }

        # List of roles to process, including their corresponding DataFrame column names
        roles = [
            ("SCH_FTESECURITY_LEO", "law_enforcement_officers"),
            ("SCH_FTESECURITY_GUA", "security_guards"),
            ("SCH_FTESERVICES_NUR", "nurses"),
            ("SCH_FTESERVICES_PSY", "psychologists"),
            ("SCH_FTESERVICES_SOC", "social_workers"),
            ("SCH_FTECOUNSELORS", "school_counselors"),
            ("SCH_FTETEACH_TOT", "fte_teachers")
        ]

        # Process each role and add to the dictionary if the count is greater than 0
        for role_key, role_property in roles:
            role_count = row.get(role_key, 0)  # Use 0 as default for missing data
            if role_count > 0:
                school_data[role_property] = role_count

        # Calculate the rate of certified teachers and add to the dictionary if applicable
        total_teachers = row.get("SCH_FTETEACH_TOT", 0)
        certified_teachers = row.get("SCH_FTETEACH_CERT", 0)
        if total_teachers > 0:
            cert_rate = (certified_teachers / total_teachers) * 100
            school_data["certificated_teacher_rate"] = f"{cert_rate:.2f}%"

        transformed_data.append(school_data)

    return transformed_data

support_df_subset = support_df.head(200)
transformed_support_data = transform_support_data(support_df_subset, 2020)
print(transformed_support_data[:5])

[{'institution_id': '010000201705', 'institution_name': 'Wallace Sch - Mt Meigs Campus', 'jurisdiction_name': 'Alabama', 'lea_id': '0100002', 'lea_name': 'Alabama Youth Services', 'school_counselors': 1.0, 'fte_teachers': 24.0, 'certificated_teacher_rate': '100.00%'}, {'institution_id': '010000201706', 'institution_name': 'Mcneel Sch - Vacca Campus', 'jurisdiction_name': 'Alabama', 'lea_id': '0100002', 'lea_name': 'Alabama Youth Services', 'school_counselors': 1.0, 'fte_teachers': 11.0, 'certificated_teacher_rate': '100.00%'}, {'institution_id': '010000299995', 'institution_name': 'Autauga Campus', 'jurisdiction_name': 'Alabama', 'lea_id': '0100002', 'lea_name': 'Alabama Youth Services', 'fte_teachers': 3.0, 'certificated_teacher_rate': '100.00%'}, {'institution_id': '010000500870', 'institution_name': 'Albertville Middle School', 'jurisdiction_name': 'Alabama', 'lea_id': '0100005', 'lea_name': 'Albertville City', 'law_enforcement_officers': 1.0, 'nurses': 1.0, 'school_counselors': 2.0

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

def insert_test_data(session, data):
    cypher_query = """
    UNWIND $data AS schoolData
    MERGE (institution:Institution {ncesID: schoolData.institution_id})
    ON CREATE SET
        institution.name = schoolData.institution_name,
        institution.lawEnforcementOfficers = CASE WHEN schoolData.law_enforcement_officers > 0 THEN schoolData.law_enforcement_officers END,
        institution.securityGuards = CASE WHEN schoolData.security_guards > 0 THEN schoolData.security_guards END,
        institution.nurses = CASE WHEN schoolData.nurses > 0 THEN schoolData.nurses END,
        institution.psychologists = CASE WHEN schoolData.psychologists > 0 THEN schoolData.psychologists END,
        institution.socialWorkers = CASE WHEN schoolData.social_workers > 0 THEN schoolData.social_workers END,
        institution.schoolCounselors = CASE WHEN schoolData.school_counselors > 0 THEN schoolData.school_counselors END,
        institution.fteTeachers = CASE WHEN schoolData.fte_teachers > 0 THEN schoolData.fte_teachers END,
        institution.certificatedTeacherRate = schoolData.certificated_teacher_rate
    ON MATCH SET
        institution.lawEnforcement_officers = CASE WHEN schoolData.law_enforcement_officers > 0 THEN schoolData.law_enforcement_officers END,
        institution.securityGuards = CASE WHEN schoolData.security_guards > 0 THEN schoolData.security_guards END,
        institution.nurses = CASE WHEN schoolData.nurses > 0 THEN schoolData.nurses END,
        institution.psychologists = CASE WHEN schoolData.psychologists > 0 THEN schoolData.psychologists END,
        institution.socialWorkers = CASE WHEN schoolData.social_workers > 0 THEN schoolData.social_workers END,
        institution.schoolCounselors = CASE WHEN schoolData.school_counselors > 0 THEN schoolData.school_counselors END,
        institution.fteTeachers = CASE WHEN schoolData.fte_teachers > 0 THEN schoolData.fte_teachers END,
        institution.certificatedTeacherRate = schoolData.certificated_teacher_rate

    """
    session.run(cypher_query, {"data": data})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Here we take a subset for demonstration
test_data = transformed_support_data

# Execute the insertion for the chemistry data
with driver.session() as session:
  for i in tqdm(range(0, len(test_data) - 1, 1000)):
    sub_data = test_data [i:i+1000]
    insert_test_data(session, sub_data)

driver.close()

100%|██████████| 98/98 [00:15<00:00,  6.33it/s]


## CRDC Suspensions

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
suspension_path = './Suspensions.csv'  # Update with the actual file path
suspension_df = pd.read_csv(suspension_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Suspension Preview:")
print(suspension_df.head(), "\n")

Suspension Preview:
  LEA_STATE LEA_STATE_NAME    LEAID                LEA_NAME  SCHID  \
0        AL        Alabama  0100002  Alabama Youth Services   1705   
1        AL        Alabama  0100002  Alabama Youth Services   1706   
2        AL        Alabama  0100002  Alabama Youth Services  99995   
3        AL        Alabama  0100005        Albertville City    870   
4        AL        Alabama  0100005        Albertville City    871   

                        SCH_NAME      COMBOKEY   JJ  SCH_PSDISC_OOMOOS_HI_M  \
0  Wallace Sch - Mt Meigs Campus  010000201705  Yes                      -9   
1      Mcneel Sch - Vacca Campus  010000201706  Yes                      -9   
2                 Autauga Campus  010000299995  Yes                      -9   
3      Albertville Middle School  010000500870   No                      -9   
4        Albertville High School  010000500871   No                      -9   

   SCH_PSDISC_OOMOOS_HI_F  ...  SCH_DAYSMISSED_TR_M  SCH_DAYSMISSED_TR_F  \
0       

### Transform

In [ ]:
from collections import defaultdict
import pandas as pd

# Suspension Types and corresponding prefix in column names
suspension_types = {
    "In-school Suspension": "ISS",
    "Out-of-school Suspension (Single)": "SINGOOS",
    "Out-of-school Suspension (Multiple)": "MULTOOS"
}

# Define a mapping for subgroup categories based on column name parts
subgroup_categories = {
    "HI": "Hispanic",
    "AM": "American Indian/Alaska Native",
    "AS": "Asian",
    "HP": "Native Hawaiian/Pacific Islander",
    "BL": "Black",
    "WH": "White",
    "TR": "Two or More Races",
    "LEP": "Limited English Proficiency",
    "IDEA": "Student (IDEA)",
    "504": "Student (504)",
    "ALL": "All Students"
}

# Contexts for the types of suspensions
contexts = {
    "PSDISC": "Preschool",
    "DISCWODIS": "Without Disability",
    "DISCWDIS": "With Disability"
}

def transform_suspension_data(df, year):
    transformed_data = []

    for _, row in df.iterrows():
        school_data = {
            "institution_id": row['COMBOKEY'],
            "institution_name": row['SCH_NAME'],
            "jurisdiction_name": row['LEA_STATE_NAME'],
            "lea_id": row['LEAID'],
            "lea_name": row['LEA_NAME'],
            "year": year,
            "suspensions": defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: {"Male": 0, "Female": 0, "Total": 0})))
        }

        has_positive_data = False  # Flag to track if any positive data exists for this school

        # Process each key in the row
        for key, value in row.items():
            if isinstance(value, (int, float)) and value > 0:  # Only process positive values
                parts = key.split("_")
                if parts[0] in ["TOT", "SCH"]:  # Valid entries start with TOT or SCH
                    if len(parts) > 1:
                        context_key = contexts.get(parts[1], None)  # Directly use the second part as the context key
                        if context_key:
                            suspension_key = next((s for s in suspension_types if suspension_types[s] in key), None)
                            if suspension_key:
                                # Subgroup logic based on context
                                # Handling totals specifically for 'Without Disability' in 'In-school Suspension'
                                if "TOT" == parts[0]:
                                    gender = "Male" if "M" in parts[-1] else "Female"
                                    school_data['suspensions'][suspension_key][context_key]["Total"][gender] += value
                                    school_data['suspensions'][suspension_key][context_key]["Total"]["Total"] += value
                                    has_positive_data = True
                                else:
                                    subgroup_code = parts[-2]  # Standard subgroup identification
                                    subgroup = subgroup_categories.get(subgroup_code, "Unknown")
                                    # IDEA or 504 as part of the subgroup if context is 'With Disability'
                                    if context_key == "With Disability" and "IDEA" in parts:
                                        subgroup += " (IDEA)"
                                    gender = "Male" if "_M" in key else "Female"
                                    school_data['suspensions'][suspension_key][context_key][subgroup][gender] += value
                                    school_data['suspensions'][suspension_key][context_key][subgroup]["Total"] += value
                                    has_positive_data = True


        if has_positive_data:
            # Convert defaultdict to regular dictionary for cleaner output
            for type_key, context_dict in school_data['suspensions'].items():
                for context_type, subgroup_dict in context_dict.items():
                    school_data['suspensions'][type_key][context_type] = dict(subgroup_dict)
                school_data['suspensions'][type_key] = dict(context_dict)
            school_data['suspensions'] = dict(school_data['suspensions'])
            transformed_data.append(school_data)  # Append only if there's positive data

    return transformed_data

suspension_df_subset = suspension_df #.head(200)

# Now, use this subset to transform the data
transformed_suspension_data = transform_suspension_data(suspension_df_subset, 2020)

# Optionally, print or inspect part of the transformed data to check its structure
print(transformed_suspension_data[:5])  # Print details of the first 5 schools for inspection

[{'institution_id': '010000500870', 'institution_name': 'Albertville Middle School', 'jurisdiction_name': 'Alabama', 'lea_id': '0100005', 'lea_name': 'Albertville City', 'year': 2020, 'suspensions': {'In-school Suspension': {'Without Disability': {'Hispanic': {'Male': 7, 'Female': 1, 'Total': 8}, 'American Indian/Alaska Native': {'Male': 1, 'Female': 0, 'Total': 1}, 'Black': {'Male': 2, 'Female': 0, 'Total': 2}, 'White': {'Male': 1, 'Female': 0, 'Total': 1}, 'Two or More Races': {'Male': 3, 'Female': 0, 'Total': 3}, 'Total': {'Male': 14, 'Female': 1, 'Total': 15}, 'Limited English Proficiency': {'Male': 6, 'Female': 1, 'Total': 7}}, 'With Disability': {'Hispanic (IDEA)': {'Male': 3, 'Female': 0, 'Total': 3}, 'White (IDEA)': {'Male': 2, 'Female': 0, 'Total': 2}, 'Total': {'Male': 5, 'Female': 0, 'Total': 5}, 'Limited English Proficiency': {'Male': 3, 'Female': 0, 'Total': 3}}}, 'Out-of-school Suspension (Single)': {'Without Disability': {'Hispanic': {'Male': 12, 'Female': 6, 'Total': 18

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

# Function to insert suspension data into Neo4j
def insert_test_data(session, data):
    cypher_query = """
    UNWIND $data AS schoolData
    MATCH (institution:Institution {ncesID: schoolData.institution_id})

    WITH schoolData, institution
    UNWIND ['In-school Suspension', 'Out-of-school Suspension (Single)', 'Out-of-school Suspension (Multiple)'] AS suspensionType
    MERGE (intervention:Intervention {name: suspensionType})
    WITH schoolData, institution, intervention, suspensionType

    // Process each context and subgroup to create unique ASSIGNS relationships
    UNWIND keys(schoolData.suspensions[suspensionType]) AS contextType
    UNWIND keys(schoolData.suspensions[suspensionType][contextType]) AS baseSubgroupName
    WITH institution, intervention, contextType, baseSubgroupName, schoolData.year AS year,
        schoolData.suspensions[suspensionType][contextType][baseSubgroupName] AS details

    // Simplify subgroup name construction by directly appending contextType
    WITH institution, intervention,
        (baseSubgroupName + ' ' + contextType) AS subgroupName,
        year, details

    MERGE (institution)-[assigns:ASSIGNS {subgroup: subgroupName, year: year}]->(intervention)
      ON CREATE SET
        assigns.male = details.Male,
        assigns.female = details.Female,
        assigns.total = details.Total
      ON MATCH SET
        assigns.male = details.Male,
        assigns.female = details.Female,
        assigns.total = details.Total
    """
    session.run(cypher_query, {"data": data})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Here we take a subset for demonstration
test_data = transformed_suspension_data

# Execute the insertion for the suspension data
with driver.session() as session:
#    insert_test_data(session, test_data)

  for i in tqdm(range(0, len(test_data) - 1, 1000)):
    sub_data = test_data [i:i+1000]
    insert_test_data(session, sub_data)

driver.close()

100%|██████████| 56/56 [01:50<00:00,  1.97s/it]


## CRDC Transfers

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
transfers_path = './Transfers.csv'  # Update with the actual file path
transfers_df = pd.read_csv(transfers_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Transfers Preview:")
print(transfers_df.head(), "\n")

Transfers Preview:
  LEA_STATE LEA_STATE_NAME    LEAID                LEA_NAME  SCHID  \
0        AL        Alabama  0100002  Alabama Youth Services   1705   
1        AL        Alabama  0100002  Alabama Youth Services   1706   
2        AL        Alabama  0100002  Alabama Youth Services  99995   
3        AL        Alabama  0100005        Albertville City    870   
4        AL        Alabama  0100005        Albertville City    871   

                        SCH_NAME      COMBOKEY   JJ  \
0  Wallace Sch - Mt Meigs Campus  010000201705  Yes   
1      Mcneel Sch - Vacca Campus  010000201706  Yes   
2                 Autauga Campus  010000299995  Yes   
3      Albertville Middle School  010000500870   No   
4        Albertville High School  010000500871   No   

   SCH_DISCWODIS_TFRALT_HI_M  SCH_DISCWODIS_TFRALT_HI_F  ...  \
0                         -9                         -9  ...   
1                         -9                         -9  ...   
2                         -9         

### Transform

In [ ]:
from collections import defaultdict

def transform_transfers_data(df, year):
    race_categories = {
        "HI": "Hispanic",
        "AM": "American Indian/Alaska Native",
        "AS": "Asian",
        "HP": "Native Hawaiian/Pacific Islander",
        "BL": "Black",
        "WH": "White",
        "TR": "Two or More Races",
        "LEP": "Limited English Proficiency",
        "504": "Section 504",
        "IDEA": "IDEA"
    }

    transformed_data = []
    for _, row in df.iterrows():
        enrollments = {}
        total_students_with_disabilities = {"Male": 0, "Female": 0, "Total": 0}
        total_students_without_disabilities = {"Male": 0, "Female": 0, "Total": 0}

        # Process total enrollments for students with and without disabilities
        total_students_with_disabilities["Male"] = int(row.get("TOT_DISCWDIS_TFRALT_IDEA_M", 0))
        total_students_with_disabilities["Female"] = int(row.get("TOT_DISCWDIS_TFRALT_IDEA_F", 0))
        total_students_with_disabilities["Total"] = total_students_with_disabilities["Male"] + total_students_with_disabilities["Female"]

        total_students_without_disabilities["Male"] = int(row.get("TOT_DISCWODIS_TFRALT_M", 0))
        total_students_without_disabilities["Female"] = int(row.get("TOT_DISCWODIS_TFRALT_F", 0))
        total_students_without_disabilities["Total"] = total_students_without_disabilities["Male"] + total_students_without_disabilities["Female"]

        # Ensure subgroups for both conditions
        for key, value in row.items():
            if 'TFRALT' in key and 'TOT_' not in key and value > 0:
                parts = key.split("_")
                if len(parts) < 5:
                    continue

                race_code = parts[-2]
                race_name = race_categories.get(race_code, "Unknown")
                gender = "Male" if parts[-1] == "M" else "Female"
                condition = "With Disabilities" if "DISCWDIS" in key else "Without Disabilities"
                subgroup = f"{race_name} {condition}"

                if subgroup not in enrollments:
                    enrollments[subgroup] = {"Male": 0, "Female": 0, "Total": 0}

                increment = int(value)
                enrollments[subgroup][gender] += increment
                enrollments[subgroup]["Total"] += increment

        # Adding "Total Students" subgroup with calculated totals for students with and without disabilities
        if total_students_with_disabilities["Total"] > 0:
            enrollments["Total Students With Disabilities"] = total_students_with_disabilities
        if total_students_without_disabilities["Total"] > 0:
            enrollments["Total Students Without Disabilities"] = total_students_without_disabilities

        if enrollments:  # Only include if there's at least one enrollment
            school_data = {
                "institution_id": row["COMBOKEY"],
                "institution_name": row["SCH_NAME"],
                "jurisdiction_name": row["LEA_STATE_NAME"],
                "lea_id": row["LEAID"],
                "lea_name": row["LEA_NAME"],
                "year": year,
                "enrollments": enrollments
            }
            transformed_data.append(school_data)

    return transformed_data


#transformed_transfers_data = transform_transfers_data(transfers_df, 2020)

transfers_df_subset = transfers_df.head(200)

# Now, use this subset to transform the data
transformed_transfers_data = transform_transfers_data(transfers_df_subset, 2020)

# Optionally, print or inspect part of the transformed data to check its structure
print(transformed_transfers_data[:5])  # Print details of the first 5 schools for inspection

[{'institution_id': 10000600193, 'institution_name': 'Kate Duncan Smith Dar Middle', 'jurisdiction_name': 'Alabama', 'lea_id': 100006, 'lea_name': 'Marshall County', 'year': 2020, 'enrollments': {'White With Disabilities': {'Male': 2, 'Female': 0, 'Total': 2}, 'Total Students With Disabilities': {'Male': 2, 'Female': 0, 'Total': 2}}}, {'institution_id': 10000600872, 'institution_name': 'Asbury High School', 'jurisdiction_name': 'Alabama', 'lea_id': 100006, 'lea_name': 'Marshall County', 'year': 2020, 'enrollments': {'White Without Disabilities': {'Male': 1, 'Female': 0, 'Total': 1}, 'Total Students Without Disabilities': {'Male': 1, 'Female': 0, 'Total': 1}}}, {'institution_id': 10000600883, 'institution_name': 'Kate D Smith Dar High School', 'jurisdiction_name': 'Alabama', 'lea_id': 100006, 'lea_name': 'Marshall County', 'year': 2020, 'enrollments': {'White Without Disabilities': {'Male': 0, 'Female': 1, 'Total': 1}, 'White With Disabilities': {'Male': 1, 'Female': 0, 'Total': 1}, 'To

In [ ]:
institution_id_to_find = int('010177000611')
for record in transformed_transfers_data:
    if record["institution_id"] == institution_id_to_find:
        print(record)
        break
else:
    print("Institution not found.")

{'institution_id': 10177000611, 'institution_name': 'Ashford High School', 'jurisdiction_name': 'Alabama', 'lea_id': 101770, 'lea_name': 'Houston County', 'year': 2020, 'enrollments': {'Hispanic Without Disabilities': {'Male': 1, 'Female': 1, 'Total': 2}, 'Black Without Disabilities': {'Male': 8, 'Female': 7, 'Total': 15}, 'White Without Disabilities': {'Male': 22, 'Female': 14, 'Total': 36}, 'Two or More Races Without Disabilities': {'Male': 1, 'Female': 0, 'Total': 1}, 'Limited English Proficiency Without Disabilities': {'Male': 1, 'Female': 0, 'Total': 1}, 'Hispanic With Disabilities': {'Male': 1, 'Female': 1, 'Total': 2}, 'Black With Disabilities': {'Male': 9, 'Female': 2, 'Total': 11}, 'White With Disabilities': {'Male': 14, 'Female': 6, 'Total': 20}, 'Two or More Races With Disabilities': {'Male': 1, 'Female': 0, 'Total': 1}, 'Section 504 With Disabilities': {'Male': 1, 'Female': 1, 'Total': 2}, 'Total Students With Disabilities': {'Male': 25, 'Female': 9, 'Total': 34}, 'Total St

### Load

In [ ]:
from neo4j import GraphDatabase

# Function to insert suspension data into Neo4j
def insert_transfers_data(session, data):
    cypher_query = """
    UNWIND $data AS schoolData
    // Create or find Jurisdiction
    MERGE (jurisdiction:Jurisdiction {name: schoolData.jurisdiction_name})
    // Create or find LEA
    MERGE (lea:LEA {ncesID: schoolData.lea_id, name: schoolData.lea_name})
    MERGE (jurisdiction)-[:INCLUDES]->(lea)
    // Create or find Institution
    MERGE (institution:Institution {ncesID: schoolData.institution_id, name: schoolData.institution_name})
    ON CREATE SET institution.name = schoolData.institution_name
    MERGE (lea)-[:INCLUDES]->(institution)

    // Define the Intervention
    MERGE (intervention:Intervention {name: "Transferred to an Alternative School for Disciplinary Reasons", date: schoolData.year})
    WITH institution, intervention, schoolData
    UNWIND keys(schoolData.enrollments) AS subgroupName
    WITH institution, intervention, schoolData, subgroupName, schoolData.enrollments[subgroupName] AS enrollmentData

    // Create a unique ASSIGN relationship for each subgroup
    MERGE (institution)-[assigns:ASSIGNS {date: schoolData.year, subgroup: subgroupName }]->(intervention)
        ON CREATE SET assigns.male = enrollmentData.Male, assigns.female = enrollmentData.Female, assigns.total = enrollmentData.Total
        ON MATCH SET assigns.male = enrollmentData.Male, assigns.female = enrollmentData.Female, assigns.total = enrollmentData.Total
    """
    session.run(cypher_query, {"data": data})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Assuming your transformed suspension data is stored in a variable named transformed_suspension_data
# Here we take a subset for demonstration
test_data_transfers = transformed_transfers_data[:10]

# Execute the insertion for the suspension data
with driver.session() as session:
    insert_transfers_data(session, test_data_transfers)

driver.close()

## CRDC Expulsions

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
expulsions_path = './Expulsions.csv'  # Update with the actual file path
expulsions_df = pd.read_csv(expulsions_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Expulsions Preview:")
print(expulsions_df.head(), "\n")

Expulsions Preview:
  LEA_STATE LEA_STATE_NAME    LEAID                LEA_NAME  SCHID  \
0        AL        Alabama  0100002  Alabama Youth Services   1705   
1        AL        Alabama  0100002  Alabama Youth Services   1706   
2        AL        Alabama  0100002  Alabama Youth Services  99995   
3        AL        Alabama  0100005        Albertville City    870   
4        AL        Alabama  0100005        Albertville City    871   

                        SCH_NAME      COMBOKEY   JJ  SCH_PSDISC_EXP_HI_M  \
0  Wallace Sch - Mt Meigs Campus  010000201705  Yes                   -9   
1      Mcneel Sch - Vacca Campus  010000201706  Yes                   -9   
2                 Autauga Campus  010000299995  Yes                   -9   
3      Albertville Middle School  010000500870   No                   -9   
4        Albertville High School  010000500871   No                   -9   

   SCH_PSDISC_EXP_HI_F  ...  SCH_DISCWDIS_EXPZT_IDEA_WH_M  \
0                   -9  ...              

### Transform

In [ ]:
from collections import defaultdict

def transform_expulsions_data(df, year):
    race_categories = {
        "HI": "Hispanic",
        "AM": "American Indian/Alaska Native",
        "AS": "Asian",
        "HP": "Native Hawaiian/Pacific Islander",
        "BL": "Black",
        "WH": "White",
        "TR": "Two or More Races",
        "LEP": "Limited English Proficiency",
        "IDEA": "IDEA",
        "504": "Section 504"
    }

    expulsion_types = {
        "Expulsions with Educational Services": "EXPWE",
        "Expulsions without Educational Services": "EXPWOE",
        "Expulsions under Zero Tolerance Policies": "EXPZT"
    }

    transformed_data = []
    for _, row in df.iterrows():
        enrollments = {etype: {} for etype in expulsion_types.keys()}

        # Reset totals for each expulsion type
        totals = {etype: {"Male": 0, "Female": 0, "Total": 0} for etype in expulsion_types.keys()}

        for key, value in row.items():
            if 'EXP' not in key or value in ['-9', -9, ''] or 'TOT_' in key:
                continue

            increment = max(int(value), 0)
            if increment == 0:
                continue

            for etype, prefix in expulsion_types.items():
                if prefix in key:
                    parts = key.split("_")
                    race_code = parts[-2]
                    gender = "Male" if "M" in parts[-1] else "Female"
                    race_name = race_categories.get(race_code, "Unknown")
                    condition = "With Disabilities" if "DISCWDIS" in key else "Without Disabilities"
                    subgroup = f"{race_name} {condition}".strip()

                    if subgroup not in enrollments[etype]:
                        enrollments[etype][subgroup] = {"Male": 0, "Female": 0, "Total": 0}

                    enrollments[etype][subgroup][gender] += increment
                    enrollments[etype][subgroup]["Total"] += increment

                    # Update totals for each expulsion type
                    totals[etype][gender] += increment
                    totals[etype]["Total"] += increment
                    break

        # Assign calculated totals to each expulsion type
        for etype in totals:
            if etype in enrollments:
                enrollments[etype]["Total"] = totals[etype]

        # Cleanup: Remove subgroups and types with 0 values
        for etype in list(enrollments.keys()):
            for subgroup in list(enrollments[etype].keys()):
                if all(value == 0 for value in enrollments[etype][subgroup].values()):
                    del enrollments[etype][subgroup]
            if not enrollments[etype]:  # If no subgroups have data, remove the expulsion type
                del enrollments[etype]

        if enrollments:  # Add school data if there's any expulsion data
            school_data = {
                "institution_id": row["COMBOKEY"],
                "institution_name": row["SCH_NAME"],
                "jurisdiction_name": row["LEA_STATE_NAME"],
                "lea_id": row["LEAID"],
                "lea_name": row["LEA_NAME"],
                "year": year,
                "expulsions": enrollments
            }
            transformed_data.append(school_data)

    return transformed_data


#transformed_expulsions_data = transform_expulsions_data(expulsions_df, 2020)

expulsions_df_subset = expulsions_df.head(200)

# Now, use this subset to transform the data
transformed_expulsions_data = transform_expulsions_data(expulsions_df_subset, 2020)

# Optionally, print or inspect part of the transformed data to check its structure
print(transformed_expulsions_data[:5])  # Print details of the first 5 schools for inspection

[{'institution_id': '010000600193', 'institution_name': 'Kate Duncan Smith Dar Middle', 'jurisdiction_name': 'Alabama', 'lea_id': '0100006', 'lea_name': 'Marshall County', 'year': 2020, 'expulsions': {'Expulsions with Educational Services': {'White With Disabilities': {'Male': 2, 'Female': 0, 'Total': 2}, 'Total': {'Male': 2, 'Female': 0, 'Total': 2}}}}, {'institution_id': '010000600872', 'institution_name': 'Asbury High School', 'jurisdiction_name': 'Alabama', 'lea_id': '0100006', 'lea_name': 'Marshall County', 'year': 2020, 'expulsions': {'Expulsions with Educational Services': {'White Without Disabilities': {'Male': 1, 'Female': 0, 'Total': 1}, 'Total': {'Male': 1, 'Female': 0, 'Total': 1}}}}, {'institution_id': '010000600883', 'institution_name': 'Kate D Smith Dar High School', 'jurisdiction_name': 'Alabama', 'lea_id': '0100006', 'lea_name': 'Marshall County', 'year': 2020, 'expulsions': {'Expulsions with Educational Services': {'White Without Disabilities': {'Male': 0, 'Female': 1

### Load

In [ ]:
from neo4j import GraphDatabase

# Function to insert suspension data into Neo4j
def insert_expulsions_data(session, data):
    cypher_query = """
    UNWIND $data AS schoolData
    // Create or find Jurisdiction
    MERGE (jurisdiction:Jurisdiction {name: schoolData.jurisdiction_name})
    // Create or find LEA
    MERGE (lea:LEA {ncesID: schoolData.lea_id, name: schoolData.lea_name})
    MERGE (jurisdiction)-[:INCLUDES]->(lea)
    // Create or find Institution
    MERGE (institution:Institution {ncesID: schoolData.institution_id, name: schoolData.institution_name})
    ON CREATE SET institution.name = schoolData.institution_name
    MERGE (lea)-[:INCLUDES]->(institution)

    WITH schoolData, institution
    UNWIND keys(schoolData.expulsions) AS expulsionType
    WITH institution, expulsionType, schoolData.expulsions[expulsionType] AS expulsionDetails, schoolData // Include schoolData here
    MERGE (intervention:Intervention {name: expulsionType, date: schoolData.year})

    WITH institution, intervention, expulsionDetails, schoolData // Include schoolData here again
    // Exclude the "Total" key for processing individual subgroups
    UNWIND keys(expulsionDetails) AS subgroupName
    WITH institution, intervention, subgroupName, expulsionDetails[subgroupName] AS enrollmentData, schoolData // And here

    // Create a unique ASSIGN relationship for each subgroup
    MERGE (institution)-[assigns:ASSIGNS {date: schoolData.year, subgroup: subgroupName}]->(intervention)
        ON CREATE SET assigns.male = enrollmentData.Male, assigns.female = enrollmentData.Female, assigns.total = enrollmentData.Total
        ON MATCH SET assigns.male = enrollmentData.Male, assigns.female = enrollmentData.Female, assigns.total = enrollmentData.Total

    """
    session.run(cypher_query, {"data": data})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Assuming your transformed suspension data is stored in a variable named transformed_suspension_data
# Here we take a subset for demonstration
test_data_expulsions = transformed_expulsions_data[:10]

# Execute the insertion for the suspension data
with driver.session() as session:
    insert_expulsions_data(session, test_data_expulsions)

driver.close()

## CRDC Restraint and Seclusion

### Extract

In [ ]:
import pandas as pd

rs_path = './Restraint and Seclusion.csv'  # Update with the actual file path
rs_df = pd.read_csv(rs_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Restraint and Seclusion Preview:")
print(rs_df.head(), "\n")

Restraint and Seclusion Preview:
  LEA_STATE LEA_STATE_NAME    LEAID                LEA_NAME  SCHID  \
0        AL        Alabama  0100002  Alabama Youth Services   1705   
1        AL        Alabama  0100002  Alabama Youth Services   1706   
2        AL        Alabama  0100002  Alabama Youth Services  99995   
3        AL        Alabama  0100005        Albertville City    870   
4        AL        Alabama  0100005        Albertville City    871   

                        SCH_NAME      COMBOKEY   JJ  SCH_RS_NONIDEA_MECH_HI_M  \
0  Wallace Sch - Mt Meigs Campus  010000201705  Yes                         0   
1      Mcneel Sch - Vacca Campus  010000201706  Yes                         0   
2                 Autauga Campus  010000299995  Yes                         0   
3      Albertville Middle School  010000500870   No                         0   
4        Albertville High School  010000500871   No                         0   

   SCH_RS_NONIDEA_MECH_HI_F  ...  SCH_RS_IDEA_SECL_LEP_F  \

### Transform

In [ ]:
from collections import defaultdict

def transform_rs_data(df, year):
    race_categories = {
        "HI": "Hispanic",
        "AM": "American Indian/Alaska Native",
        "AS": "Asian",
        "HP": "Native Hawaiian/Pacific Islander",
        "BL": "Black",
        "WH": "White",
        "TR": "Two or More Races",
        "LEP": "Limited English Proficiency",
        "504": "Section 504",
    }

    intervention_types = {
        "MECH": "Mechanical Restraint",
        "PHYS": "Physical Restraint",
        "SECL": "Seclusion",
    }

    transformed_data = []
    for _, row in df.iterrows():
        instances = {}
        students = {intervention: {"Non-IDEA": {}, "With Disabilities (IDEA)": {}} for intervention in intervention_types.values()}
        school_has_valid_data = False

        # Instances - populate only non-zero values
        for intervention, intervention_name in intervention_types.items():
            instances_without_disabilities = int(row.get(f"SCH_RSINSTANCES_{intervention}_WODIS", 0))
            instances_with_idea = int(row.get(f"SCH_RSINSTANCES_{intervention}_IDEA", 0))
            instances_with_504 = int(row.get(f"SCH_RSINSTANCES_{intervention}_504", 0))

            if instances_without_disabilities > 0:
                instances.setdefault(intervention_name, {})["Without Disabilities"] = instances_without_disabilities
            if instances_with_idea > 0:
                instances.setdefault(intervention_name, {})["With Disabilities (IDEA)"] = instances_with_idea
            if instances_with_504 > 0:
                instances.setdefault(intervention_name, {})["WitH Disabilities (504)"] = instances_with_504

        # Students - aggregate by NONIDEA (including 504) and IDEA, only non-zero values
        for key, value in row.items():
            if 'RS_' in key and value > 0 and 'INSTANCES' not in key and 'TOT_' not in key:
                parts = key.split("_")
                intervention_prefix = parts[3][:4]  # MECH, PHYS, or SECL
                intervention_name = intervention_types.get(intervention_prefix)
                if not intervention_name: continue

                disability_status = "Non-IDEA" if "NONIDEA" in key else "With Disabilities (IDEA)"
                race_code = parts[-2]
                race_name = race_categories.get(race_code, "Unknown")
                gender = "Male" if "M" in parts[-1] else "Female"

                # Ensure race_name and gender counts are initialized
                students[intervention_name][disability_status].setdefault(race_name, {"Male": 0, "Female": 0, "Total": 0})
                students[intervention_name][disability_status][race_name][gender] += value
                students[intervention_name][disability_status][race_name]["Total"] += value
                school_has_valid_data = True

        # Cleanup: Remove empty categories and subgroups
        for intervention in list(students.keys()):
            for status in list(students[intervention].keys()):
                for race in list(students[intervention][status].keys()):
                    if students[intervention][status][race]["Total"] == 0:
                        del students[intervention][status][race]
                if not students[intervention][status]:
                    del students[intervention][status]
            if not students[intervention]:
                del students[intervention]

        if school_has_valid_data:
            for intervention in students:
                for disability_code in ['NONIDEA', 'IDEA']:
                    disability_status = "Non-IDEA" if disability_code == "NONIDEA" else "With Disabilities (IDEA)"

                    total_male_key = f"TOT_RS_{disability_code}_{intervention[:4].upper()}_M"
                    total_female_key = f"TOT_RS_{disability_code}_{intervention[:4].upper()}_F"
                    total_male = int(row.get(total_male_key, 0))
                    total_female = int(row.get(total_female_key, 0))
                    total = total_male + total_female

                    # Ensure the "Total" key exists for each disability status within an intervention
                    if total > 0:
                        students[intervention][disability_status].setdefault("Total", {})
                        students[intervention][disability_status]["Total"]["Male"] = total_male
                        students[intervention][disability_status]["Total"]["Female"] = total_female
                        students[intervention][disability_status]["Total"]["Total"] = total

            school_data = {
                "institution_id": row["COMBOKEY"],
                "institution_name": row["SCH_NAME"],
                "jurisdiction_name": row["LEA_STATE_NAME"],
                "lea_id": row["LEAID"],
                "lea_name": row["LEA_NAME"],
                "year": year,
                "instances": instances,
                "studentsAssigned": students,
            }
            transformed_data.append(school_data)

    # Filter out schools without any instances or students data
    transformed_data = [data for data in transformed_data if data["instances"] or data["studentsAssigned"]]

    return transformed_data


#transformed_rs_data = transform_rs_data(rs_df, 2020)

rs_df_subset = rs_df.head(200)

# Now, use this subset to transform the data
transformed_rs_data = transform_rs_data(rs_df_subset, 2020)

# Optionally, print or inspect part of the transformed data to check its structure
print(transformed_rs_data[:5])  # Print details of the first 5 schools for inspection

[{'institution_id': 10000501616, 'institution_name': 'Albertville Kindergarten And Prek', 'jurisdiction_name': 'Alabama', 'lea_id': 100005, 'lea_name': 'Albertville City', 'year': 2020, 'instances': {'Physical Restraint': {'Without Disabilities': 2}}, 'studentsAssigned': {'Physical Restraint': {'Non-IDEA': {'Black': {'Male': 2, 'Female': 0, 'Total': 2}, 'Total': {'Male': 2, 'Female': 0, 'Total': 2}}}}}, {'institution_id': 10000600880, 'institution_name': 'Brindlee Mountain Elementary School', 'jurisdiction_name': 'Alabama', 'lea_id': 100006, 'lea_name': 'Marshall County', 'year': 2020, 'instances': {'Physical Restraint': {'With Disabilities (IDEA)': 4}}, 'studentsAssigned': {'Physical Restraint': {'With Disabilities (IDEA)': {'White': {'Male': 1, 'Female': 0, 'Total': 1}, 'Total': {'Male': 1, 'Female': 0, 'Total': 1}}}}}, {'institution_id': 10000600887, 'institution_name': 'Brindlee Mountain Primary School', 'jurisdiction_name': 'Alabama', 'lea_id': 100006, 'lea_name': 'Marshall County

### Load

In [ ]:
from neo4j import GraphDatabase

# Function to insert suspension data into Neo4j
def insert_rs_data(session, data):
    cypher_query = """
    UNWIND $data AS schoolData
    // Create or find Jurisdiction, LEA, and Institution
    MERGE (jurisdiction:Jurisdiction {name: schoolData.jurisdiction_name})
    MERGE (lea:LEA {ncesID: schoolData.lea_id, name: schoolData.lea_name})
    MERGE (jurisdiction)-[:INCLUDES]->(lea)
    MERGE (institution:Institution {ncesID: schoolData.institution_id, name: schoolData.institution_name})
    MERGE (lea)-[:INCLUDES]->(institution)

    // Create ASSIGNS relationships for each intervention and subgroup in 'studentsAssigned'.
    WITH schoolData, institution
    UNWIND keys(schoolData.studentsAssigned) AS interventionName
    MERGE (intervention:Intervention {name: interventionName, date: schoolData.year})
    WITH schoolData, institution, intervention, interventionName
    UNWIND keys(schoolData.studentsAssigned[interventionName]) AS disabilityStatus
    UNWIND keys(schoolData.studentsAssigned[interventionName][disabilityStatus]) AS subgroupName
    WITH institution, intervention, disabilityStatus, subgroupName, schoolData.studentsAssigned[interventionName][disabilityStatus][subgroupName] AS studentDetails, schoolData
    MERGE (institution)-[r:ASSIGNS {date: schoolData.year, subgroup: (subgroupName + ' ' + disabilityStatus)}]->(intervention)
    ON CREATE SET r.male = studentDetails.Male, r.female = studentDetails.Female, r.total = studentDetails.Total


    // Now, add instance data for "Total With Disabilities (IDEA)" and "Total Non-IDEA" relationships
    WITH institution, intervention, schoolData
    MATCH (institution)-[r:ASSIGNS]->(intervention)
    WHERE r.subgroup ENDS WITH "With Disabilities (IDEA)" OR r.subgroup ENDS WITH "Non-IDEA"
    SET r.instancesIDEA = CASE WHEN schoolData.instances[intervention.name]["With Disabilities (IDEA)"] IS NOT NULL AND schoolData.instances[intervention.name]["With Disabilities (IDEA)"] > 0 THEN schoolData.instances[intervention.name]["With Disabilities (IDEA)"] ELSE null END,
        r.instances504 = CASE WHEN schoolData.instances[intervention.name]["With Disabilities (504)"] IS NOT NULL AND schoolData.instances[intervention.name]["With Disabilities (504)"] > 0 THEN schoolData.instances[intervention.name]["With Disabilities (504)"] ELSE null END,
        r.instancesWithoutDisability = CASE WHEN schoolData.instances[intervention.name]["Without Disabilities"] IS NOT NULL AND schoolData.instances[intervention.name]["Without Disabilities"] > 0 THEN schoolData.instances[intervention.name]["Without Disabilities"] ELSE null END
    """
    session.run(cypher_query, {"data": data})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Assuming your transformed suspension data is stored in a variable named transformed_suspension_data
# Here we take a subset for demonstration
test_data_rs = transformed_rs_data[:10]

# Execute the insertion for the suspension data
with driver.session() as session:
    insert_rs_data(session, test_data_rs)

driver.close()

## CRDC Referrals and Arrests

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
referarrest_path = './Referrals and Arrests.csv'  # Update with the actual file path
referarrest_df = pd.read_csv(referarrest_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Referrals and Arrests Preview:")
print(referarrest_df.head(), "\n")

Referrals and Arrests Preview:
  LEA_STATE LEA_STATE_NAME    LEAID                LEA_NAME  SCHID  \
0        AL        Alabama  0100002  Alabama Youth Services   1705   
1        AL        Alabama  0100002  Alabama Youth Services   1706   
2        AL        Alabama  0100002  Alabama Youth Services  99995   
3        AL        Alabama  0100005        Albertville City    870   
4        AL        Alabama  0100005        Albertville City    871   

                        SCH_NAME      COMBOKEY   JJ  SCH_DISCWODIS_REF_HI_M  \
0  Wallace Sch - Mt Meigs Campus  010000201705  Yes                      -9   
1      Mcneel Sch - Vacca Campus  010000201706  Yes                      -9   
2                 Autauga Campus  010000299995  Yes                      -9   
3      Albertville Middle School  010000500870   No                       0   
4        Albertville High School  010000500871   No                       0   

   SCH_DISCWODIS_REF_HI_F  ...  SCH_DISCWDIS_ARR_IDEA_WH_M  \
0          

### Transform

In [ ]:
from collections import defaultdict

def transform_referarrest_data(df, year):
    race_categories = {
        "HI": "Hispanic",
        "AM": "American Indian/Alaska Native",
        "AS": "Asian",
        "HP": "Native Hawaiian/Pacific Islander",
        "BL": "Black",
        "WH": "White",
        "TR": "Two or More Races",
        "LEP": "Limited English Proficiency",
        "IDEA": "IDEA",
        "504": "Section 504"
    }

    referral_arrest_types = {
        "Referral to Law Enforcement": "REF",
        "School Related Arrest": "ARR"
    }

    transformed_data = []
    for _, row in df.iterrows():
        interventions = {itype: {"With Disabilities": {}, "Without Disabilities": {}} for itype in referral_arrest_types.keys()}

        # Directly use TOT_ fields for totals
        for itype, prefix in referral_arrest_types.items():
            total_with_disabilities_male = max(int(row.get(f"TOT_DISCWDIS_{prefix}_M", 0)), 0)
            total_with_disabilities_female = max(int(row.get(f"TOT_DISCWDIS_{prefix}_F", 0)), 0)
            total_without_disabilities_male = max(int(row.get(f"TOT_DISCWODIS_{prefix}_M", 0)), 0)
            total_without_disabilities_female = max(int(row.get(f"TOT_DISCWODIS_{prefix}_F", 0)), 0)

            if total_with_disabilities_male + total_with_disabilities_female > 0:
                interventions[itype]["With Disabilities"]["Total"] = {
                    "Male": total_with_disabilities_male,
                    "Female": total_with_disabilities_female,
                    "Total": total_with_disabilities_male + total_with_disabilities_female
                }

            if total_without_disabilities_male + total_without_disabilities_female > 0:
                interventions[itype]["Without Disabilities"]["Total"] = {
                    "Male": total_without_disabilities_male,
                    "Female": total_without_disabilities_female,
                    "Total": total_without_disabilities_male + total_without_disabilities_female
                }

        # Populate subgroups excluding TOT_ categories and negative values
        for key, value in row.items():
            if value in ['-9', -9, ''] or 'TOT_' in key or 'DISC' not in:
                continue

            increment = max(int(value), 0)
            if increment == 0:
                continue

            for itype, prefix in referral_arrest_types.items():
                if prefix in key and 'DISC' in key:
                    parts = key.split("_")
                    race_code = parts[-2]
                    gender = "Male" if "M" in parts[-1] else "Female"
                    race_name = race_categories.get(race_code, "Unknown")
                    condition = "With Disabilities" if "DISCWDIS" in key else "Without Disabilities"
                    subgroup = f"{race_name}"

                    if condition == "With Disabilities" and subgroup not in interventions[itype][condition]:
                        interventions[itype][condition][subgroup] = {"Male": 0, "Female": 0, "Total": 0}
                    elif condition == "Without Disabilities" and subgroup not in interventions[itype][condition]:
                        interventions[itype][condition][subgroup] = {"Male": 0, "Female": 0, "Total": 0}

                    interventions[itype][condition][subgroup][gender] += increment
                    interventions[itype][condition][subgroup]["Total"] += increment

        # Cleanup: Remove empty categories and subgroups
        for itype in list(interventions.keys()):
            for condition in list(interventions[itype].keys()):
                for subgroup in list(interventions[itype][condition].keys()):
                    if interventions[itype][condition][subgroup]["Total"] == 0:
                        del interventions[itype][condition][subgroup]
                if not interventions[itype][condition]:
                    del interventions[itype][condition]
            if not interventions[itype]:
                del interventions[itype]

        if interventions:  # Only include schools with data
            school_data = {
                "institution_id": row["COMBOKEY"],
                "institution_name": row["SCH_NAME"],
                "jurisdiction_name": row["LEA_STATE_NAME"],
                "lea_id": row["LEAID"],
                "lea_name": row["LEA_NAME"],
                "year": year,
                "interventions": interventions
            }
            transformed_data.append(school_data)

    return transformed_data

#transformed_referarrest_data = transform_referarrest_data(referarrest_df, 2020)

referarrest_df_subset = referarrest_df.head(200)

# Now, use this subset to transform the data
transformed_referarrest_data = transform_referarrest_data(referarrest_df_subset, 2020)

# Optionally, print or inspect part of the transformed data to check its structure
print(transformed_referarrest_data[:15])  # Print details of the first 5 schools for inspection

[{'institution_id': 10000600878, 'institution_name': 'Douglas High School', 'jurisdiction_name': 'Alabama', 'lea_id': 100006, 'lea_name': 'Marshall County', 'year': 2020, 'interventions': {'Referral to Law Enforcement': {'Without Disabilities': {'Total': {'Male': 1, 'Female': 0, 'Total': 1}, 'Hispanic': {'Male': 1, 'Female': 0, 'Total': 1}, 'Limited English Proficiency': {'Male': 1, 'Female': 0, 'Total': 1}}}}}, {'institution_id': 10000600883, 'institution_name': 'Kate D Smith Dar High School', 'jurisdiction_name': 'Alabama', 'lea_id': 100006, 'lea_name': 'Marshall County', 'year': 2020, 'interventions': {'Referral to Law Enforcement': {'Without Disabilities': {'Total': {'Male': 1, 'Female': 0, 'Total': 1}, 'White': {'Male': 1, 'Female': 0, 'Total': 1}}}}}, {'institution_id': 10000701456, 'institution_name': 'Spain Park High School', 'jurisdiction_name': 'Alabama', 'lea_id': 100007, 'lea_name': 'Hoover City', 'year': 2020, 'interventions': {'Referral to Law Enforcement': {'Without Disa

### Load

In [ ]:
from neo4j import GraphDatabase

# Function to insert suspension data into Neo4j
def insert_referarrest_data(session, data):
    cypher_query = """
    UNWIND $data AS schoolData
    // Create or find Jurisdiction, LEA, and Institution
    MERGE (jurisdiction:Jurisdiction {name: schoolData.jurisdiction_name})
    MERGE (lea:LEA {ncesID: schoolData.lea_id, name: schoolData.lea_name})
    MERGE (jurisdiction)-[:INCLUDES]->(lea)
    MERGE (institution:Institution {ncesID: schoolData.institution_id, name: schoolData.institution_name})
    MERGE (lea)-[:INCLUDES]->(institution)

    // Loop through each intervention type in 'interventions'
    WITH schoolData, institution
    UNWIND keys(schoolData.interventions) AS interventionName
    MERGE (intervention:Intervention {name: interventionName, date: schoolData.year})

    // Process each disability status within the intervention type
    WITH schoolData, institution, intervention, interventionName
    UNWIND keys(schoolData.interventions[interventionName]) AS disabilityStatus
    UNWIND keys(schoolData.interventions[interventionName][disabilityStatus]) AS subgroupName
    WITH institution, intervention, disabilityStatus, subgroupName, schoolData.interventions[interventionName][disabilityStatus][subgroupName] AS studentDetails, schoolData
    MERGE (institution)-[r:ASSIGNS {date: schoolData.year, subgroup: (subgroupName + ' ' + disabilityStatus)}]->(intervention)
        ON CREATE SET r.male = studentDetails.Male, r.female = studentDetails.Female, r.total = studentDetails.Total
    """
    session.run(cypher_query, {"data": data})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Assuming your transformed suspension data is stored in a variable named transformed_suspension_data
# Here we take a subset for demonstration
test_data_referarrest = transformed_referarrest_data[:10]

# Execute the insertion for the suspension data
with driver.session() as session:
    insert_referarrest_data(session, test_data_referarrest)

driver.close()

100%|██████████| 12/12 [48:46<00:00, 243.84s/it]


## CRDC Offenses

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
offenses_path = './Offenses.csv'  # Update with the actual file path
offenses_df = pd.read_csv(offenses_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Offenses Preview:")
print(offenses_df.head(), "\n")

Offenses Preview:
  LEA_STATE LEA_STATE_NAME    LEAID                LEA_NAME  SCHID  \
0        AL        Alabama  0100002  Alabama Youth Services   1705   
1        AL        Alabama  0100002  Alabama Youth Services   1706   
2        AL        Alabama  0100002  Alabama Youth Services  99995   
3        AL        Alabama  0100005        Albertville City    870   
4        AL        Alabama  0100005        Albertville City    871   

                        SCH_NAME      COMBOKEY   JJ SCH_FIREARM_IND  \
0  Wallace Sch - Mt Meigs Campus  010000201705  Yes              No   
1      Mcneel Sch - Vacca Campus  010000201706  Yes              No   
2                 Autauga Campus  010000299995  Yes              No   
3      Albertville Middle School  010000500870   No              No   
4        Albertville High School  010000500871   No              No   

  SCH_HOMICIDE_IND  SCH_OFFENSE_RAPE  SCH_OFFENSE_BATT  SCH_OFFENSE_ROBWW  \
0               No                 0                 0   

### Transform

In [ ]:
from collections import defaultdict

def transform_offenses_data(df, year):
    offense_types = {
        "Firearm Incident": "SCH_FIREARM_IND",
        "Homicide": "SCH_HOMICIDE_IND",
        "Rape or Attempted Rape": "SCH_OFFENSE_RAPE",
        "Sexual Assault (Other than Rape)": "SCH_OFFENSE_BATT",
        "Robbery with a Weapon": "SCH_OFFENSE_ROBWW",
        "Robbery with a Firearm or Explosive": "SCH_OFFENSE_ROBWX",
        "Robbery without a Weapon": "SCH_OFFENSE_ROBWOW",
        "Physical Attack or Fight with a Weapon": "SCH_OFFENSE_ATTWW",
        "Physical Attack or Fight with a Firearm or Explosive": "SCH_OFFENSE_ATTWX",
        "Physical Attack or Fight without a Weapon": "SCH_OFFENSE_ATTWOW",
        "Threats of Physical Attack with a Weapon": "SCH_OFFENSE_THRWW",
        "Threats of Physical Attack with a Firearm or Explosive": "SCH_OFFENSE_THRWX",
        "Threats of Physical Attack without a Weapon": "SCH_OFFENSE_THRWOW",
        "Possession of a Firearm or Explosive": "SCH_OFFENSE_POSSWX"
    }

    transformed_data = []
    for _, row in df.iterrows():
        offenses = {}

        # Process each offense type
        for offense_name, offense_key in offense_types.items():
            if "IND" in offense_key:  # For Yes/No indicators
                if row[offense_key] == "Yes":
                    offenses[offense_name] = True
            else:  # For numeric offense counts
                count = max(int(row.get(offense_key, 0)), 0)
                if count > 0:
                    offenses[offense_name] = count

        # Check if there are any offenses to report
        if any(offenses.values()):
            school_data = {
                "institution_id": row["COMBOKEY"],
                "institution_name": row["SCH_NAME"],
                "jurisdiction_name": row["LEA_STATE_NAME"],
                "lea_id": row["LEAID"],
                "lea_name": row["LEA_NAME"],
                "year": year,
                "offenses": offenses
            }
            transformed_data.append(school_data)

    return transformed_data

#transformed_offenses_data = transform_offenses_data(offenses_df, 2020)

offenses_df_subset = offenses_df.head(200)

# Now, use this subset to transform the data
transformed_offenses_data = transform_offenses_data(offenses_df_subset, 2020)

# Optionally, print or inspect part of the transformed data to check its structure
print(transformed_offenses_data[:15])  # Print details of the first 5 schools for inspection

[{'institution_id': 10000500870, 'institution_name': 'Albertville Middle School', 'jurisdiction_name': 'Alabama', 'lea_id': 100005, 'lea_name': 'Albertville City', 'year': 2020, 'offenses': {'Physical Attack or Fight with a Weapon': 1, 'Physical Attack or Fight without a Weapon': 9}}, {'institution_id': 10000500871, 'institution_name': 'Albertville High School', 'jurisdiction_name': 'Alabama', 'lea_id': 100005, 'lea_name': 'Albertville City', 'year': 2020, 'offenses': {'Physical Attack or Fight without a Weapon': 8}}, {'institution_id': 10000700251, 'institution_name': 'Hoover High School', 'jurisdiction_name': 'Alabama', 'lea_id': 100007, 'lea_name': 'Hoover City', 'year': 2020, 'offenses': {'Sexual Assault (Other than Rape)': 1}}, {'institution_id': 10000800303, 'institution_name': 'Horizon Elementary School', 'jurisdiction_name': 'Alabama', 'lea_id': 100008, 'lea_name': 'Madison City', 'year': 2020, 'offenses': {'Threats of Physical Attack without a Weapon': 6}}, {'institution_id': 

### Load

In [ ]:
from neo4j import GraphDatabase

# Function to insert suspension data into Neo4j
def insert_offenses_data(session, data):
    cypher_query = """
    UNWIND $data AS schoolData
    // Create or find Jurisdiction, LEA, and Institution
    MERGE (jurisdiction:Jurisdiction {name: schoolData.jurisdiction_name})
    MERGE (lea:LEA {ncesID: schoolData.lea_id, name: schoolData.lea_name})
    MERGE (jurisdiction)-[:INCLUDES]->(lea)
    MERGE (institution:Institution {ncesID: schoolData.institution_id, name: schoolData.institution_name})
    MERGE (lea)-[:INCLUDES]->(institution)

    // Loop through each offense in 'offenses'
    WITH schoolData, institution
    UNWIND keys(schoolData.offenses) AS offenseName
    MERGE (outcome:Outcome {name: offenseName, date: schoolData.year})

    // Create REPORTS relationship with the number of incidents and date
    WITH institution, outcome, schoolData.offenses[offenseName] AS incidentCount, schoolData
    MERGE (institution)-[reports:REPORTS {date: schoolData.year}]->(outcome)
        ON CREATE SET reports.incidents = incidentCount
    """
    session.run(cypher_query, {"data": data})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Assuming your transformed suspension data is stored in a variable named transformed_suspension_data
# Here we take a subset for demonstration
test_data_offenses = transformed_offenses_data[:10]

# Execute the insertion for the suspension data
with driver.session() as session:
    insert_offenses_data(session, test_data_offenses)

driver.close()

 93%|█████████▎| 28/30 [2:37:57<09:55, 297.64s/it]

## CRDC Harassment and Bullying

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
harassbully_path = './Harassment and Bullying.csv'  # Update with the actual file path
harassbully_df = pd.read_csv(harassbully_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Harassment and Bullying Preview:")
print(harassbully_df.head(), "\n")

Harassment and Bullying Preview:
  LEA_STATE LEA_STATE_NAME    LEAID                LEA_NAME  SCHID  \
0        AL        Alabama  0100002  Alabama Youth Services   1705   
1        AL        Alabama  0100002  Alabama Youth Services   1706   
2        AL        Alabama  0100002  Alabama Youth Services  99995   
3        AL        Alabama  0100005        Albertville City    870   
4        AL        Alabama  0100005        Albertville City    871   

                        SCH_NAME      COMBOKEY   JJ  SCH_HBALLEGATIONS_SEX  \
0  Wallace Sch - Mt Meigs Campus  010000201705  Yes                      0   
1      Mcneel Sch - Vacca Campus  010000201706  Yes                      0   
2                 Autauga Campus  010000299995  Yes                      0   
3      Albertville Middle School  010000500870   No                      0   
4        Albertville High School  010000500871   No                      0   

   SCH_HBALLEGATIONS_RAC  ...  SCH_HBDISCIPLINED_DIS_TR_M  \
0               

### Transform

In [ ]:
from collections import defaultdict

def transform_harassbully_data(df, year):
    race_categories = {
        "HI": "Hispanic",
        "AM": "American Indian/Alaska Native",
        "AS": "Asian",
        "HP": "Native Hawaiian/Pacific Islander",
        "BL": "Black",
        "WH": "White",
        "TR": "Two or More Races",
        "LEP": "Limited English Proficiency",
        "IDEA": "Students with Disabilities (IDEA)",
        "504": "Students with Disabilities (Section 504)"
    }

    harassment_categories = {
        "SEX": "Sex",
        "RAC": "Race, Color, and National Origin",
        "DIS": "Disability",
        "ORI": "Sexual Orientation",
        "REL": "Religion"
    }

    transformed_data = []
    for _, row in df.iterrows():
        outcomes = {}
        for cat_prefix, cat_full in harassment_categories.items():
            # Process allegations
            total_allegations = int(row.get(f"SCH_HBALLEGATIONS_{cat_prefix}", 0))
            if total_allegations > 0:
                outcomes[f"Allegations of harassment or bullying on the basis of {cat_full}"] = {"Total": total_allegations}

            # Process each category for reported and disciplined
            for process_prefix, process_description in {"HBREPORTED": "reported as harassed or bullied", "HBDISCIPLINED": "disciplined for bullying and harassment"}.items():
                process_full_name = f"Students {process_description} on the basis of {cat_full}"
                outcome_details = {}
                total_male_count = 0
                total_female_count = 0

                # Subgroup data
                for subgroup_prefix, subgroup_name in race_categories.items():
                    for gender_suffix, gender in [("M", "Male"), ("F", "Female")]:
                        key = f"SCH_{process_prefix}_{cat_prefix}_{subgroup_prefix}_{gender_suffix}"
                        count = int(row.get(key, 0))
                        if count > 0:
                            subgroup_full_name = f"{subgroup_name}"
                            outcome_details[subgroup_full_name] = {"Male": 0, "Female": 0, "Total": 0}
                            outcome_details[subgroup_full_name][gender] = count
                            outcome_details[subgroup_full_name]["Total"] += count

                            if gender == "M":
                                total_male_count += count
                            else:
                                total_female_count += count

                # Total data
                if total_male_count > 0 or total_female_count > 0:
                    outcome_details["Total Students"] = {"Male": total_male_count, "Female": total_female_count, "Total": total_male_count + total_female_count}

                if outcome_details:  # Add outcomes only if there are details
                    outcomes[process_full_name] = outcome_details


        # Cleanup: Remove empty outcomes and subgroups
        outcomes = {k: v for k, v in outcomes.items() if v and (isinstance(v, dict) and any(v.values()) or v > 0)}

        if outcomes:  # Only add institution if there are outcomes
            transformed_data.append({
                "institution_id": row["COMBOKEY"],
                "institution_name": row["SCH_NAME"],
                "jurisdiction_name": row["LEA_STATE_NAME"],
                "lea_id": row["LEAID"],
                "lea_name": row["LEA_NAME"],
                "year": year,
                "outcomes": outcomes
            })

    return transformed_data


#transformed_harassbully_data = transform_harassbully_data(harassbully_df, 2020)

harassbully_df_subset = harassbully_df #.head(200)

# Now, use this subset to transform the data
transformed_harassbully_data = transform_harassbully_data(harassbully_df_subset, 2020)

# Optionally, print or inspect part of the transformed data to check its structure
print(transformed_harassbully_data[:5])  # Print details of the first 5 schools for inspection

[{'institution_id': 10000802198, 'institution_name': 'James Clemens High School', 'jurisdiction_name': 'Alabama', 'lea_id': 100008, 'lea_name': 'Madison City', 'year': 2020, 'outcomes': {'Students disciplined for bullying and harassment on the basis of Sex': {'Black': {'Male': 1, 'Female': 0, 'Total': 1}, 'Students with Disabilities (IDEA)': {'Male': 1, 'Female': 0, 'Total': 1}, 'Total Students': {'Male': 0, 'Female': 2, 'Total': 2}}, 'Students disciplined for bullying and harassment on the basis of Race, Color, and National Origin': {'Black': {'Male': 1, 'Female': 0, 'Total': 1}, 'Students with Disabilities (IDEA)': {'Male': 1, 'Female': 0, 'Total': 1}, 'Total Students': {'Male': 0, 'Female': 2, 'Total': 2}}}}, {'institution_id': 10001102094, 'institution_name': 'Leeds Elementary School', 'jurisdiction_name': 'Alabama', 'lea_id': 100011, 'lea_name': 'Leeds City', 'year': 2020, 'outcomes': {'Allegations of harassment or bullying on the basis of Sex': {'Total': 1}, 'Students reported as

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

# Function to insert suspension data into Neo4j
def insert_test_data(session, data):
    cypher_query = """
    UNWIND $data AS schoolData
    // Create or find Jurisdiction, LEA, and Institution
    MERGE (jurisdiction:Jurisdiction {name: schoolData.jurisdiction_name})
    MERGE (lea:LEA {ncesID: schoolData.lea_id, name: schoolData.lea_name})
    MERGE (jurisdiction)-[:INCLUDES]->(lea)
    MERGE (institution:Institution {ncesID: schoolData.institution_id, name: schoolData.institution_name})
    MERGE (lea)-[:INCLUDES]->(institution)

    WITH schoolData, institution
    UNWIND keys(schoolData.outcomes) AS outcomeType
    WITH outcomeType, schoolData, institution
    WHERE outcomeType CONTAINS 'disciplined'
    MERGE (intervention:Intervention {name: outcomeType})
    WITH schoolData.outcomes[outcomeType] AS subgroups, intervention, institution, schoolData
    UNWIND keys(subgroups) AS subgroupName
    WITH subgroups[subgroupName] AS details, intervention, institution, subgroupName, schoolData
    MERGE (institution)-[r:ASSIGNS {date: schoolData.year, subgroup: subgroupName}]->(intervention)
    ON CREATE SET r.male = details.Male, r.female = details.Female, r.total = details.Total

    WITH schoolData, institution
    UNWIND keys(schoolData.outcomes) AS outcomeType
    WITH outcomeType, schoolData, institution
    WHERE outcomeType CONTAINS 'reported'
    MERGE (outcome:Outcome {name: outcomeType})
    WITH schoolData.outcomes[outcomeType] AS subgroups, outcome, institution, schoolData
    UNWIND keys(subgroups) AS subgroupName
    WITH subgroups[subgroupName] AS details, outcome, institution, subgroupName, schoolData
    MERGE (institution)-[r:REPORTS {date: schoolData.year, subgroup: subgroupName}]->(outcome)
    ON CREATE SET r.male = details.Male, r.female = details.Female, r.total = details.Total

    WITH schoolData, institution
    UNWIND keys(schoolData.outcomes) AS outcomeType
    WITH outcomeType, schoolData, institution
    WHERE outcomeType CONTAINS 'Allegations'
    MERGE (outcome:Outcome {name: outcomeType})
    WITH schoolData.outcomes[outcomeType]["Total"] AS total, outcome, institution, schoolData
    MERGE (institution)-[r:REPORTS {date: schoolData.year, subgroup: 'Total'}]->(outcome)
    ON CREATE SET r.total = total

    """
    session.run(cypher_query, {"data": data})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Here we take a subset for demonstration
test_data = transformed_harassbully_data

# Execute the insertion for the harass and bully data
with driver.session() as session:
  for i in tqdm(range(6000, len(test_data) - 1, 1000)):
    sub_data = test_data [i:i+1000]
    insert_test_data(session, sub_data)

driver.close()

100%|██████████| 15/15 [58:40<00:00, 234.69s/it]


## CRDC Enrollment

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
enrollment_path = './Enrollment.csv'  # Update with the actual file path
enrollment_df = pd.read_csv(enrollment_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Enrollment Preview:")
print(enrollment_df.head(), "\n")

Enrollment Preview:
  LEA_STATE LEA_STATE_NAME    LEAID                LEA_NAME  SCHID  \
0        AL        Alabama  0100002  Alabama Youth Services   1705   
1        AL        Alabama  0100002  Alabama Youth Services   1706   
2        AL        Alabama  0100002  Alabama Youth Services  99995   
3        AL        Alabama  0100005        Albertville City    870   
4        AL        Alabama  0100005        Albertville City    871   

                        SCH_NAME      COMBOKEY   JJ  SCH_PSENR_HI_M  \
0  Wallace Sch - Mt Meigs Campus  010000201705  Yes              -9   
1      Mcneel Sch - Vacca Campus  010000201706  Yes              -9   
2                 Autauga Campus  010000299995  Yes              -9   
3      Albertville Middle School  010000500870   No              -9   
4        Albertville High School  010000500871   No              -9   

   SCH_PSENR_HI_F  ...  SCH_504ENR_BL_M  SCH_504ENR_BL_F  SCH_504ENR_WH_M  \
0              -9  ...                0                

### Transform

In [ ]:
from collections import defaultdict

def transform_enrollment_data(df, year):
    race_categories = {
        "AM": "American Indian/Alaska Native",
        "AS": "Asian",
        "HI": "Hispanic",
        "BL": "Black",
        "WH": "White",
        "HP": "Native Hawaiian/Pacific Islander",
        "TR": "Two or More Races"
    }

    program_categories = {
        "PSENR": "Preschool",
        "ENR": "General Education",
        "LEPENR": "Limited English Proficiency",
        "LEPPROGENR": "Limited English Proficiency Program",
        "IDEAENR": "Special Education (IDEA)",
        "504ENR": "Special Education (504)"
    }

    transformed_data = []
    for _, row in df.iterrows():
        categories_data = {}
        for program_prefix, category_name in program_categories.items():
            category_data = {}
            for race_code, race_name in race_categories.items():
                for gender_suffix, gender_name in [("M", "Male"), ("F", "Female")]:
                    key = f"SCH_{program_prefix}_{race_code}_{gender_suffix}"
                    count = row.get(key, -9)
                    if count > 0:  # Include positive counts only
                        if race_name not in category_data:
                            category_data[race_name] = {"Male": 0, "Female": 0, "Total": 0}
                        category_data[race_name][gender_name] += count
                        category_data[race_name]["Total"] += count

            # Handle total counts for each category
            total_male_key = f"TOT_{program_prefix}_M"
            total_female_key = f"TOT_{program_prefix}_F"
            total_male = row.get(total_male_key, -9)
            total_female = row.get(total_female_key, -9)
            if total_male > 0 or total_female > 0:
                category_total = total_male + total_female
                category_data["Total"] = {"Male": total_male, "Female": total_female, "Total": category_total}

            if category_data:  # If there's data in the category
                categories_data[category_name] = category_data

        if categories_data:
            transformed_data.append({
                "institution_id": row["COMBOKEY"],
                "institution_name": row["SCH_NAME"],
                "jurisdiction_name": row["LEA_STATE_NAME"],
                "lea_id": row["LEAID"],
                "lea_name": row["LEA_NAME"],
                "year": year,
                "enrollments": categories_data
            })

    return transformed_data


#transformed_enrollment_data = transform_enrollment_data(enrollment_df, 2020)

enrollment_df_subset = enrollment_df #.head(200)

# Now, use this subset to transform the data
transformed_enrollment_data = transform_enrollment_data(enrollment_df_subset, 2020)

# Optionally, print or inspect part of the transformed data to check its structure
print(transformed_enrollment_data[:15])  # Print details of the first 5 schools for inspection

[{'institution_id': '010000201705', 'institution_name': 'Wallace Sch - Mt Meigs Campus', 'jurisdiction_name': 'Alabama', 'lea_id': '0100002', 'lea_name': 'Alabama Youth Services', 'year': 2020, 'enrollments': {'General Education': {'Hispanic': {'Male': 1, 'Female': 0, 'Total': 1}, 'Black': {'Male': 63, 'Female': 0, 'Total': 63}, 'White': {'Male': 46, 'Female': 0, 'Total': 46}, 'Total': {'Male': 110, 'Female': 0, 'Total': 110}}, 'Limited English Proficiency': {'Hispanic': {'Male': 1, 'Female': 0, 'Total': 1}, 'Total': {'Male': 1, 'Female': 0, 'Total': 1}}, 'Limited English Proficiency Program': {'Hispanic': {'Male': 1, 'Female': 0, 'Total': 1}, 'Total': {'Male': 1, 'Female': 0, 'Total': 1}}, 'Special Education (IDEA)': {'Hispanic': {'Male': 1, 'Female': 0, 'Total': 1}, 'Black': {'Male': 7, 'Female': 0, 'Total': 7}, 'White': {'Male': 4, 'Female': 0, 'Total': 4}, 'Total': {'Male': 12, 'Female': 0, 'Total': 12}}}}, {'institution_id': '010000201706', 'institution_name': 'Mcneel Sch - Vacca 

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

# Function to insert suspension data into Neo4j
def insert_test_data(session, data):
    cypher_query = """
    UNWIND $data AS schoolData
    MERGE (institution:Institution {ncesID: schoolData.institution_id})

    // Correctly chaining UNWIND, WITH, and WHERE
    WITH schoolData, institution
    UNWIND keys(schoolData.enrollments) AS categoryName
    WITH schoolData, institution, categoryName
    WHERE categoryName <> 'Limited English Proficiency Program'
    UNWIND keys(schoolData.enrollments[categoryName]) AS subgroupName
    WITH institution, categoryName, subgroupName, schoolData.year AS year,schoolData,
        schoolData.enrollments[categoryName][subgroupName] AS subgroupDetails

    // Construct full subgroup name including the context as needed
    WITH institution, schoolData,
        CASE WHEN categoryName = 'General Education' THEN subgroupName
              ELSE categoryName + ' ' + subgroupName
        END AS fullSubgroupName,
        year, subgroupDetails

    MERGE (subgroupNode:Subgroup {name: fullSubgroupName})
    MERGE (institution)-[r:HAS_SUBGROUP {date: year}]->(subgroupNode)
        ON CREATE SET
            r.male = subgroupDetails.Male,
            r.female = subgroupDetails.Female,
            r.total = subgroupDetails.Total
        ON MATCH SET
            r.male = subgroupDetails.Male,
            r.female = subgroupDetails.Female,
            r.total = subgroupDetails.Total

    // Separately handling 'Limited English Proficiency Program'
    WITH schoolData, institution
    WHERE 'Limited English Proficiency Program' IN keys(schoolData.enrollments)
    UNWIND keys(schoolData.enrollments['Limited English Proficiency Program']) AS lepSubgroupName
    WITH institution, lepSubgroupName, schoolData.year AS year, schoolData,
        schoolData.enrollments['Limited English Proficiency Program'][lepSubgroupName] AS lepDetails

    MERGE (lepProgram:Intervention {name: 'Limited English Proficiency Program', date: year})
    MERGE (institution)-[r:ASSIGNS {date: year, subgroup: lepSubgroupName}]->(lepProgram)
        ON CREATE SET
            r.male = lepDetails.Male,
            r.female = lepDetails.Female,
            r.total = lepDetails.Total
        ON MATCH SET
            r.male = lepDetails.Male,
            r.female = lepDetails.Female,
            r.total = lepDetails.Total
    """
    session.run(cypher_query, {"data": data})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Here we take a subset for demonstration (add [:10])
test_data = transformed_enrollment_data

# Execute the insertion for the suspension data
with driver.session() as session:
#    insert_test_data(session, test_data)

  for i in tqdm(range(0, len(test_data) - 1, 1000)):
    sub_data = test_data [i:i+1000]
    insert_test_data(session, sub_data)

driver.close()

100%|██████████| 98/98 [16:50<00:00, 10.31s/it]


## CRDC LEA High School Equivalency

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
hse_path = './High School Equivalency.csv'  # Update with the actual file path
hse_df = pd.read_csv(hse_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("High School Equivalency Preview:")
print(hse_df.head(), "\n")

High School Equivalency Preview:
  LEA_STATE LEA_STATE_NAME    LEAID                LEA_NAME  \
0        AL        Alabama  0100002  Alabama Youth Services   
1        AL        Alabama  0100005        Albertville City   
2        AL        Alabama  0100006         Marshall County   
3        AL        Alabama  0100007             Hoover City   
4        AL        Alabama  0100008            Madison City   

                   LEA_ADDRESS      LEA_CITY  LEA_ZIP  CJJ LEA_HSE_IND  \
0  1000 Industrial School Road      Mt Meigs    36057  Yes         Yes   
1         107 West Main Street   Albertville    35950   No          No   
2       12380 Us Highway 431 S  Guntersville    35976   No          No   
3        2810 Metropolitan Way        Hoover    35243   No          No   
4             211 Celtic Drive       Madison    35758  Yes          No   

   LEA_HSEPART_HI_M  ...  LEA_HSEPART_WH_M  LEA_HSEPART_WH_F  \
0                 2  ...                 5                 0   
1              

### Transform

In [ ]:
from collections import defaultdict

def transform_hse_data(df, year):
    race_categories = {
        "HI": "Hispanic",
        "AM": "American Indian/Alaska Native",
        "AS": "Asian",
        "HP": "Native Hawaiian/Pacific Islander",
        "BL": "Black",
        "WH": "White",
        "TR": "Two or More Races",
        "LEP": "Limited English Proficiency",
        "IDEA": "Student With Disability (IDEA)"
    }

    transformed_data = []
    for _, row in df.iterrows():
        enrollments = {}
        # Use TOT_ fields for "Total Students"
        total_male = int(row.get("TOT_HSEPART_M", -9))
        total_female = int(row.get("TOT_HSEPART_F", -9))

        # Check for valid positive values
        if total_male > 0 and total_female > 0:
            total_students = {
                "Male": total_male,
                "Female": total_female,
                "Total": total_male + total_female
            }

            # Ensure the "Total Students" subgroup is included
            enrollments["Total Students"] = total_students

        else:
            # Skip this LEA if no valid total student data
            continue

        for key, value in row.items():
            # Process enrollments except for the TOT_ fields
            if 'HSEPART' in key and not key.startswith("TOT_") and value not in [-9, '-9', '']:
                parts = key.split("_")
                race_code = parts[-2]
                gender = "Male" if parts[-1] == "M" else "Female"
                count = int(value)

                # Skip processing if count is not positive
                if count <= 0:
                    continue

                race_name = race_categories.get(race_code, "Unknown")
                subgroup = f"{race_name}"

                if subgroup not in enrollments:
                    enrollments[subgroup] = {"Male": 0, "Female": 0, "Total": 0}

                enrollments[subgroup][gender] = count
                # Adjust the total for each subgroup
                enrollments[subgroup]["Total"] += count

        # Ensure the "Total Students" subgroup is included
        enrollments["Total Students"] = total_students

        lea_data = {
            "lea_id": row["LEAID"],
            "lea_name": row["LEA_NAME"],
            "jurisdiction_name": row["LEA_STATE_NAME"],
            "year": year,
            "hse_enrollments": enrollments
        }

        transformed_data.append(lea_data)

    return transformed_data

hse_df_subset = hse_df #.head(200)
transformed_hse_data = transform_hse_data(hse_df_subset, 2020)
print(transformed_hse_data[:5])

[{'lea_id': '0100011', 'lea_name': 'Leeds City', 'jurisdiction_name': 'Alabama', 'year': 2020, 'hse_enrollments': {'Total Students': {'Male': 1, 'Female': 1, 'Total': 2}, 'White': {'Male': 1, 'Female': 1, 'Total': 2}}}, {'lea_id': '0100038', 'lea_name': 'Pathway Inc', 'jurisdiction_name': 'Alabama', 'year': 2020, 'hse_enrollments': {'Total Students': {'Male': 28, 'Female': 11, 'Total': 39}, 'Hispanic': {'Male': 1, 'Female': 0, 'Total': 1}, 'Black': {'Male': 20, 'Female': 0, 'Total': 20}, 'White': {'Male': 7, 'Female': 11, 'Total': 18}, 'Student With Disability (IDEA)': {'Male': 6, 'Female': 0, 'Total': 6}}}, {'lea_id': '0100041', 'lea_name': 'Developing Alabama Youth Prog', 'jurisdiction_name': 'Alabama', 'year': 2020, 'hse_enrollments': {'Total Students': {'Male': 9, 'Female': 3, 'Total': 12}, 'Hispanic': {'Male': 3, 'Female': 1, 'Total': 4}, 'Black': {'Male': 0, 'Female': 1, 'Total': 1}, 'White': {'Male': 6, 'Female': 1, 'Total': 7}, 'Limited English Proficiency': {'Male': 3, 'Female

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

def insert_test_data(session, data):
    cypher_query = """
    UNWIND $data AS leaData
    // Create or find Jurisdiction
    MERGE (jurisdiction:Jurisdiction {name: leaData.jurisdiction_name})
    // Create or find LEA
    MERGE (lea:LEA {ncesID: leaData.lea_id, name: leaData.lea_name})
    MERGE (jurisdiction)-[:INCLUDES]->(lea)

    // Create the Intervention for High School Equivalency Preparation Program
    MERGE (intervention:Intervention {name: "High School Equivalency Preparation Program", date: leaData.year})

    // Process each subgroup in hse_enrollments
    WITH lea, intervention, leaData
    UNWIND keys(leaData.hse_enrollments) AS subgroupName
    WITH lea, intervention, leaData, subgroupName, leaData.hse_enrollments[subgroupName] AS enrollmentData

    // For each subgroup, including "Total Students", create or find the Subgroup node and create ASSIGNS relationship
    MERGE (subgroup:Subgroup {name: subgroupName})
    WITH lea, intervention, subgroup, enrollmentData, leaData, subgroupName
    MERGE (lea)-[r:ASSIGNS {date: leaData.year, subgroup: subgroupName}]->(intervention)
        ON CREATE SET r.male = enrollmentData.Male,
                      r.female = enrollmentData.Female,
                      r.total = enrollmentData.Total
    """
    session.run(cypher_query, {"data": data})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Here we take a subset for demonstration
test_data = transformed_hse_data

# Execute the insertion for the chemistry data
with driver.session() as session:
  for i in tqdm(range(0, len(test_data) - 1, 1000)):
    sub_data = test_data [i:i+1000]
    insert_test_data(session, sub_data)

driver.close()

100%|██████████| 1/1 [00:27<00:00, 27.68s/it]


## CRDC LEA Distance Education

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
disted_path = './Distance Education.csv'  # Update with the actual file path
disted_df = pd.read_csv(disted_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Distance Education Preview:")
print(disted_df.head(), "\n")

Distance Education Preview:
  LEA_STATE LEA_STATE_NAME    LEAID                LEA_NAME  \
0        AL        Alabama  0100002  Alabama Youth Services   
1        AL        Alabama  0100005        Albertville City   
2        AL        Alabama  0100006         Marshall County   
3        AL        Alabama  0100007             Hoover City   
4        AL        Alabama  0100008            Madison City   

                   LEA_ADDRESS      LEA_CITY  LEA_ZIP  CJJ LEA_DISTED_IND  \
0  1000 Industrial School Road      Mt Meigs    36057  Yes             No   
1         107 West Main Street   Albertville    35950   No            Yes   
2       12380 Us Highway 431 S  Guntersville    35976   No            Yes   
3        2810 Metropolitan Way        Hoover    35243   No            Yes   
4             211 Celtic Drive       Madison    35758  Yes            Yes   

   LEA_DISTEDENR_HI_M  ...  LEA_DISTEDENR_WH_M  LEA_DISTEDENR_WH_F  \
0                  -9  ...                  -9              

### Transform

In [ ]:
from collections import defaultdict

def transform_disted_data(df, year):
    race_categories = {
        "HI": "Hispanic",
        "AM": "American Indian/Alaska Native",
        "AS": "Asian",
        "HP": "Native Hawaiian/Pacific Islander",
        "BL": "Black",
        "WH": "White",
        "TR": "Two or More Races",
        "LEP": "Limited English Proficiency",
        "IDEA": "Student With Disability (IDEA)"
    }

    transformed_data = []
    for _, row in df.iterrows():
        enrollments = {}
        # Use TOT_ fields for "Total Students"
        total_male = int(row.get("TOT_DISTEDENR_M", -9))
        total_female = int(row.get("TOT_DISTEDENR_F", -9))

        # Check for valid positive values
        if total_male > 0 and total_female > 0:
            total_students = {
                "Male": total_male,
                "Female": total_female,
                "Total": total_male + total_female
            }

            # Ensure the "Total Students" subgroup is included
            enrollments["Total Students"] = total_students

        else:
            # Skip this LEA if no valid total student data
            continue

        for key, value in row.items():
            # Process enrollments except for the TOT_ fields
            if 'DISTEDENR' in key and not key.startswith("TOT_") and value not in [-9, '-9', '']:
                parts = key.split("_")
                race_code = parts[-2]
                gender = "Male" if parts[-1] == "M" else "Female"
                count = int(value)

                # Skip processing if count is not positive
                if count <= 0:
                    continue

                race_name = race_categories.get(race_code, "Unknown")
                subgroup = f"{race_name}"

                if subgroup not in enrollments:
                    enrollments[subgroup] = {"Male": 0, "Female": 0, "Total": 0}

                enrollments[subgroup][gender] = count
                # Adjust the total for each subgroup
                enrollments[subgroup]["Total"] += count


        lea_data = {
            "lea_id": row["LEAID"],
            "lea_name": row["LEA_NAME"],
            "jurisdiction_name": row["LEA_STATE_NAME"],
            "year": year,
            "disted_enrollments": enrollments
        }

        transformed_data.append(lea_data)

    return transformed_data

disted_df_subset = disted_df #.head(200)
transformed_disted_data = transform_disted_data(disted_df_subset, 2020)
print(transformed_disted_data[:5])

[{'lea_id': '0100005', 'lea_name': 'Albertville City', 'jurisdiction_name': 'Alabama', 'year': 2020, 'disted_enrollments': {'Total Students': {'Male': 12, 'Female': 25, 'Total': 37}, 'Hispanic': {'Male': 2, 'Female': 10, 'Total': 12}, 'Black': {'Male': 1, 'Female': 0, 'Total': 1}, 'White': {'Male': 9, 'Female': 15, 'Total': 24}, 'Limited English Proficiency': {'Male': 1, 'Female': 0, 'Total': 1}, 'Student With Disability (IDEA)': {'Male': 5, 'Female': 4, 'Total': 9}}}, {'lea_id': '0100006', 'lea_name': 'Marshall County', 'jurisdiction_name': 'Alabama', 'year': 2020, 'disted_enrollments': {'Total Students': {'Male': 18, 'Female': 37, 'Total': 55}, 'Hispanic': {'Male': 7, 'Female': 14, 'Total': 21}, 'Black': {'Male': 0, 'Female': 2, 'Total': 2}, 'White': {'Male': 11, 'Female': 21, 'Total': 32}, 'Limited English Proficiency': {'Male': 0, 'Female': 1, 'Total': 1}, 'Student With Disability (IDEA)': {'Male': 1, 'Female': 0, 'Total': 1}}}, {'lea_id': '0100007', 'lea_name': 'Hoover City', 'jur

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

def insert_test_data(session, data):
    cypher_query = """
    UNWIND $data AS leaData
    // Create or find Jurisdiction
    MERGE (jurisdiction:Jurisdiction {name: leaData.jurisdiction_name})
    // Create or find LEA
    MERGE (lea:LEA {ncesID: leaData.lea_id, name: leaData.lea_name})
    MERGE (jurisdiction)-[:INCLUDES]->(lea)

    // Create the Intervention for High School Equivalency Preparation Program
    MERGE (intervention:Intervention {name: "Distance Education", date: leaData.year})

    // Process each subgroup in disted_enrollments
    WITH lea, intervention, leaData
    UNWIND keys(leaData.disted_enrollments) AS subgroupName
    WITH lea, intervention, leaData, subgroupName, leaData.disted_enrollments[subgroupName] AS enrollmentData

    // For each subgroup, including "Total Students", create or find the Subgroup node and create ASSIGNS relationship
    MERGE (subgroup:Subgroup {name: subgroupName})
    WITH lea, intervention, subgroup, enrollmentData, leaData, subgroupName
    MERGE (lea)-[r:ASSIGNS {date: leaData.year, subgroup: subgroupName}]->(intervention)
        ON CREATE SET r.male = enrollmentData.Male,
                      r.female = enrollmentData.Female,
                      r.total = enrollmentData.Total
    """
    session.run(cypher_query, {"data": data})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Here we take a subset for demonstration
test_data = transformed_disted_data

# Execute the insertion for the chemistry data
with driver.session() as session:
  for i in tqdm(range(0, len(test_data) - 1, 1000)):
    sub_data = test_data [i:i+1000]
    insert_test_data(session, sub_data)

driver.close()

100%|██████████| 8/8 [03:34<00:00, 26.81s/it]


## Edfacts Math Assessment

### Extract

In [ ]:
import pandas as pd

dtype_dict = {
    'LEAID': str,
    'NCESSCH': str,
}

# Load the chronic absenteeism data
math_assessment_path = './EDF_SCH_AP_MTH_2021_PUB_033022.csv'
math_assessment_df = pd.read_csv(math_assessment_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("School Support Preview:")
print(math_assessment_df.head(), "\n")

School Support Preview:
  SCHOOL_YEAR    STNAM  FIPST    LEAID ST_LEAID             LEANM  \
0   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   
1   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   
2   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   
3   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   
4   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   

        NCESSCH     ST_SCHID                     SCHNAM SUBJECT GRADE  \
0  010000500870  AL-101-0010  Albertville Middle School     MTH    00   
1  010000500870  AL-101-0010  Albertville Middle School     MTH    00   
2  010000500870  AL-101-0010  Albertville Middle School     MTH    00   
3  010000500870  AL-101-0010  Albertville Middle School     MTH    00   
4  010000500870  AL-101-0010  Albertville Middle School     MTH    00   

  CATEGORY DATE_CUR  NUMVALID PCTPROF  
0      ALL  30MAR22     884.0      11  
1      CWD  30MAR22      74.0   10-14  
2 

In [ ]:
unique_schools_in_assessment = set(math_assessment_df['NCESSCH'].unique())
print(len(unique_schools_in_assessment))

68522


In [ ]:
from collections import defaultdict
import json
import re
from tqdm import tqdm

category_map = {
    "ALL": "Total",
    "CWD": "Students with disabilities (IDEA)",
    "ECD": "Economically disadvantaged",
    "F": "Female",
    "FCS": "Foster Care Status",
    "HOM": "Homeless enrolled",
    "LEP": "English Learner",
    "M": "Male",
    "MAM": "American Indian/Alaska Native",
    "MAS": "Asian/Pacific Islander",
    "MBL": "Black",
    "MHI": "Hispanic",
    "MIG": "Migrant",
    "MIL": "Military Connected Student Status",
    "MTR": "Two or More Races",
    "MWH": "White"
}

grade_map = {
    "00": "All Grades",
    "03": "Grade 3",
    "04": "Grade 4",
    "05": "Grade 5",
    "06": "Grade 6",
    "07": "Grade 7",
    "08": "Grade 8",
    "HS": "High School Grades"
}

pct_proficient_symbols = {
    'GE': '≥',
    'LE': '≤',
    'GT': '>',
    'LT': '<',
    'PS': 'Data suppressed for privacy'
}

def convert_pct_proficient(value):
    # Check if the value is a string
    if isinstance(value, str):
        # Replace symbols with their corresponding abbreviations
        for abbreviation, symbol in pct_proficient_symbols.items():
            # Use regex to replace instances of the abbreviation followed by digits or in case of PS
            if abbreviation == 'PS':
                value = value.replace(abbreviation, symbol)
            else:
                value = re.sub(f'{abbreviation}(?=\d)', symbol, value)
    return value

def process_school_data(school_data):
    transformed_data = []

    first_row = school_data.iloc[0]
    school_id, institution_name, jurisdiction_name = first_row["NCESSCH"], first_row["SCHNAM"], first_row["STNAM"].title()
    lea_id, lea_name, year = first_row["LEAID"], first_row["LEANM"], first_row["SCHOOL_YEAR"][5:9]

    math_assessments = defaultdict(dict)

    # Create a sorted list of grade codes based on the order in grade_map
    grade_codes = sorted(set(school_data["GRADE"].dropna().astype(str)), key=lambda x: list(grade_map).index(x) if x in grade_map else float('inf'))

    for grade_code in grade_codes:
        grade = grade_map.get(grade_code, "Unknown Grade")
        grade_data = school_data[school_data["GRADE"].astype(str) == grade_code]

        grade_assessments = {}
        for _, grade_row in grade_data.iterrows():
            category = category_map.get(grade_row["CATEGORY"], "Unknown Category")
            valid_assessments, pct_proficient = grade_row["NUMVALID"], convert_pct_proficient(grade_row["PCTPROF"])

            if valid_assessments and valid_assessments > 0:
                grade_assessments[category] = {
                    "validAssessments": int(valid_assessments),
                    "perProficient": pct_proficient
                }

        if grade_assessments:
            math_assessments[grade] = grade_assessments

    if math_assessments:
        transformed_data.append({
            "institution_id": school_id,
            "institution_name": institution_name,
            "jurisdiction_name": jurisdiction_name,
            "lea_id": lea_id,
            "lea_name": lea_name,
            "year": year,
            "mathAssessment": dict(math_assessments)
        })

    return transformed_data

def transform_math_assessment_data(df):
    transformed_data = []

    # Directly iterate over each group and process
    for _, group in tqdm(df.groupby("NCESSCH")):
        transformed_data.extend(process_school_data(group))

    return transformed_data

math_assessment_df_subset = math_assessment_df.head(200)

print("Transforming math assessment data")
transformed_math_assessment_data = transform_math_assessment_data(math_assessment_df_subset)

#with open("math_assessment_data.json", "w+") as f:
#    json.dump(transformed_math_assessment_data, f)

print(transformed_math_assessment_data[:5])


Transforming math assessment data


100%|██████████| 5/5 [00:00<00:00, 145.26it/s]

[{'institution_id': '010000500870', 'institution_name': 'Albertville Middle School', 'jurisdiction_name': 'Alabama', 'lea_id': '0100005', 'lea_name': 'Albertville City', 'year': '2021', 'mathAssessment': {'All Grades': {'Total': {'validAssessments': 884, 'perProficient': '11'}, 'Students with disabilities (IDEA)': {'validAssessments': 74, 'perProficient': '10-14'}, 'Economically disadvantaged': {'validAssessments': 342, 'perProficient': '5'}, 'Female': {'validAssessments': 435, 'perProficient': '10'}, 'Foster Care Status': {'validAssessments': 1, 'perProficient': 'Data suppressed for privacy'}, 'Homeless enrolled': {'validAssessments': 6, 'perProficient': '<50'}, 'English Learner': {'validAssessments': 165, 'perProficient': '≤5'}, 'Male': {'validAssessments': 449, 'perProficient': '11'}, 'American Indian/Alaska Native': {'validAssessments': 2, 'perProficient': 'Data suppressed for privacy'}, 'Asian/Pacific Islander': {'validAssessments': 4, 'perProficient': 'Data suppressed for privacy

In [ ]:
import json
import numpy as np

# Custom JSON encoder subclass to convert NumPy types to Python native types for JSON serialization
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NumpyEncoder, self).default(obj)

# Use this custom encoder in your json.dump call
file_path = '/content/transformed_math_assessment_data.json'
with open(file_path, 'w') as file:
    json.dump(transformed_math_assessment_data, file, cls=NumpyEncoder, indent=4)

print(f"Data saved to {file_path}")

Data saved to /content/transformed_math_assessment_data.json


### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

def insert_math_assessment_data(session, data, batch_size=1000):
    # Calculate total number of batches for tqdm
    total_batches = len(data) // batch_size + (1 if len(data) % batch_size else 0)

    for i in tqdm(range(0, len(data), batch_size), total=total_batches, desc="Inserting batches"):
        batch = data[i:i + batch_size]
        cypher_query = """
        UNWIND $data AS schoolData
        MERGE (institution:Institution {ncesID: schoolData.institution_id})


        // Loop through each grade in 'mathAssessment'
        WITH schoolData, institution
        UNWIND keys(schoolData.mathAssessment) AS grade
        WITH institution, grade, schoolData.mathAssessment[grade] AS gradeAssessments, schoolData

        WITH schoolData, institution
        UNWIND keys(schoolData.mathAssessment) AS grade
        MERGE (outcome:Outcome {grade: grade, year: schoolData.year, name: grade + ' Math Achievement'})
        WITH institution, outcome, schoolData.mathAssessment[grade] AS gradeData, grade, schoolData
        UNWIND keys(gradeData) AS subgroup
            MERGE (institution)-[reports:REPORTS {subgroup: subgroup, grade: grade, year: schoolData.year}]->(outcome)
            ON CREATE SET reports.validAssessments = gradeData[subgroup].validAssessments,
                          reports.percentProficient = gradeData[subgroup].perProficient
        """
        session.run(cypher_query, {"data": batch})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
with driver.session() as session:
    insert_math_assessment_data(session, transformed_math_assessment_data, batch_size=1000)

driver.close()

Inserting batches: 100%|██████████| 1/1 [00:01<00:00,  1.46s/it]


## Edfacts Reading/LA Assessment

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
reading_assessment_path = './EDF_SCH_AP_RLA_2021_PUB_033022.csv'  # Update with the actual file path
reading_assessment_df = pd.read_csv(reading_assessment_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Reading Assessment Preview:")
print(reading_assessment_df.head(), "\n")

Reading Assessment Preview:
  SCHOOL_YEAR    STNAM  FIPST    LEAID ST_LEAID             LEANM  \
0   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   
1   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   
2   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   
3   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   
4   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   

        NCESSCH     ST_SCHID                     SCHNAM SUBJECT GRADE  \
0  010000500870  AL-101-0010  Albertville Middle School     RLA    00   
1  010000500870  AL-101-0010  Albertville Middle School     RLA    00   
2  010000500870  AL-101-0010  Albertville Middle School     RLA    00   
3  010000500870  AL-101-0010  Albertville Middle School     RLA    00   
4  010000500870  AL-101-0010  Albertville Middle School     RLA    00   

  CATEGORY DATE_CUR  NUMVALID PCTPROF  
0      ALL  30MAR22     881.0      42  
1      CWD  30MAR22      74.0   10-14 

In [ ]:
from collections import defaultdict
import re

category_map = {
    "ALL": "Total",
    "CWD": "Students with disabilities (IDEA)",
    "ECD": "Economically disadvantaged",
    "F": "Female",
    "FCS": "Foster Care Status",
    "HOM": "Homeless enrolled",
    "LEP": "English Learner",
    "M": "Male",
    "MAM": "American Indian/Alaska Native",
    "MAS": "Asian/Pacific Islander",
    "MBL": "Black",
    "MHI": "Hispanic",
    "MIG": "Migrant",
    "MIL": "Military Connected Student Status",
    "MTR": "Two or More Races",
    "MWH": "White"
}

grade_map = {
    "00": "All Grades",
    "03": "Grade 3",
    "04": "Grade 4",
    "05": "Grade 5",
    "06": "Grade 6",
    "07": "Grade 7",
    "08": "Grade 8",
    "HS": "High School Grades"
}

pct_proficient_symbols = {
    'GE': '≥',
    'LE': '≤',
    'GT': '>',
    'LT': '<',
    'PS': 'Data suppressed for privacy'
}

def convert_pct_proficient(value):
    # Check if the value is a string
    if isinstance(value, str):
        # Replace symbols with their corresponding abbreviations
        for abbreviation, symbol in pct_proficient_symbols.items():
            # Use regex to replace instances of the abbreviation followed by digits or in case of PS
            if abbreviation == 'PS':
                value = value.replace(abbreviation, symbol)
            else:
                value = re.sub(f'{abbreviation}(?=\d)', symbol, value)
    return value

def transform_reading_assessment_data(df):
    transformed_data = []
    for index, row in df.iterrows():
        school_id = row["NCESSCH"]
        institution_name = row["SCHNAM"]
        jurisdiction_name = row["LEANM"].title()
        lea_id = row["LEAID"]
        lea_name = row["LEANM"]
        year = row["SCHOOL_YEAR"][5:9]

        # Create a defaultdict to store reading assessments for each grade
        reading_assessments = defaultdict(dict)

        # Assuming grade_map and category_map are defined elsewhere in your code
        for grade_code in sorted(set(df[df["NCESSCH"] == school_id]["GRADE"])):
            # Use the grade_map dictionary to get the meaning of the grade code
            grade = grade_map.get(grade_code, "Unknown Grade")

            grade_data = df[(df["NCESSCH"] == school_id) & (df["GRADE"] == grade_code)]
            grade_assessments = {}
            for _, grade_row in grade_data.iterrows():
                category = category_map.get(grade_row["CATEGORY"], "Unknown Category")
                valid_assessments = grade_row["NUMVALID"]
                pct_proficient = grade_row["PCTPROF"]
                pct_proficient = convert_pct_proficient(pct_proficient)

                # Only add data if valid assessments are present
                if valid_assessments and valid_assessments > 0:
                    grade_assessments[category] = {
                        "validAssessments": int(valid_assessments),
                        "perProficient": pct_proficient
                    }

            # Only add grade data if grade_assessments is not empty
            if grade_assessments:
                reading_assessments[grade] = grade_assessments

        # Only proceed if reading_assessments is not empty
        if reading_assessments:
            # Create a dictionary with the relevant information including reading assessments
            school_info = {
                "institution_id": school_id,
                "institution_name": institution_name,
                "jurisdiction_name": jurisdiction_name,
                "lea_id": lea_id,
                "lea_name": lea_name,
                "year": year,
                "readingAssessment": dict(reading_assessments)
            }

            transformed_data.append(school_info)

    return transformed_data

reading_assessment_df_subset = reading_assessment_df #.head(200)

transformed_reading_assessment_data = transform_reading_assessment_data(reading_assessment_df_subset)
print(transformed_reading_assessment_data)

KeyboardInterrupt: 

In [ ]:
from collections import defaultdict
import json
import re
from tqdm import tqdm

category_map = {
    "ALL": "Total",
    "CWD": "Students with disabilities (IDEA)",
    "ECD": "Economically disadvantaged",
    "F": "Female",
    "FCS": "Foster Care Status",
    "HOM": "Homeless enrolled",
    "LEP": "English Learner",
    "M": "Male",
    "MAM": "American Indian/Alaska Native",
    "MAS": "Asian/Pacific Islander",
    "MBL": "Black",
    "MHI": "Hispanic",
    "MIG": "Migrant",
    "MIL": "Military Connected Student Status",
    "MTR": "Two or More Races",
    "MWH": "White"
}

grade_map = {
    "00": "All Grades",
    "03": "Grade 3",
    "04": "Grade 4",
    "05": "Grade 5",
    "06": "Grade 6",
    "07": "Grade 7",
    "08": "Grade 8",
    "HS": "High School Grades"
}

pct_proficient_symbols = {
    'GE': '≥',
    'LE': '≤',
    'GT': '>',
    'LT': '<',
    'PS': 'Data suppressed for privacy'
}

def convert_pct_proficient(value):
    # Check if the value is a string
    if isinstance(value, str):
        # Replace symbols with their corresponding abbreviations
        for abbreviation, symbol in pct_proficient_symbols.items():
            # Use regex to replace instances of the abbreviation followed by digits or in case of PS
            if abbreviation == 'PS':
                value = value.replace(abbreviation, symbol)
            else:
                value = re.sub(f'{abbreviation}(?=\d)', symbol, value)
    return value

def process_school_data(school_data):
    transformed_data = []

    first_row = school_data.iloc[0]
    school_id, institution_name, jurisdiction_name = first_row["NCESSCH"], first_row["SCHNAM"], first_row["STNAM"].title()
    lea_id, lea_name, year = first_row["LEAID"], first_row["LEANM"], first_row["SCHOOL_YEAR"][5:9]

    reading_assessments = defaultdict(dict)

    # Create a sorted list of grade codes based on the order in grade_map
    grade_codes = sorted(set(school_data["GRADE"].dropna().astype(str)), key=lambda x: list(grade_map).index(x) if x in grade_map else float('inf'))

    for grade_code in grade_codes:
        grade = grade_map.get(grade_code, "Unknown Grade")
        grade_data = school_data[school_data["GRADE"].astype(str) == grade_code]

        grade_assessments = {}
        for _, grade_row in grade_data.iterrows():
            category = category_map.get(grade_row["CATEGORY"], "Unknown Category")
            valid_assessments, pct_proficient = grade_row["NUMVALID"], convert_pct_proficient(grade_row["PCTPROF"])

            if valid_assessments and valid_assessments > 0:
                grade_assessments[category] = {
                    "validAssessments": int(valid_assessments),
                    "perProficient": pct_proficient
                }

        if grade_assessments:
            reading_assessments[grade] = grade_assessments

    if reading_assessments:
        transformed_data.append({
            "institution_id": school_id,
            "institution_name": institution_name,
            "jurisdiction_name": jurisdiction_name,
            "lea_id": lea_id,
            "lea_name": lea_name,
            "year": year,
            "readingAssessment": dict(reading_assessments)
        })

    return transformed_data

def transform_reading_assessment_data(df):
    transformed_data = []

    # Directly iterate over each group and process
    for _, group in tqdm(df.groupby("NCESSCH")):
        transformed_data.extend(process_school_data(group))

    return transformed_data

reading_assessment_df_subset = reading_assessment_df #.head(200)

print("Transforming reading assessment data")
transformed_reading_assessment_data = transform_reading_assessment_data(reading_assessment_df_subset)

#with open("reading_assessment_data.json", "w+") as f:
#    json.dump(transformed_reading_assessment_data, f)

print(transformed_reading_assessment_data[:5])

Transforming reading assessment data


100%|██████████| 68481/68481 [10:22<00:00, 110.04it/s]


[{'institution_id': '010000500870', 'institution_name': 'Albertville Middle School', 'jurisdiction_name': 'Alabama', 'lea_id': '0100005', 'lea_name': 'Albertville City', 'year': '2021', 'readingAssessment': {'All Grades': {'Total': {'validAssessments': 881, 'perProficient': '42'}, 'Students with disabilities (IDEA)': {'validAssessments': 74, 'perProficient': '10-14'}, 'Economically disadvantaged': {'validAssessments': 340, 'perProficient': '29'}, 'Female': {'validAssessments': 432, 'perProficient': '47'}, 'Foster Care Status': {'validAssessments': 1, 'perProficient': 'Data suppressed for privacy'}, 'Homeless enrolled': {'validAssessments': 5, 'perProficient': 'Data suppressed for privacy'}, 'English Learner': {'validAssessments': 159, 'perProficient': '≤5'}, 'Male': {'validAssessments': 449, 'perProficient': '37'}, 'American Indian/Alaska Native': {'validAssessments': 2, 'perProficient': 'Data suppressed for privacy'}, 'Asian/Pacific Islander': {'validAssessments': 3, 'perProficient': 

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

def insert_reading_assessment_data(session, data, batch_size=1000):
    # Calculate total number of batches for tqdm
    total_batches = len(data) // batch_size + (1 if len(data) % batch_size else 0)

    for i in tqdm(range(0, len(data), batch_size), total=total_batches, desc="Inserting batches"):
        batch = data[i:i + batch_size]
        cypher_query = """
        UNWIND $data AS schoolData
        MERGE (institution:Institution {ncesID: schoolData.institution_id})


        // Loop through each grade in 'readingAssessment'
        WITH schoolData, institution
        UNWIND keys(schoolData.readingAssessment) AS grade
        WITH institution, grade, schoolData.readingAssessment[grade] AS gradeAssessments, schoolData

        WITH schoolData, institution
        UNWIND keys(schoolData.readingAssessment) AS grade
        MERGE (outcome:Outcome {grade: grade, year: schoolData.year, name: grade + ' Reading Achievement'})
        WITH institution, outcome, schoolData.readingAssessment[grade] AS gradeData, grade, schoolData
        UNWIND keys(gradeData) AS subgroup
            MERGE (institution)-[reports:REPORTS {subgroup: subgroup, grade: grade, year: schoolData.year}]->(outcome)
            ON CREATE SET reports.validAssessments = gradeData[subgroup].validAssessments,
                          reports.percentProficient = gradeData[subgroup].perProficient
        """
        session.run(cypher_query, {"data": batch})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
with driver.session() as session:
    insert_reading_assessment_data(session, transformed_reading_assessment_data, batch_size=1000)

driver.close()

Inserting batches: 100%|██████████| 69/69 [14:32<00:00, 12.65s/it]


## Edfacts LEA Math Assessment

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
lea_math_assessment_path = './math-achievement-lea-sy2020-21.csv'
lea_math_assessment_df = pd.read_csv(lea_math_assessment_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("LEA Math Achievement Preview:")
print(lea_math_assessment_df.head(), "\n")


LEA Math Achievement Preview:
  SCHOOL_YEAR    STNAM  FIPST    LEAID ST_LEAID             LEANM SUBJECT  \
0   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City     MTH   
1   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City     MTH   
2   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City     MTH   
3   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City     MTH   
4   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City     MTH   

  GRADE CATEGORY DATE_CUR  NUMVALID PCTPROF  
0    00      ALL  30MAR22    3020.0      17  
1    00      CWD  30MAR22     264.0   10-14  
2    00      ECD  30MAR22    1218.0      11  
3    00        F  30MAR22    1500.0      16  
4    00      FCS  30MAR22       5.0      PS   



In [ ]:
from collections import defaultdict
import json
import re
from tqdm import tqdm

category_map = {
    "ALL": "Total",
    "CWD": "Students with disabilities (IDEA)",
    "ECD": "Economically disadvantaged",
    "F": "Female",
    "FCS": "Foster Care Status",
    "HOM": "Homeless enrolled",
    "LEP": "English Learner",
    "M": "Male",
    "MAM": "American Indian/Alaska Native",
    "MAS": "Asian/Pacific Islander",
    "MBL": "Black",
    "MHI": "Hispanic",
    "MIG": "Migrant",
    "MIL": "Military Connected Student Status",
    "MTR": "Two or More Races",
    "MWH": "White"
}

grade_map = {
    "00": "All Grades",
    "03": "Grade 3",
    "04": "Grade 4",
    "05": "Grade 5",
    "06": "Grade 6",
    "07": "Grade 7",
    "08": "Grade 8",
    "HS": "High School Grades"
}

pct_proficient_symbols = {
    'GE': '≥',
    'LE': '≤',
    'GT': '>',
    'LT': '<',
    'PS': 'Data suppressed for privacy'
}

def convert_pct_proficient(value):
    # Check if the value is a string
    if isinstance(value, str):
        # Replace symbols with their corresponding abbreviations
        for abbreviation, symbol in pct_proficient_symbols.items():
            # Use regex to replace instances of the abbreviation followed by digits or in case of PS
            if abbreviation == 'PS':
                value = value.replace(abbreviation, symbol)
            else:
                value = re.sub(f'{abbreviation}(?=\d)', symbol, value)
    return value

def process_lea_data(lea_data):
    transformed_data = []

    first_row = lea_data.iloc[0]
    jurisdiction_name = first_row["STNAM"].title()
    lea_id, lea_name, year = first_row["LEAID"], first_row["LEANM"], first_row["SCHOOL_YEAR"][5:9]

    math_assessments = defaultdict(dict)

    # Create a sorted list of grade codes based on the order in grade_map
    grade_codes = sorted(set(lea_data["GRADE"].dropna().astype(str)), key=lambda x: list(grade_map).index(x) if x in grade_map else float('inf'))

    for grade_code in grade_codes:
        grade = grade_map.get(grade_code, "Unknown Grade")
        grade_data = lea_data[lea_data["GRADE"].astype(str) == grade_code]

        grade_assessments = {}
        for _, grade_row in grade_data.iterrows():
            category = category_map.get(grade_row["CATEGORY"], "Unknown Category")
            valid_assessments, pct_proficient = grade_row["NUMVALID"], convert_pct_proficient(grade_row["PCTPROF"])

            if valid_assessments and valid_assessments > 0:
                grade_assessments[category] = {
                    "validAssessments": int(valid_assessments),
                    "perProficient": pct_proficient
                }

        if grade_assessments:
            math_assessments[grade] = grade_assessments

    if math_assessments:
        transformed_data.append({
            "jurisdiction_name": jurisdiction_name,
            "lea_id": lea_id,
            "lea_name": lea_name,
            "year": year,
            "mathAssessment": dict(math_assessments)
        })

    return transformed_data

def transform_lea_math_assessment_data(df):
    transformed_data = []

    # Directly iterate over each group and process
    for _, group in tqdm(df.groupby("LEAID")):
        transformed_data.extend(process_lea_data(group))

    return transformed_data

lea_math_assessment_df_subset = lea_math_assessment_df #.head(200)

print("Transforming math assessment data")
transformed_lea_math_assessment_data = transform_lea_math_assessment_data(lea_math_assessment_df_subset)

#with open("math_assessment_data.json", "w+") as f:
#    json.dump(transformed_math_assessment_data, f)

print(transformed_lea_math_assessment_data[:5])


Transforming math assessment data


100%|██████████| 13385/13385 [02:46<00:00, 80.24it/s]

[{'jurisdiction_name': 'Alabama', 'lea_id': '0100005', 'lea_name': 'Albertville City', 'year': '2021', 'mathAssessment': {'All Grades': {'Total': {'validAssessments': 3020, 'perProficient': '17'}, 'Students with disabilities (IDEA)': {'validAssessments': 264, 'perProficient': '10-14'}, 'Economically disadvantaged': {'validAssessments': 1218, 'perProficient': '11'}, 'Female': {'validAssessments': 1500, 'perProficient': '16'}, 'Foster Care Status': {'validAssessments': 5, 'perProficient': 'Data suppressed for privacy'}, 'Homeless enrolled': {'validAssessments': 14, 'perProficient': '<50'}, 'English Learner': {'validAssessments': 692, 'perProficient': '6'}, 'Male': {'validAssessments': 1520, 'perProficient': '18'}, 'American Indian/Alaska Native': {'validAssessments': 11, 'perProficient': '<50'}, 'Asian/Pacific Islander': {'validAssessments': 18, 'perProficient': '21-39'}, 'Black': {'validAssessments': 115, 'perProficient': '10-14'}, 'Hispanic': {'validAssessments': 1608, 'perProficient':

In [ ]:
import json
import numpy as np

# Custom JSON encoder subclass to convert NumPy types to Python native types for JSON serialization
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NumpyEncoder, self).default(obj)

# Use this custom encoder in your json.dump call
file_path = '/content/transformed_math_assessment_data.json'
with open(file_path, 'w') as file:
    json.dump(transformed_math_assessment_data, file, cls=NumpyEncoder, indent=4)

print(f"Data saved to {file_path}")

Data saved to /content/transformed_math_assessment_data.json


### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

def insert_math_assessment_data(session, data, batch_size=1000):
    # Calculate total number of batches for tqdm
    total_batches = len(data) // batch_size + (1 if len(data) % batch_size else 0)

    for i in tqdm(range(0, len(data), batch_size), total=total_batches, desc="Inserting batches"):
        batch = data[i:i + batch_size]
        cypher_query = """
        UNWIND $data AS leaData
        MERGE (LEA:LEA {ncesID: leaData.lea_id})


        // Loop through each grade in 'mathAssessment'
        WITH leaData, LEA
        UNWIND keys(leaData.mathAssessment) AS grade
        WITH LEA, grade, leaData.mathAssessment[grade] AS gradeAssessments, leaData

        WITH leaData, LEA
        UNWIND keys(leaData.mathAssessment) AS grade
        MERGE (outcome:Outcome {grade: grade, year: leaData.year, name: grade + ' Math Achievement'})
        WITH LEA, outcome, leaData.mathAssessment[grade] AS gradeData, grade, leaData
        UNWIND keys(gradeData) AS subgroup
            MERGE (LEA)-[reports:REPORTS {subgroup: subgroup, year: leaData.year}]->(outcome)
            ON CREATE SET reports.validAssessments = gradeData[subgroup].validAssessments,
                          reports.percentProficient = gradeData[subgroup].perProficient
        """
        session.run(cypher_query, {"data": batch})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
with driver.session() as session:
    insert_math_assessment_data(session, transformed_lea_math_assessment_data, batch_size=1000)

driver.close()

Inserting batches: 100%|██████████| 14/14 [05:44<00:00, 24.63s/it]


## Edfacts LEA Reading/LA Assessment

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
lea_reading_assessment_path = './rla-achievement-lea-sy2020-21.csv'
lea_reading_assessment_df = pd.read_csv(lea_reading_assessment_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("LEA Reading Achievement Preview:")
print(lea_reading_assessment_df.head(), "\n")

LEA Reading Achievement Preview:
  SCHOOL_YEAR    STNAM  FIPST    LEAID ST_LEAID             LEANM SUBJECT  \
0   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City     RLA   
1   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City     RLA   
2   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City     RLA   
3   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City     RLA   
4   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City     RLA   

  GRADE CATEGORY DATE_CUR  NUMVALID PCTPROF  
0    00      ALL  30MAR22    3007.0      39  
1    00      CWD  30MAR22     264.0   15-19  
2    00      ECD  30MAR22    1209.0      29  
3    00        F  30MAR22    1495.0      41  
4    00      FCS  30MAR22       5.0      PS   



In [ ]:
from collections import defaultdict
import json
import re
from tqdm import tqdm

category_map = {
    "ALL": "Total",
    "CWD": "Students with disabilities (IDEA)",
    "ECD": "Economically disadvantaged",
    "F": "Female",
    "FCS": "Foster Care Status",
    "HOM": "Homeless enrolled",
    "LEP": "English Learner",
    "M": "Male",
    "MAM": "American Indian/Alaska Native",
    "MAS": "Asian/Pacific Islander",
    "MBL": "Black",
    "MHI": "Hispanic",
    "MIG": "Migrant",
    "MIL": "Military Connected Student Status",
    "MTR": "Two or More Races",
    "MWH": "White"
}

grade_map = {
    "00": "All Grades",
    "03": "Grade 3",
    "04": "Grade 4",
    "05": "Grade 5",
    "06": "Grade 6",
    "07": "Grade 7",
    "08": "Grade 8",
    "HS": "High School Grades"
}

pct_proficient_symbols = {
    'GE': '≥',
    'LE': '≤',
    'GT': '>',
    'LT': '<',
    'PS': 'Data suppressed for privacy'
}

def convert_pct_proficient(value):
    # Check if the value is a string
    if isinstance(value, str):
        # Replace symbols with their corresponding abbreviations
        for abbreviation, symbol in pct_proficient_symbols.items():
            # Use regex to replace instances of the abbreviation followed by digits or in case of PS
            if abbreviation == 'PS':
                value = value.replace(abbreviation, symbol)
            else:
                value = re.sub(f'{abbreviation}(?=\d)', symbol, value)
    return value

def process_lea_data(lea_data):
    transformed_data = []

    first_row = lea_data.iloc[0]
    jurisdiction_name = first_row["STNAM"].title()
    lea_id, lea_name, year = first_row["LEAID"], first_row["LEANM"], first_row["SCHOOL_YEAR"][5:9]

    reading_assessments = defaultdict(dict)

    # Create a sorted list of grade codes based on the order in grade_map
    grade_codes = sorted(set(lea_data["GRADE"].dropna().astype(str)), key=lambda x: list(grade_map).index(x) if x in grade_map else float('inf'))

    for grade_code in grade_codes:
        grade = grade_map.get(grade_code, "Unknown Grade")
        grade_data = lea_data[lea_data["GRADE"].astype(str) == grade_code]

        grade_assessments = {}
        for _, grade_row in grade_data.iterrows():
            category = category_map.get(grade_row["CATEGORY"], "Unknown Category")
            valid_assessments, pct_proficient = grade_row["NUMVALID"], convert_pct_proficient(grade_row["PCTPROF"])

            if valid_assessments and valid_assessments > 0:
                grade_assessments[category] = {
                    "validAssessments": int(valid_assessments),
                    "perProficient": pct_proficient
                }

        if grade_assessments:
            reading_assessments[grade] = grade_assessments

    if reading_assessments:
        transformed_data.append({
            "jurisdiction_name": jurisdiction_name,
            "lea_id": lea_id,
            "lea_name": lea_name,
            "year": year,
            "readingAssessment": dict(reading_assessments)
        })

    return transformed_data

def transform_lea_reading_assessment_data(df):
    transformed_data = []

    # Directly iterate over each group and process
    for _, group in tqdm(df.groupby("LEAID")):
        transformed_data.extend(process_lea_data(group))

    return transformed_data

lea_reading_assessment_df_subset = lea_reading_assessment_df #.head(200)

print("Transforming math assessment data")
transformed_lea_reading_assessment_data = transform_lea_reading_assessment_data(lea_reading_assessment_df_subset)

#with open("math_assessment_data.json", "w+") as f:
#    json.dump(transformed_math_assessment_data, f)

print(transformed_lea_reading_assessment_data[:5])


Transforming math assessment data


100%|██████████| 16290/16290 [03:53<00:00, 69.85it/s] 

[{'jurisdiction_name': 'Alabama', 'lea_id': '0100005', 'lea_name': 'Albertville City', 'year': '2021', 'readingAssessment': {'All Grades': {'Total': {'validAssessments': 3007, 'perProficient': '39'}, 'Students with disabilities (IDEA)': {'validAssessments': 264, 'perProficient': '15-19'}, 'Economically disadvantaged': {'validAssessments': 1209, 'perProficient': '29'}, 'Female': {'validAssessments': 1495, 'perProficient': '41'}, 'Foster Care Status': {'validAssessments': 5, 'perProficient': 'Data suppressed for privacy'}, 'Homeless enrolled': {'validAssessments': 13, 'perProficient': '<50'}, 'English Learner': {'validAssessments': 675, 'perProficient': '9'}, 'Male': {'validAssessments': 1512, 'perProficient': '36'}, 'American Indian/Alaska Native': {'validAssessments': 11, 'perProficient': '<50'}, 'Asian/Pacific Islander': {'validAssessments': 17, 'perProficient': '40-59'}, 'Black': {'validAssessments': 113, 'perProficient': '30-34'}, 'Hispanic': {'validAssessments': 1594, 'perProficien

In [ ]:
import json
import numpy as np

# Custom JSON encoder subclass to convert NumPy types to Python native types for JSON serialization
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NumpyEncoder, self).default(obj)

# Use this custom encoder in your json.dump call
file_path = '/content/transformed_math_assessment_data.json'
with open(file_path, 'w') as file:
    json.dump(transformed_math_assessment_data, file, cls=NumpyEncoder, indent=4)

print(f"Data saved to {file_path}")

Data saved to /content/transformed_math_assessment_data.json


### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

def insert_reading_assessment_data(session, data, batch_size=1000):
    # Calculate total number of batches for tqdm
    total_batches = len(data) // batch_size + (1 if len(data) % batch_size else 0)

    for i in tqdm(range(0, len(data), batch_size), total=total_batches, desc="Inserting batches"):
        batch = data[i:i + batch_size]
        cypher_query = """
        UNWIND $data AS leaData
        MERGE (LEA:LEA {ncesID: leaData.lea_id})

        // Loop through each grade in 'readingAssessment'
        WITH leaData, LEA
        UNWIND keys(leaData.readingAssessment) AS grade
        WITH LEA, grade, leaData.readingAssessment[grade] AS gradeAssessments, leaData
        MERGE (outcome:Outcome {grade: grade, year: leaData.year, name: grade + ' Reading Achievement'})
        WITH LEA, outcome, leaData.readingAssessment[grade] AS gradeData, leaData
        UNWIND keys(gradeData) AS subgroup
            MERGE (LEA)-[reports:REPORTS {subgroup: subgroup, year: leaData.year}]->(outcome)
            ON CREATE SET reports.validAssessments = gradeData[subgroup].validAssessments,
                          reports.percentProficient = gradeData[subgroup].perProficient
        """
        session.run(cypher_query, {"data": batch})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
with driver.session() as session:
    insert_reading_assessment_data(session, transformed_lea_reading_assessment_data, batch_size=1000)

driver.close()

Inserting batches: 100%|██████████| 17/17 [03:30<00:00, 12.40s/it]


## Edfacts Math Assessment Participation

### Extract

In [ ]:
import pandas as pd

# Load the data
school_math_participation_data_path = './EDF_SCH_PART_MTH_2021_PUB_033022.csv'
school_math_participation_df = pd.read_csv(school_math_participation_data_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Math Participation Preview:")
print(school_math_participation_df.head(), "\n")

Math Participation Preview:
  SCHOOL_YEAR    STNAM  FIPST    LEAID ST_LEAID             LEANM  \
0   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   
1   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   
2   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   
3   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   
4   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   

        NCESSCH     ST_SCHID                     SCHNAM SUBJECT GRADE  \
0  010000500870  AL-101-0010  Albertville Middle School     MTH    00   
1  010000500870  AL-101-0010  Albertville Middle School     MTH    00   
2  010000500870  AL-101-0010  Albertville Middle School     MTH    00   
3  010000500870  AL-101-0010  Albertville Middle School     MTH    00   
4  010000500870  AL-101-0010  Albertville Middle School     MTH    00   

  CATEGORY DATE_CUR  NUMPART PCTPART  
0      ALL  30MAR22    884.0      98  
1      CWD  30MAR22     74.0    GE95  
2

In [ ]:
unique_schools_in_participation = set(school_math_participation_df['NCESSCH'].unique())
print(len(unique_schools_in_participation))

80621


In [ ]:
from collections import defaultdict
import json
import re
from tqdm import tqdm

category_map = {
    "ALL": "Total",
    "CWD": "Students with disabilities (IDEA)",
    "ECD": "Economically disadvantaged",
    "F": "Female",
    "FCS": "Foster Care Status",
    "HOM": "Homeless enrolled",
    "LEP": "English Learner",
    "M": "Male",
    "MAM": "American Indian/Alaska Native",
    "MAS": "Asian/Pacific Islander",
    "MBL": "Black",
    "MHI": "Hispanic",
    "MIG": "Migrant",
    "MIL": "Military Connected Student Status",
    "MTR": "Two or More Races",
    "MWH": "White"
}

grade_map = {
    "00": "All Grades",
    "03": "Grade 3",
    "04": "Grade 4",
    "05": "Grade 5",
    "06": "Grade 6",
    "07": "Grade 7",
    "08": "Grade 8",
    "HS": "High School Grades"
}

pct_proficient_symbols = {
    'GE': '≥',
    'LE': '≤',
    'GT': '>',
    'LT': '<',
    'PS': 'Data suppressed for privacy'
}

def convert_pct_proficient(value):
    # Check if the value is a string
    if isinstance(value, str):
        # Replace symbols with their corresponding abbreviations
        for abbreviation, symbol in pct_proficient_symbols.items():
            # Use regex to replace instances of the abbreviation followed by digits or in case of PS
            if abbreviation == 'PS':
                value = value.replace(abbreviation, symbol)
            else:
                value = re.sub(f'{abbreviation}(?=\d)', symbol, value)
    return value

def process_school_data(school_data):
    transformed_data = []

    first_row = school_data.iloc[0]
    school_id, institution_name, jurisdiction_name = first_row["NCESSCH"], first_row["SCHNAM"], first_row["STNAM"].title()
    lea_id, lea_name, year = first_row["LEAID"], first_row["LEANM"], first_row["SCHOOL_YEAR"][5:9]

    math_assessments = defaultdict(dict)

    # Create a sorted list of grade codes based on the order in grade_map
    grade_codes = sorted(set(school_data["GRADE"].dropna().astype(str)), key=lambda x: list(grade_map).index(x) if x in grade_map else float('inf'))

    for grade_code in grade_codes:
        grade = grade_map.get(grade_code, "Unknown Grade")
        grade_data = school_data[school_data["GRADE"].astype(str) == grade_code]

        grade_assessments = {}
        for _, grade_row in grade_data.iterrows():
            category = category_map.get(grade_row["CATEGORY"], "Unknown Category")
            num_participating, pct_participating = grade_row["NUMPART"], convert_pct_proficient(grade_row["PCTPART"])

            if num_participating and num_participating > 0:
                grade_assessments[category] = {
                    "numberParticipating": int(num_participating),
                    "perParticipating": pct_participating
                }

        if grade_assessments:
            math_assessments[grade] = grade_assessments

    if math_assessments:
        transformed_data.append({
            "institution_id": school_id,
            "institution_name": institution_name,
            "jurisdiction_name": jurisdiction_name,
            "lea_id": lea_id,
            "lea_name": lea_name,
            "year": year,
            "mathAssessment": dict(math_assessments)
        })

    return transformed_data

def transform_math_assessment_data(df):
    transformed_data = []

    # Directly iterate over each group and process
    for _, group in tqdm(df.groupby("NCESSCH")):
        transformed_data.extend(process_school_data(group))

    return transformed_data

school_math_participation_df_subset = school_math_participation_df #.head(200)

print("Transforming math assessment data")
transformed_math_assessment_part_data = transform_math_assessment_data(school_math_participation_df_subset)

#with open("math_assessment_data.json", "w+") as f:
#    json.dump(transformed_math_assessment_data, f)

print(transformed_math_assessment_part_data[:5])


Transforming math assessment data


100%|██████████| 80621/80621 [10:39<00:00, 126.09it/s]


[{'institution_id': '010000500870', 'institution_name': 'Albertville Middle School', 'jurisdiction_name': 'Alabama', 'lea_id': '0100005', 'lea_name': 'Albertville City', 'year': '2021', 'mathAssessment': {'All Grades': {'Total': {'numberParticipating': 884, 'perParticipating': '98'}, 'Students with disabilities (IDEA)': {'numberParticipating': 74, 'perParticipating': '≥95'}, 'Economically disadvantaged': {'numberParticipating': 342, 'perParticipating': '98'}, 'Female': {'numberParticipating': 435, 'perParticipating': '≥99'}, 'Foster Care Status': {'numberParticipating': 1, 'perParticipating': 'Data suppressed for privacy'}, 'Homeless enrolled': {'numberParticipating': 6, 'perParticipating': '≥50'}, 'English Learner': {'numberParticipating': 165, 'perParticipating': '≥95'}, 'Male': {'numberParticipating': 449, 'perParticipating': '98'}, 'American Indian/Alaska Native': {'numberParticipating': 2, 'perParticipating': 'Data suppressed for privacy'}, 'Asian/Pacific Islander': {'numberPartic

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

def insert_math_assessment_data(session, data, batch_size=1000):
    # Calculate total number of batches for tqdm
    total_batches = len(data) // batch_size + (1 if len(data) % batch_size else 0)

    for i in tqdm(range(0, len(data), batch_size), total=total_batches, desc="Inserting batches"):
        batch = data[i:i + batch_size]
        cypher_query = """
        UNWIND $data AS schoolData
        MATCH (institution:Institution {ncesID: schoolData.institution_id})
        UNWIND keys(schoolData.mathAssessment) AS grade
        MATCH (institution)-[reports:REPORTS {grade: grade, year: schoolData.year}]->(outcome:Outcome {name: grade + ' Math Achievement'})
        UNWIND keys(schoolData.mathAssessment[grade]) AS subgroup
        WITH reports, schoolData.mathAssessment[grade][subgroup] AS gradeData
        WHERE reports.subgroup = subgroup
        SET reports.percentParticipating = gradeData.perParticipating
        """
        session.run(cypher_query, {"data": batch})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
with driver.session() as session:
    insert_math_assessment_data(session, transformed_math_assessment_part_data, batch_size=1000)

driver.close()

NameError: name 'transformed_math_assessment_part_data' is not defined

## Edfacts Reading Assessment Participation

### Extract

In [ ]:
import pandas as pd

reading_participation_data_path = './EDF_SCH_PART_RLA_2021_PUB_033022.csv'

# Load the data
school_reading_participation_df = pd.read_csv(reading_participation_data_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Preview:")
print(school_reading_participation_df.head(), "\n")

Preview:
  SCHOOL_YEAR    STNAM  FIPST    LEAID ST_LEAID             LEANM  \
0   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   
1   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   
2   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   
3   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   
4   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   

        NCESSCH     ST_SCHID                     SCHNAM SUBJECT GRADE  \
0  010000500870  AL-101-0010  Albertville Middle School     RLA    00   
1  010000500870  AL-101-0010  Albertville Middle School     RLA    00   
2  010000500870  AL-101-0010  Albertville Middle School     RLA    00   
3  010000500870  AL-101-0010  Albertville Middle School     RLA    00   
4  010000500870  AL-101-0010  Albertville Middle School     RLA    00   

  CATEGORY DATE_CUR  NUMPART PCTPART  
0      ALL  30MAR22    890.0    GE99  
1      CWD  30MAR22     74.0    GE95  
2      ECD  30MAR22 

In [ ]:
unique_schools_in_reading_participation = set(school_reading_participation_df['NCESSCH'].unique())
print(len(unique_schools_in_reading_participation))

80561


In [ ]:
from collections import defaultdict
import json
import re
from tqdm import tqdm

category_map = {
    "ALL": "Total",
    "CWD": "Students with disabilities (IDEA)",
    "ECD": "Economically disadvantaged",
    "F": "Female",
    "FCS": "Foster Care Status",
    "HOM": "Homeless enrolled",
    "LEP": "English Learner",
    "M": "Male",
    "MAM": "American Indian/Alaska Native",
    "MAS": "Asian/Pacific Islander",
    "MBL": "Black",
    "MHI": "Hispanic",
    "MIG": "Migrant",
    "MIL": "Military Connected Student Status",
    "MTR": "Two or More Races",
    "MWH": "White"
}

grade_map = {
    "00": "All Grades",
    "03": "Grade 3",
    "04": "Grade 4",
    "05": "Grade 5",
    "06": "Grade 6",
    "07": "Grade 7",
    "08": "Grade 8",
    "HS": "High School Grades"
}

pct_proficient_symbols = {
    'GE': '≥',
    'LE': '≤',
    'GT': '>',
    'LT': '<',
    'PS': 'Data suppressed for privacy'
}

def convert_pct_proficient(value):
    # Check if the value is a string
    if isinstance(value, str):
        # Replace symbols with their corresponding abbreviations
        for abbreviation, symbol in pct_proficient_symbols.items():
            # Use regex to replace instances of the abbreviation followed by digits or in case of PS
            if abbreviation == 'PS':
                value = value.replace(abbreviation, symbol)
            else:
                value = re.sub(f'{abbreviation}(?=\d)', symbol, value)
    return value

def process_school_data(school_data):
    transformed_data = []

    first_row = school_data.iloc[0]
    school_id, institution_name, jurisdiction_name = first_row["NCESSCH"], first_row["SCHNAM"], first_row["STNAM"].title()
    lea_id, lea_name, year = first_row["LEAID"], first_row["LEANM"], first_row["SCHOOL_YEAR"][5:9]

    reading_assessments = defaultdict(dict)

    # Create a sorted list of grade codes based on the order in grade_map
    grade_codes = sorted(set(school_data["GRADE"].dropna().astype(str)), key=lambda x: list(grade_map).index(x) if x in grade_map else float('inf'))

    for grade_code in grade_codes:
        grade = grade_map.get(grade_code, "Unknown Grade")
        grade_data = school_data[school_data["GRADE"].astype(str) == grade_code]

        grade_assessments = {}
        for _, grade_row in grade_data.iterrows():
            category = category_map.get(grade_row["CATEGORY"], "Unknown Category")
            num_participating, pct_participating = grade_row["NUMPART"], convert_pct_proficient(grade_row["PCTPART"])

            if num_participating and num_participating > 0:
                grade_assessments[category] = {
                    "numberParticipating": int(num_participating),
                    "perParticipating": pct_participating
                }

        if grade_assessments:
            reading_assessments[grade] = grade_assessments

    if reading_assessments:
        transformed_data.append({
            "institution_id": school_id,
            "institution_name": institution_name,
            "jurisdiction_name": jurisdiction_name,
            "lea_id": lea_id,
            "lea_name": lea_name,
            "year": year,
            "readingAssessment": dict(reading_assessments)
        })

    return transformed_data

def transform_reading_assessment_data(df):
    transformed_data = []

    # Directly iterate over each group and process
    for _, group in tqdm(df.groupby("NCESSCH")):
        transformed_data.extend(process_school_data(group))

    return transformed_data

school_reading_participation_df_subset = school_reading_participation_df #.head(200)

print("Transforming reading assessment data")
transformed_reading_assessment_part_data = transform_reading_assessment_data(school_reading_participation_df_subset)

#with open("reading_assessment_data.json", "w+") as f:
#    json.dump(transformed_reading_assessment_data, f)

print(transformed_reading_assessment_part_data[:5])


Transforming reading assessment data


100%|██████████| 80561/80561 [10:55<00:00, 122.96it/s]

[{'institution_id': '010000500870', 'institution_name': 'Albertville Middle School', 'jurisdiction_name': 'Alabama', 'lea_id': '0100005', 'lea_name': 'Albertville City', 'year': '2021', 'readingAssessment': {'All Grades': {'Total': {'numberParticipating': 890, 'perParticipating': '≥99'}, 'Students with disabilities (IDEA)': {'numberParticipating': 74, 'perParticipating': '≥95'}, 'Economically disadvantaged': {'numberParticipating': 343, 'perParticipating': '98'}, 'Female': {'numberParticipating': 438, 'perParticipating': '≥99'}, 'Foster Care Status': {'numberParticipating': 1, 'perParticipating': 'Data suppressed for privacy'}, 'Homeless enrolled': {'numberParticipating': 6, 'perParticipating': '≥50'}, 'English Learner': {'numberParticipating': 168, 'perParticipating': '≥95'}, 'Male': {'numberParticipating': 452, 'perParticipating': '98'}, 'American Indian/Alaska Native': {'numberParticipating': 2, 'perParticipating': 'Data suppressed for privacy'}, 'Asian/Pacific Islander': {'numberPa

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

def insert_reading_assessment_data(session, data, batch_size=1000):
    # Calculate total number of batches for tqdm
    total_batches = len(data) // batch_size + (1 if len(data) % batch_size else 0)

    for i in tqdm(range(0, len(data), batch_size), total=total_batches, desc="Inserting batches"):
        batch = data[i:i + batch_size]
        cypher_query = """
        UNWIND $data AS schoolData
        MATCH (institution:Institution {ncesID: schoolData.institution_id})
        UNWIND keys(schoolData.readingAssessment) AS grade
        MATCH (institution)-[reports:REPORTS {grade: grade, year: schoolData.year}]->(outcome:Outcome {name: grade + ' Reading/LA Achievement'})
        UNWIND keys(schoolData.readingAssessment[grade]) AS subgroup
        WITH reports, schoolData.readingAssessment[grade][subgroup] AS gradeData
        WHERE reports.subgroup = subgroup
        SET reports.percentParticipating = gradeData.perParticipating
        """
        session.run(cypher_query, {"data": batch})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
with driver.session() as session:
    insert_reading_assessment_data(session, transformed_reading_assessment_part_data, batch_size=1000)

driver.close()

Inserting batches: 100%|██████████| 81/81 [05:11<00:00,  3.84s/it]


## Edfacts LEA Math Assessment Participation

### Extract

In [ ]:
import pandas as pd

# Load the  data
lea_math_assessment_participation_path = './math-participation-lea-sy2020-21.csv'
lea_math_assessment_participation_df = pd.read_csv(lea_math_assessment_participation_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("LEA Math Participation Preview:")
print(lea_math_assessment_participation_df.head(), "\n")


LEA Math Participation Preview:
  SCHOOL_YEAR    STNAM  FIPST    LEAID ST_LEAID             LEANM SUBJECT  \
0   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City     MTH   
1   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City     MTH   
2   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City     MTH   
3   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City     MTH   
4   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City     MTH   

  GRADE CATEGORY DATE_CUR  NUMPART PCTPART  
0    00      ALL  30MAR22   3020.0      98  
1    00      CWD  30MAR22    264.0    GE95  
2    00      ECD  30MAR22   1218.0      98  
3    00        F  30MAR22   1500.0      98  
4    00      FCS  30MAR22      5.0      PS   



In [ ]:
from collections import defaultdict
import json
import re
from tqdm import tqdm

category_map = {
    "ALL": "Total",
    "CWD": "Students with disabilities (IDEA)",
    "ECD": "Economically disadvantaged",
    "F": "Female",
    "FCS": "Foster Care Status",
    "HOM": "Homeless enrolled",
    "LEP": "English Learner",
    "M": "Male",
    "MAM": "American Indian/Alaska Native",
    "MAS": "Asian/Pacific Islander",
    "MBL": "Black",
    "MHI": "Hispanic",
    "MIG": "Migrant",
    "MIL": "Military Connected Student Status",
    "MTR": "Two or More Races",
    "MWH": "White"
}

grade_map = {
    "00": "All Grades",
    "03": "Grade 3",
    "04": "Grade 4",
    "05": "Grade 5",
    "06": "Grade 6",
    "07": "Grade 7",
    "08": "Grade 8",
    "HS": "High School Grades"
}

pct_proficient_symbols = {
    'GE': '≥',
    'LE': '≤',
    'GT': '>',
    'LT': '<',
    'PS': 'Data suppressed for privacy'
}

def convert_pct_proficient(value):
    # Check if the value is a string
    if isinstance(value, str):
        # Replace symbols with their corresponding abbreviations
        for abbreviation, symbol in pct_proficient_symbols.items():
            # Use regex to replace instances of the abbreviation followed by digits or in case of PS
            if abbreviation == 'PS':
                value = value.replace(abbreviation, symbol)
            else:
                value = re.sub(f'{abbreviation}(?=\d)', symbol, value)
    return value

def process_lea_participation_data(lea_data):
    transformed_data = []

    first_row = lea_data.iloc[0]
    jurisdiction_name = first_row["STNAM"].title()
    lea_id, lea_name, year = first_row["LEAID"], first_row["LEANM"], first_row["SCHOOL_YEAR"][5:9]

    math_assessments = defaultdict(dict)

    # Create a sorted list of grade codes based on the order in grade_map
    grade_codes = sorted(set(lea_data["GRADE"].dropna().astype(str)), key=lambda x: list(grade_map).index(x) if x in grade_map else float('inf'))

    for grade_code in grade_codes:
        grade = grade_map.get(grade_code, "Unknown Grade")
        grade_data = lea_data[lea_data["GRADE"].astype(str) == grade_code]

        grade_assessments = {}
        for _, grade_row in grade_data.iterrows():
            category = category_map.get(grade_row["CATEGORY"], "Unknown Category")
            number_participating, pct_participating = grade_row["NUMPART"], convert_pct_proficient(grade_row["PCTPART"])

            if number_participating and number_participating > 0:
                grade_assessments[category] = {
                    "numberParticipating": int(number_participating),
                    "perParticipating": pct_participating
                }

        if grade_assessments:
            math_assessments[grade] = grade_assessments

    if math_assessments:
        transformed_data.append({
            "jurisdiction_name": jurisdiction_name,
            "lea_id": lea_id,
            "lea_name": lea_name,
            "year": year,
            "mathAssessment": dict(math_assessments)
        })

    return transformed_data

def transform_lea_math_assessment_participation_data(df):
    transformed_data = []

    # Directly iterate over each group and process
    for _, group in tqdm(df.groupby("LEAID")):
        transformed_data.extend(process_lea_participation_data(group))

    return transformed_data

lea_math_assessment_participation_df_subset = lea_math_assessment_participation_df #.head(200)

print("Transforming math assessment data")
transformed_lea_math_assessment_participation_data = transform_lea_math_assessment_participation_data(lea_math_assessment_participation_df_subset)

#with open("math_assessment_data.json", "w+") as f:
#    json.dump(transformed_math_assessment_data, f)

print(transformed_lea_math_assessment_participation_data[:5])


Transforming math assessment data


100%|██████████| 16301/16301 [03:58<00:00, 68.31it/s]


[{'jurisdiction_name': 'Alabama', 'lea_id': '0100005', 'lea_name': 'Albertville City', 'year': '2021', 'mathAssessment': {'All Grades': {'Total': {'numberParticipating': 3020, 'perParticipating': '98'}, 'Students with disabilities (IDEA)': {'numberParticipating': 264, 'perParticipating': '≥95'}, 'Economically disadvantaged': {'numberParticipating': 1218, 'perParticipating': '98'}, 'Female': {'numberParticipating': 1500, 'perParticipating': '98'}, 'Foster Care Status': {'numberParticipating': 5, 'perParticipating': 'Data suppressed for privacy'}, 'Homeless enrolled': {'numberParticipating': 14, 'perParticipating': '≥80'}, 'English Learner': {'numberParticipating': 692, 'perParticipating': '97'}, 'Male': {'numberParticipating': 1520, 'perParticipating': '98'}, 'American Indian/Alaska Native': {'numberParticipating': 11, 'perParticipating': '≥50'}, 'Asian/Pacific Islander': {'numberParticipating': 18, 'perParticipating': '≥80'}, 'Black': {'numberParticipating': 115, 'perParticipating': '≥

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

def insert_math_assessment_participation_data(session, data, batch_size=1000):
    total_batches = len(data) // batch_size + (1 if len(data) % batch_size else 0)

    for i in tqdm(range(0, len(data), batch_size), total=total_batches, desc="Inserting batches"):
        batch = data[i:i + batch_size]
        cypher_query = """
        UNWIND $data AS leaData
        MATCH (lea:LEA {ncesID: leaData.lea_id})
        UNWIND keys(leaData.mathAssessment) AS gradeKey
        WITH lea, gradeKey, leaData.year AS year, leaData.mathAssessment[gradeKey] AS gradeDetails
        UNWIND keys(gradeDetails) AS subgroupKey
        MATCH (lea)-[reports:REPORTS {year: year, subgroup: subgroupKey}]->(outcome:Outcome {name: gradeKey + ' Math Achievement'})
        WITH reports, lea, gradeKey, subgroupKey, gradeDetails[subgroupKey] AS details
        SET reports.percentParticipating = details.perParticipating
        RETURN lea.ncesID AS LEA_ID, gradeKey AS Grade, subgroupKey AS Subgroup, reports.percentParticipating AS Updated_PercentParticipating

        """
        session.run(cypher_query, {"data": batch})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
with driver.session() as session:
    insert_math_assessment_participation_data(session, transformed_lea_math_assessment_participation_data, batch_size=1000)

driver.close()

Inserting batches: 100%|██████████| 17/17 [04:40<00:00, 16.49s/it]


## Edfacts LEA Reading Assessment Participation

### Extract

In [ ]:
import pandas as pd

# Load the  data
lea_rla_assessment_participation_path = './rla-participation-lea-sy2020-21.csv'
lea_rla_assessment_participation_df = pd.read_csv(lea_rla_assessment_participation_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("LEA reading Participation Preview:")
print(lea_rla_assessment_participation_df.head(), "\n")


LEA reading Participation Preview:
  SCHOOL_YEAR    STNAM  FIPST    LEAID ST_LEAID             LEANM SUBJECT  \
0   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City     RLA   
1   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City     RLA   
2   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City     RLA   
3   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City     RLA   
4   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City     RLA   

  GRADE CATEGORY DATE_CUR  NUMPART PCTPART  
0    00      ALL  30MAR22   3021.0      98  
1    00      CWD  30MAR22    264.0    GE95  
2    00      ECD  30MAR22   1215.0      98  
3    00        F  30MAR22   1505.0    GE99  
4    00      FCS  30MAR22      5.0      PS   



###Transform

In [ ]:
from collections import defaultdict
import json
import re
from tqdm import tqdm

category_map = {
    "ALL": "Total",
    "CWD": "Students with disabilities (IDEA)",
    "ECD": "Economically disadvantaged",
    "F": "Female",
    "FCS": "Foster Care Status",
    "HOM": "Homeless enrolled",
    "LEP": "English Learner",
    "M": "Male",
    "MAM": "American Indian/Alaska Native",
    "MAS": "Asian/Pacific Islander",
    "MBL": "Black",
    "MHI": "Hispanic",
    "MIG": "Migrant",
    "MIL": "Military Connected Student Status",
    "MTR": "Two or More Races",
    "MWH": "White"
}

grade_map = {
    "00": "All Grades",
    "03": "Grade 3",
    "04": "Grade 4",
    "05": "Grade 5",
    "06": "Grade 6",
    "07": "Grade 7",
    "08": "Grade 8",
    "HS": "High School Grades"
}

pct_proficient_symbols = {
    'GE': '≥',
    'LE': '≤',
    'GT': '>',
    'LT': '<',
    'PS': 'Data suppressed for privacy'
}

def convert_pct_proficient(value):
    # Check if the value is a string
    if isinstance(value, str):
        # Replace symbols with their corresponding abbreviations
        for abbreviation, symbol in pct_proficient_symbols.items():
            # Use regex to replace instances of the abbreviation followed by digits or in case of PS
            if abbreviation == 'PS':
                value = value.replace(abbreviation, symbol)
            else:
                value = re.sub(f'{abbreviation}(?=\d)', symbol, value)
    return value

def process_lea_participation_data(lea_data):
    transformed_data = []

    first_row = lea_data.iloc[0]
    jurisdiction_name = first_row["STNAM"].title()
    lea_id, lea_name, year = first_row["LEAID"], first_row["LEANM"], first_row["SCHOOL_YEAR"][5:9]

    rla_assessments = defaultdict(dict)

    # Create a sorted list of grade codes based on the order in grade_map
    grade_codes = sorted(set(lea_data["GRADE"].dropna().astype(str)), key=lambda x: list(grade_map).index(x) if x in grade_map else float('inf'))

    for grade_code in grade_codes:
        grade = grade_map.get(grade_code, "Unknown Grade")
        grade_data = lea_data[lea_data["GRADE"].astype(str) == grade_code]

        grade_assessments = {}
        for _, grade_row in grade_data.iterrows():
            category = category_map.get(grade_row["CATEGORY"], "Unknown Category")
            number_participating, pct_participating = grade_row["NUMPART"], convert_pct_proficient(grade_row["PCTPART"])

            if number_participating and number_participating > 0:
                grade_assessments[category] = {
                    "numberParticipating": int(number_participating),
                    "perParticipating": pct_participating
                }

        if grade_assessments:
            rla_assessments[grade] = grade_assessments

    if rla_assessments:
        transformed_data.append({
            "jurisdiction_name": jurisdiction_name,
            "lea_id": lea_id,
            "lea_name": lea_name,
            "year": year,
            "rlaAssessment": dict(rla_assessments)
        })

    return transformed_data

def transform_lea_rla_assessment_participation_data(df):
    transformed_data = []

    # Directly iterate over each group and process
    for _, group in tqdm(df.groupby("LEAID")):
        transformed_data.extend(process_lea_participation_data(group))

    return transformed_data

lea_rla_assessment_participation_df_subset = lea_rla_assessment_participation_df #.head(200)

print("Transforming rla assessment data")
transformed_lea_rla_assessment_participation_data = transform_lea_rla_assessment_participation_data(lea_rla_assessment_participation_df_subset)

#with open("rla_assessment_data.json", "w+") as f:
#    json.dump(transformed_rla_assessment_data, f)

print(transformed_lea_rla_assessment_participation_data[:5])


Transforming rla assessment data


100%|██████████| 16293/16293 [04:50<00:00, 56.13it/s] 


[{'jurisdiction_name': 'Alabama', 'lea_id': '0100005', 'lea_name': 'Albertville City', 'year': '2021', 'rlaAssessment': {'All Grades': {'Total': {'numberParticipating': 3021, 'perParticipating': '98'}, 'Students with disabilities (IDEA)': {'numberParticipating': 264, 'perParticipating': '≥95'}, 'Economically disadvantaged': {'numberParticipating': 1215, 'perParticipating': '98'}, 'Female': {'numberParticipating': 1505, 'perParticipating': '≥99'}, 'Foster Care Status': {'numberParticipating': 5, 'perParticipating': 'Data suppressed for privacy'}, 'Homeless enrolled': {'numberParticipating': 14, 'perParticipating': '≥80'}, 'English Learner': {'numberParticipating': 689, 'perParticipating': '96'}, 'Male': {'numberParticipating': 1516, 'perParticipating': '97'}, 'American Indian/Alaska Native': {'numberParticipating': 11, 'perParticipating': '≥50'}, 'Asian/Pacific Islander': {'numberParticipating': 18, 'perParticipating': '≥80'}, 'Black': {'numberParticipating': 117, 'perParticipating': '≥

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

def insert_rla_assessment_participation_data(session, data, batch_size=1000):
    total_batches = len(data) // batch_size + (1 if len(data) % batch_size else 0)

    for i in tqdm(range(0, len(data), batch_size), total=total_batches, desc="Inserting batches"):
        batch = data[i:i + batch_size]
        cypher_query = """
        UNWIND $data AS leaData
        MATCH (lea:LEA {ncesID: leaData.lea_id})
        UNWIND keys(leaData.rlaAssessment) AS gradeKey
        WITH lea, gradeKey, leaData.year AS year, leaData.rlaAssessment[gradeKey] AS gradeDetails
        UNWIND keys(gradeDetails) AS subgroupKey
        MATCH (lea)-[reports:REPORTS {year: year, subgroup: subgroupKey}]->(outcome:Outcome {name: gradeKey + ' Reading Achievement'})
        WITH reports, lea, gradeKey, subgroupKey, gradeDetails[subgroupKey] AS details
        SET reports.percentParticipating = details.perParticipating
        RETURN lea.ncesID AS LEA_ID, gradeKey AS Grade, subgroupKey AS Subgroup, reports.percentParticipating AS Updated_PercentParticipating

        """
        session.run(cypher_query, {"data": batch})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
with driver.session() as session:
    insert_rla_assessment_participation_data(session, transformed_lea_rla_assessment_participation_data, batch_size=1000)

driver.close()

Inserting batches: 100%|██████████| 17/17 [05:18<00:00, 18.71s/it]


## Edfacts School Graduation

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
school_graduation_path = './acgr-sch-sy2020-21-long.csv'
school_graduation_df = pd.read_csv(school_graduation_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("School Support Preview:")
print(school_graduation_df.head(), "\n")

School Support Preview:
  SCHOOL_YEAR    STNAM  FIPST    LEAID ST_LEAID             LEANM  \
0   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   
1   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   
2   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   
3   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   
4   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   

        NCESSCH     ST_SCHID                   SCHNAM CATEGORY  COHORT   RATE  \
0  010000500871  AL-101-0020  Albertville High School      ALL     385     93   
1  010000500871  AL-101-0020  Albertville High School      CWD      20  60-79   
2  010000500871  AL-101-0020  Albertville High School      ECD     174  90-94   
3  010000500871  AL-101-0020  Albertville High School      HOM       2     PS   
4  010000500871  AL-101-0020  Albertville High School      LEP      57  80-89   

  DATE_CUR  
0  25MAY22  
1  25MAY22  
2  25MAY22  
3  25MAY22  
4  25MAY2

In [ ]:
unique_schools_in_graduation = set(school_graduation_df['NCESSCH'].unique())
print(len(unique_schools_in_graduation))

22066


In [ ]:
from collections import defaultdict
import json
import re
from tqdm import tqdm

category_map = {
    "ALL": "Total",
    "CWD": "Students with disabilities (IDEA)",
    "ECD": "Economically disadvantaged",
    "FCS": "Foster Care Status",
    "HOM": "Homeless enrolled",
    "LEP": "English Learner",
    "MAM": "American Indian/Alaska Native",
    "MAS": "Asian/Pacific Islander",
    "MBL": "Black",
    "MHI": "Hispanic",
    "MIG": "Migrant",
    "MIL": "Military Connected Student Status",
    "MTR": "Two or More Races",
    "MWH": "White"
}

pct_proficient_symbols = {
    'GE': '≥',
    'LE': '≤',
    'GT': '>',
    'LT': '<',
    'PS': 'Data suppressed for privacy'
}

def convert_rate(value):
    # Check if the value is a string
    if isinstance(value, str):
        # Replace symbols with their corresponding abbreviations
        for abbreviation, symbol in pct_proficient_symbols.items():
            # Use regex to replace instances of the abbreviation followed by digits or in case of PS
            if abbreviation == 'PS':
                value = value.replace(abbreviation, symbol)
            else:
                value = re.sub(f'{abbreviation}(?=\d)', symbol, value)
    return value

def process_school_data(school_data):
    transformed_data = []

    first_row = school_data.iloc[0]
    institution_id, institution_name, jurisdiction_name = first_row["NCESSCH"], first_row["SCHNAM"], first_row["STNAM"].title()
    lea_id, lea_name, year = first_row["LEAID"], first_row["LEANM"], first_row["SCHOOL_YEAR"][5:9]

    # Initialize dictionary to hold graduation data by category
    graduation_rates = {}

    for _, row in school_data.iterrows():
        category_code = row["CATEGORY"]
        category_description = category_map.get(category_code, "Unknown Category")
        cohort_size = row["COHORT"]
        graduation_rate = convert_rate(row["RATE"])

        graduation_rates[category_description] = {
            "cohortSize": cohort_size,
            "graduationRate": graduation_rate
        }

    if graduation_rates:
        transformed_data.append({
            "institution_id": institution_id,
            "institution_name": institution_name,
            "lea_id": lea_id,
            "lea_name": lea_name,
            "year": year[-4:],
            "graduationRates": graduation_rates
        })

    return transformed_data

def transform_school_graduation_data(df):
    transformed_data = []

    # Directly iterate over each group and process
    for _, group in tqdm(df.groupby("NCESSCH")):
        transformed_data.extend(process_school_data(group))

    return transformed_data

school_graduation_df_subset = school_graduation_df #.head(200)

print("Transforming school_graduation data")
transformed_school_graduation_data = transform_school_graduation_data(school_graduation_df_subset)

#with open("school_graduation_data.json", "w+") as f:
#    json.dump(transformed_school_graduation_data, f)

print(transformed_school_graduation_data[:5])


Transforming school_graduation data


100%|██████████| 22066/22066 [00:27<00:00, 800.96it/s]

[{'institution_id': '010000500871', 'institution_name': 'Albertville High School', 'lea_id': '0100005', 'lea_name': 'Albertville City', 'year': '2021', 'graduationRates': {'Total': {'cohortSize': 385, 'graduationRate': '93'}, 'Students with disabilities (IDEA)': {'cohortSize': 20, 'graduationRate': '60-79'}, 'Economically disadvantaged': {'cohortSize': 174, 'graduationRate': '90-94'}, 'Homeless enrolled': {'cohortSize': 2, 'graduationRate': 'Data suppressed for privacy'}, 'English Learner': {'cohortSize': 57, 'graduationRate': '80-89'}, 'American Indian/Alaska Native': {'cohortSize': 1, 'graduationRate': 'Data suppressed for privacy'}, 'Asian/Pacific Islander': {'cohortSize': 1, 'graduationRate': 'Data suppressed for privacy'}, 'Black': {'cohortSize': 17, 'graduationRate': '≥80'}, 'Hispanic': {'cohortSize': 170, 'graduationRate': '90-94'}, 'Two or More Races': {'cohortSize': 13, 'graduationRate': '≥50'}, 'White': {'cohortSize': 183, 'graduationRate': '90-94'}}}, {'institution_id': '010

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

def insert_school_graduation_data(session, data, batch_size=1000):
    # Calculate total number of batches for tqdm
    total_batches = len(data) // batch_size + (1 if len(data) % batch_size else 0)

    for i in tqdm(range(0, len(data), batch_size), total=total_batches, desc="Inserting batches"):
        batch = data[i:i + batch_size]
        cypher_query = """
        UNWIND $data AS schoolData
        MERGE (institution:Institution {ncesID: schoolData.institution_id})

        WITH schoolData, institution
        MERGE (outcome:Outcome {name: 'High School Graduation'})

        WITH institution, outcome, schoolData, schoolData.graduationRates AS gradData
        UNWIND keys(gradData) AS subgroup
        MERGE (institution)-[reports:REPORTS {subgroup: subgroup, year: schoolData.year}]->(outcome)
        ON CREATE SET reports.cohortSize = gradData[subgroup].cohortSize, reports.graduationRate = gradData[subgroup].graduationRate
        ON MATCH SET reports.cohortSize = gradData[subgroup].cohortSize, reports.graduationRate = gradData[subgroup].graduationRate
        """
        session.run(cypher_query, {"data": batch})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
with driver.session() as session:
    insert_school_graduation_data(session, transformed_school_graduation_data, batch_size=1000)

driver.close()

Inserting batches: 100%|██████████| 23/23 [00:41<00:00,  1.82s/it]


## Edfacts School Graduation

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
school_graduation_path = './acgr-sch-sy2020-21-long.csv'
school_graduation_df = pd.read_csv(school_graduation_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("School Support Preview:")
print(school_graduation_df.head(), "\n")

School Support Preview:
  SCHOOL_YEAR    STNAM  FIPST    LEAID ST_LEAID             LEANM  \
0   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   
1   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   
2   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   
3   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   
4   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City   

        NCESSCH     ST_SCHID                   SCHNAM CATEGORY  COHORT   RATE  \
0  010000500871  AL-101-0020  Albertville High School      ALL     385     93   
1  010000500871  AL-101-0020  Albertville High School      CWD      20  60-79   
2  010000500871  AL-101-0020  Albertville High School      ECD     174  90-94   
3  010000500871  AL-101-0020  Albertville High School      HOM       2     PS   
4  010000500871  AL-101-0020  Albertville High School      LEP      57  80-89   

  DATE_CUR  
0  25MAY22  
1  25MAY22  
2  25MAY22  
3  25MAY22  
4  25MAY2

In [ ]:
unique_schools_in_graduation = set(school_graduation_df['NCESSCH'].unique())
print(len(unique_schools_in_graduation))

22066


In [ ]:
from collections import defaultdict
import json
import re
from tqdm import tqdm

category_map = {
    "ALL": "Total",
    "CWD": "Students with disabilities (IDEA)",
    "ECD": "Economically disadvantaged",
    "FCS": "Foster Care Status",
    "HOM": "Homeless enrolled",
    "LEP": "English Learner",
    "MAM": "American Indian/Alaska Native",
    "MAS": "Asian/Pacific Islander",
    "MBL": "Black",
    "MHI": "Hispanic",
    "MIG": "Migrant",
    "MIL": "Military Connected Student Status",
    "MTR": "Two or More Races",
    "MWH": "White"
}

pct_proficient_symbols = {
    'GE': '≥',
    'LE': '≤',
    'GT': '>',
    'LT': '<',
    'PS': 'Data suppressed for privacy'
}

def convert_rate(value):
    # Check if the value is a string
    if isinstance(value, str):
        # Replace symbols with their corresponding abbreviations
        for abbreviation, symbol in pct_proficient_symbols.items():
            # Use regex to replace instances of the abbreviation followed by digits or in case of PS
            if abbreviation == 'PS':
                value = value.replace(abbreviation, symbol)
            else:
                value = re.sub(f'{abbreviation}(?=\d)', symbol, value)
    return value

def process_school_data(school_data):
    transformed_data = []

    first_row = school_data.iloc[0]
    institution_id, institution_name, jurisdiction_name = first_row["NCESSCH"], first_row["SCHNAM"], first_row["STNAM"].title()
    lea_id, lea_name, year = first_row["LEAID"], first_row["LEANM"], first_row["SCHOOL_YEAR"][5:9]

    # Initialize dictionary to hold graduation data by category
    graduation_rates = {}

    for _, row in school_data.iterrows():
        category_code = row["CATEGORY"]
        category_description = category_map.get(category_code, "Unknown Category")
        cohort_size = row["COHORT"]
        graduation_rate = convert_rate(row["RATE"])

        graduation_rates[category_description] = {
            "cohortSize": cohort_size,
            "graduationRate": graduation_rate
        }

    if graduation_rates:
        transformed_data.append({
            "institution_id": institution_id,
            "institution_name": institution_name,
            "lea_id": lea_id,
            "lea_name": lea_name,
            "year": year[-4:],
            "graduationRates": graduation_rates
        })

    return transformed_data

def transform_school_graduation_data(df):
    transformed_data = []

    # Directly iterate over each group and process
    for _, group in tqdm(df.groupby("NCESSCH")):
        transformed_data.extend(process_school_data(group))

    return transformed_data

school_graduation_df_subset = school_graduation_df #.head(200)

print("Transforming school_graduation data")
transformed_school_graduation_data = transform_school_graduation_data(school_graduation_df_subset)

#with open("school_graduation_data.json", "w+") as f:
#    json.dump(transformed_school_graduation_data, f)

print(transformed_school_graduation_data[:5])


Transforming school_graduation data


100%|██████████| 22066/22066 [00:27<00:00, 800.96it/s]

[{'institution_id': '010000500871', 'institution_name': 'Albertville High School', 'lea_id': '0100005', 'lea_name': 'Albertville City', 'year': '2021', 'graduationRates': {'Total': {'cohortSize': 385, 'graduationRate': '93'}, 'Students with disabilities (IDEA)': {'cohortSize': 20, 'graduationRate': '60-79'}, 'Economically disadvantaged': {'cohortSize': 174, 'graduationRate': '90-94'}, 'Homeless enrolled': {'cohortSize': 2, 'graduationRate': 'Data suppressed for privacy'}, 'English Learner': {'cohortSize': 57, 'graduationRate': '80-89'}, 'American Indian/Alaska Native': {'cohortSize': 1, 'graduationRate': 'Data suppressed for privacy'}, 'Asian/Pacific Islander': {'cohortSize': 1, 'graduationRate': 'Data suppressed for privacy'}, 'Black': {'cohortSize': 17, 'graduationRate': '≥80'}, 'Hispanic': {'cohortSize': 170, 'graduationRate': '90-94'}, 'Two or More Races': {'cohortSize': 13, 'graduationRate': '≥50'}, 'White': {'cohortSize': 183, 'graduationRate': '90-94'}}}, {'institution_id': '010

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

def insert_school_graduation_data(session, data, batch_size=1000):
    # Calculate total number of batches for tqdm
    total_batches = len(data) // batch_size + (1 if len(data) % batch_size else 0)

    for i in tqdm(range(0, len(data), batch_size), total=total_batches, desc="Inserting batches"):
        batch = data[i:i + batch_size]
        cypher_query = """
        UNWIND $data AS schoolData
        MERGE (institution:Institution {ncesID: schoolData.institution_id})

        WITH schoolData, institution
        MERGE (outcome:Outcome {name: 'High School Graduation'})

        WITH institution, outcome, schoolData, schoolData.graduationRates AS gradData
        UNWIND keys(gradData) AS subgroup
        MERGE (institution)-[reports:REPORTS {subgroup: subgroup, year: schoolData.year}]->(outcome)
        ON CREATE SET reports.cohortSize = gradData[subgroup].cohortSize, reports.graduationRate = gradData[subgroup].graduationRate
        ON MATCH SET reports.cohortSize = gradData[subgroup].cohortSize, reports.graduationRate = gradData[subgroup].graduationRate
        """
        session.run(cypher_query, {"data": batch})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
with driver.session() as session:
    insert_school_graduation_data(session, transformed_school_graduation_data, batch_size=1000)

driver.close()

Inserting batches: 100%|██████████| 23/23 [00:41<00:00,  1.82s/it]


## Edfacts LEA Graduation

### Extract

In [ ]:
import pandas as pd

# Load the chronic absenteeism data
lea_graduation_path = './acgr-lea-sy2020-21-long.csv'
lea_graduation_df = pd.read_csv(lea_graduation_path, dtype=dtype_dict)

# Display the first few rows of each dataframe to understand their structure
print("Lea Support Preview:")
print(lea_graduation_df.head(), "\n")

Lea Support Preview:
  SCHOOL_YEAR    STNAM  FIPST    LEAID ST_LEAID             LEANM CATEGORY  \
0   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City      ALL   
1   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City      CWD   
2   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City      ECD   
3   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City      HOM   
4   2020-2021  ALABAMA      1  0100005   AL-101  Albertville City      LEP   

   COHORT   RATE DATE_CUR  
0     385     93  25MAY22  
1      20  60-79  25MAY22  
2     174  90-94  25MAY22  
3       2     PS  25MAY22  
4      57  80-89  25MAY22   



In [ ]:
unique_leas_in_graduation = set(lea_graduation_df['LEAID'].unique())
print(len(unique_leas_in_graduation))

11673


In [ ]:
from collections import defaultdict
import json
import re
from tqdm import tqdm

category_map = {
    "ALL": "Total",
    "CWD": "Students with disabilities (IDEA)",
    "ECD": "Economically disadvantaged",
    "FCS": "Foster Care Status",
    "HOM": "Homeless enrolled",
    "LEP": "English Learner",
    "MAM": "American Indian/Alaska Native",
    "MAS": "Asian/Pacific Islander",
    "MBL": "Black",
    "MHI": "Hispanic",
    "MIG": "Migrant",
    "MIL": "Military Connected Student Status",
    "MTR": "Two or More Races",
    "MWH": "White"
}

pct_proficient_symbols = {
    'GE': '≥',
    'LE': '≤',
    'GT': '>',
    'LT': '<',
    'PS': 'Data suppressed for privacy'
}

def convert_rate(value):
    # Check if the value is a string
    if isinstance(value, str):
        # Replace symbols with their corresponding abbreviations
        for abbreviation, symbol in pct_proficient_symbols.items():
            # Use regex to replace instances of the abbreviation followed by digits or in case of PS
            if abbreviation == 'PS':
                value = value.replace(abbreviation, symbol)
            else:
                value = re.sub(f'{abbreviation}(?=\d)', symbol, value)
    return value

def process_lea_data(lea_data):
    transformed_data = []

    first_row = lea_data.iloc[0]
    jurisdiction_name = first_row["STNAM"].title()
    lea_id, lea_name, year = first_row["LEAID"], first_row["LEANM"], first_row["SCHOOL_YEAR"][5:9]

    # Initialize dictionary to hold graduation data by category
    graduation_rates = {}

    for _, row in lea_data.iterrows():
        category_code = row["CATEGORY"]
        category_description = category_map.get(category_code, "Unknown Category")
        cohort_size = row["COHORT"]
        graduation_rate = convert_rate(row["RATE"])

        graduation_rates[category_description] = {
            "cohortSize": cohort_size,
            "graduationRate": graduation_rate
        }

    if graduation_rates:
        transformed_data.append({
            "jurisdiction": jurisdiction_name,
            "lea_id": lea_id,
            "lea_name": lea_name,
            "year": year[-4:],
            "graduationRates": graduation_rates
        })

    return transformed_data

def transform_lea_graduation_data(df):
    transformed_data = []

    # Directly iterate over each group and process
    for _, group in tqdm(df.groupby("LEAID")):
        transformed_data.extend(process_lea_data(group))

    return transformed_data

lea_graduation_df_subset = lea_graduation_df #.head(200)

print("Transforming lea_graduation data")
transformed_lea_graduation_data = transform_lea_graduation_data(lea_graduation_df_subset)

#with open("lea_graduation_data.json", "w+") as f:
#    json.dump(transformed_lea_graduation_data, f)

print(transformed_lea_graduation_data[:5])


Transforming lea_graduation data


100%|██████████| 11673/11673 [00:13<00:00, 851.23it/s] 

[{'jurisdiction': 'Alabama', 'lea_id': '0100005', 'lea_name': 'Albertville City', 'year': '2021', 'graduationRates': {'Total': {'cohortSize': 385, 'graduationRate': '93'}, 'Students with disabilities (IDEA)': {'cohortSize': 20, 'graduationRate': '60-79'}, 'Economically disadvantaged': {'cohortSize': 174, 'graduationRate': '90-94'}, 'Homeless enrolled': {'cohortSize': 2, 'graduationRate': 'Data suppressed for privacy'}, 'English Learner': {'cohortSize': 57, 'graduationRate': '80-89'}, 'American Indian/Alaska Native': {'cohortSize': 1, 'graduationRate': 'Data suppressed for privacy'}, 'Asian/Pacific Islander': {'cohortSize': 1, 'graduationRate': 'Data suppressed for privacy'}, 'Black': {'cohortSize': 17, 'graduationRate': '≥80'}, 'Hispanic': {'cohortSize': 170, 'graduationRate': '90-94'}, 'Two or More Races': {'cohortSize': 13, 'graduationRate': '≥50'}, 'White': {'cohortSize': 183, 'graduationRate': '90-94'}}}, {'jurisdiction': 'Alabama', 'lea_id': '0100006', 'lea_name': 'Marshall County

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

def insert_lea_graduation_data(session, data, batch_size=1000):
    # Calculate total number of batches for tqdm
    total_batches = len(data) // batch_size + (1 if len(data) % batch_size else 0)

    for i in tqdm(range(0, len(data), batch_size), total=total_batches, desc="Inserting batches"):
        batch = data[i:i + batch_size]
        cypher_query = """
        UNWIND $data AS leaData
        MERGE (lea:LEA {ncesID: leaData.lea_id})

        WITH leaData, lea
        MERGE (outcome:Outcome {name: 'High School Graduation'})

        WITH lea, outcome, leaData, leaData.graduationRates AS gradData
        UNWIND keys(gradData) AS subgroup
        MERGE (lea)-[reports:REPORTS {subgroup: subgroup, year: leaData.year}]->(outcome)
        ON CREATE SET reports.cohortSize = gradData[subgroup].cohortSize, reports.graduationRate = gradData[subgroup].graduationRate
        ON MATCH SET reports.cohortSize = gradData[subgroup].cohortSize, reports.graduationRate = gradData[subgroup].graduationRate
        """
        session.run(cypher_query, {"data": batch})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
with driver.session() as session:
    insert_lea_graduation_data(session, transformed_lea_graduation_data, batch_size=1000)

driver.close()

Inserting batches: 100%|██████████| 12/12 [00:33<00:00,  2.79s/it]


## CCD State Fiscal

### Extract

In [ ]:
import pandas as pd

# Load the state fiscal data
state_fiscal_path = './stfis21_1a.tsv'
state_fiscal_df = pd.read_csv((state_fiscal_path), delimiter='\t')


# Display the first few rows of each dataframe to understand their structure
print("State Fiscal Preview:")
print(state_fiscal_df.head(), "\n")

State Fiscal Preview:
   SURVYEAR  FIPS STABR      STNAME          R1A         R1B         R1C  \
0      2021     1    AL     Alabama           -2          -2  1457124498   
1      2021     2    AK      Alaska           -2          -2   292770219   
2      2021     4    AZ     Arizona   3666990661          -2          -2   
3      2021     5    AR    Arkansas   2095524529     3008644          -2   
4      2021     6    CA  California  30374365025  1488179463          -2   

         R1D      R1E        R1F  ...  IAR7  IAR8  IAE1  IAE2  IAE3  IAE4  \
0  877602957  8093700     443750  ...     R     R     R     R     R     R   
1  223920279   281844      51128  ...     R     R     R     R     R     R   
2   71549711  5348782   19954998  ...     R     R     R     R     R     R   
3   13761992  8668165     289567  ...     R     R     R     R     R     R   
4         -2   465950  458217348  ...     R     R     R     R     R     R   

   IAE5  IAE6  IAE7  IAE8  
0     R     R     R     R  
1 

In [ ]:
import pandas as pd

def check_valid(value):
    """ Helper function to check if the value should be included. """
    return not pd.isnull(value) and value not in {0, -1, -2, -9}

def format_value(value):
    """ Format the financial value with commas """
    return f"${value:,.0f}"

def process_category(row, mappings):
    """ Recursive function to process each category and its subcategories if any. """
    category_data = {}
    for description, column_key_or_subdict in mappings.items():
        if isinstance(column_key_or_subdict, dict):  # It's a subcategory with further details
            subcategory_data = process_category(row, column_key_or_subdict)
            if subcategory_data:  # Only add the subcategory if it's not empty
                category_data[description] = subcategory_data
        else:
            column_key = column_key_or_subdict
            if column_key in row and check_valid(row[column_key]):
                category_data[description] = format_value(row[column_key])
    return category_data

def transform_data(df, revenue_mappings, expenditure_mappings):
    transformed_data = []

    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        state_data = {
            'State': row['STNAME'], 'ADA': row['ADA'], 'perPupilExpenditures': row['PPE15'],
            'Membership': row['MEMBR20'], 'Revenues': {}, 'Expenditures': {},
            'COVID-19 Revenues': {}, 'COVID-19 Expenditures': {}
        }

        # Process each revenue and expenditure category
        for main_key, main_value in [
            ('Revenues', revenue_mappings), ('Expenditures', expenditure_mappings), ('COVID-19 Revenues', covid_revenue_mappings),('COVID-19 Expenditures', covid_expenditure_mappings)]:
            state_data[main_key] = process_category(row, main_value)

        transformed_data.append(state_data)

    return transformed_data

# Define mappings for Revenue subcategories
revenue_mappings = {
    'Local Revenues': {
        'Property Tax': 'R1A',
        'Nonproperty Tax': 'R1B',
        'Local Government Property Tax': 'R1C',
        'Local Government Nonproperty Tax': 'R1D',
        'Individual Tuition': 'R1E',
        'Tuition from LEAs': 'R1F',
        'Transportation Fees from Individuals': 'R1G',
        'Transportation Fees from LEAs': 'R1H',
        'Earnings on Investments': 'R1I',
        'Food Service': 'R1J',
        'Student Activities': 'R1K',
        'Other Revenues': 'R1L',
        'Textbook Revenues': 'R1M',
        'Summer School': 'R1N',
        'Subtotal': 'STR1'
    },
    'Intermediate Revenues': {
        'Intermediate Revenues': 'R2'
    },
    'State Revenues': {
        'State Revenues': 'R3'
    },
    'Federal Revenues': {
        'Direct Grants': 'R4A',
        'Through State': 'R4B',
        'Through Intermediate Agencies': 'R4C',
        'Other Sources': 'R4D',
        'Federal Revenues Subtotal': 'STR4'
    },
    'Other Sources': {
        'Other Sources': 'R5'
    },
    'Total Revenues': {
        'Total Revenues': 'TR'
    }
}

expenditure_mappings = {
    'Instructional Expenditures': {
        'Salaries': 'E11',
        'Teacher Salaries': {
            'Regular Programs': 'E11A',
            'Special Education Programs': 'E11B',
            'Vocational Education Programs': 'E11C',
            'Other Education Programs': 'E11D'
        },
        'Employee Benefits': 'E12',
        'Purchased Services': 'E13',
        'Tuition to Private and Out-of-State Schools': 'E14',
        'Tuition to Other LEAs In-State': 'E15',
        'Supplies': 'E16',
        'Property': 'E17',
        'Other': 'E18',
        'Subtotal': 'STE1',
    },
    'Textbooks': 'E2',
    'Support Expenditures': {
        'Salaries': {
            'Student Support Services': 'E212',
            'Instructional Staff Support': 'E213',
            'General Administration': 'E214',
            'School Administration': 'E215',
            'Operation & Maintenance': 'E216',
            'Pupil Transportation': 'E217',
            'Other Services': 'E218',
            'Subtotal': 'TE21'
        },
        'Employee Benefits': {
            'Student Support Services': 'E222',
            'Instructional Staff Support': 'E223',
            'General Administration': 'E224',
            'School Administration': 'E225',
            'Operation & Maintenance': 'E226',
            'Pupil Transportation': 'E227',
            'Other Services': 'E228',
            'Subtotal': 'TE22'
        },
        'Purchased Services': {
            'Student Support Services': 'E232',
            'Instructional Staff Support': 'E233',
            'General Administration': 'E234',
            'School Administration': 'E235',
            'Operation & Maintenance': 'E236',
            'Pupil Transportation': 'E237',
            'Other Services': 'E238',
            'Subtotal': 'TE23'
        },
        'Supplies': {
            'Student Support Services': 'E242',
            'Instructional Staff Support': 'E243',
            'General Administration': 'E244',
            'School Administration': 'E245',
            'Operation & Maintenance': 'E246',
            'Pupil Transportation': 'E247',
            'Other Services': 'E248',
            'Subtotal': 'TE24'
        },
        'Property': {
            'Student Support Services': 'E252',
            'Instructional Staff Support': 'E253',
            'General Administration': 'E254',
            'School Administration': 'E255',
            'Operation & Maintenance': 'E256',
            'Pupil Transportation': 'E257',
            'Other Services': 'E258',
            'Subtotal': 'TE25'
        },
        'Other': {
            'Student Support Services': 'E262',
            'Instructional Staff Support': 'E263',
            'General Administration': 'E264',
            'School Administration': 'E265',
            'Operation & Maintenance': 'E266',
            'Pupil Transportation': 'E267',
            'Other Services': 'E268',
            'Subtotal': 'TE26'
        },
        'Total Support Services': 'STE2T'
    },
    'Subtotals': {
        'Instructional Staff Support': 'STE23',
        'General Administration': 'STE24',
        'School Administration': 'STE25',
        'Operation & Maintenance': 'STE26',
        'Pupil Transportation': 'STE27',
        'Other Services': 'STE28',
    },
    'Noninstructional Services': {
        'Food Services': {
            'Salaries': 'E3A11',
            'Employee Benefits': 'E3A12',
            'Purchased Services': 'E3A13',
            'Supplies': 'E3A14',
            'Property': 'E3A2',
            'Other': 'E3A16',
            'Subtotal': 'E3A1'
        },
        'Enterprise': {
            'Salaries': 'E3B11',
            'Employee Benefits': 'E3B12',
            'Purchased Services': 'E3B13',
            'Supplies': 'E3B14',
            'Property': 'E3B2',
            'Other': 'E3B16',
            'Subtotal': 'E3B1'
        },
        'Total': 'STE3'
    },
    'Direct Program Support': {
        'Textbooks': {
            'General': 'E4A1',
            'Property': 'E4A2'
        },
        'Transportation': {
            'General': 'E4B1',
            'Property': 'E4B2'
        },
        'Employee Benefits': {
            'General': 'E4C1',
            'Property': 'E4C2'
        },
        'Private School Student': 'E4D',
        'Other': {
            'General': 'E4E1',
            'Property': 'E4E2'
        },
        'Subtotal': 'STE4'
    },
    'Facilities Acquisition': {
        'Nonproperty': 'E61',
        'Property': {
            'Land and Buildings': 'E62',
            'Equipment': 'E63'
        },
        'Total': 'STE6'
    },
    'Other Uses': {
        'Debt Service': {
            'Interest': 'E7A1',
            'Redemption': 'E7A2',
            'Subtotal': 'STE7'
        }
    },
    'Community Services': {
        'Nonproperty': 'E81',
        'Property': 'E82'
    },
    'Direct Cost Programs': {
        'Nonpublic School': 'E9A',
        'Adult Education': 'E9B',
        'Community College': 'E9C',
        'Other': 'E9D',
        'Property': 'E91',
        'Subtotal': 'STE9'
    },
    'Property Total': 'TE10',
    'Total Expenditures': 'TE11',
    'Exclusions': {
        'PL 114 95 Title I': 'X12C',
        'PL 114 95 Title I Carryover': 'X12D',
        'Title V, Part A': 'X12E',
        'Title V, Part A Carryover': 'X12F',
        'Total Exclusions': 'TX12'
    },
    'Net Current Expenditures': 'NCE13',
    'Current Expenditures Paid from State and Local Funds': 'CE1',
    'Current Expenditures Paid from Federal Funds': 'CE2'
}

covid_revenue_mappings = {
    'CARES Act Revenues': {
        'ESSER I Fund': 'AR1',
        'GEER Fund': 'AR2',
        'ESF-RWP': 'AR3',
        'Coronavirus Relief Fund': 'AR6',
        'ESF Outlying Areas-SEA': 'AR7',
        'ESF Outlying Areas-Gov': 'AR8'
    },
    'CRRSA Revenues': {
        'ESSER II Fund': 'AR1A',
        'GEER II Fund': 'AR2A'
    },
    'ARP Act Revenues': {
        'ARP ESSER Fund': 'AR1B',
        'Coronavirus SLFR Funds': 'AR6A'
    }
}

covid_expenditure_mappings = {
    'Current Expenditures from COVID-19 Federal Assistance': 'AE1',
    'Instructional Expenditures from COVID-19 Federal Assistance': 'AE2',
    'Support Services Expenditures from COVID-19 Federal Assistance': 'AE3',
    'Capital Outlay Expenditures from COVID-19 Federal Assistance': 'AE4',
    'Technology-Related Expenditures from COVID-19 Federal Assistance': {
        'Supplies & Purchased Services': 'AE5',
        'Equipment': 'AE6'
    },
    'Support Services, Operations & Maintenance Expenditures from COVID-19 Federal Assistance': 'AE7',
    'Food Services Operations Expenditures from COVID-19 Federal Assistance': 'AE8'
}

# Transform the data
transformed_state_fiscal_data = transform_data(state_fiscal_df, revenue_mappings, expenditure_mappings)

# Print the first few entries to verify
for entry in transformed_state_fiscal_data[:3]:
    print(entry)

{'State': 'Alabama', 'ADA': 695526, 'perPupilExpenditures': 10654, 'Membership': 734559, 'Revenues': {'Local Revenues': {'Local Government Property Tax': '$1,457,124,498', 'Local Government Nonproperty Tax': '$877,602,957', 'Individual Tuition': '$8,093,700', 'Tuition from LEAs': '$443,750', 'Transportation Fees from LEAs': '$4,551', 'Earnings on Investments': '$33,222,970', 'Food Service': '$22,267,760', 'Student Activities': '$173,861,210', 'Other Revenues': '$320,739,344', 'Textbook Revenues': '$892,641', 'Subtotal': '$2,893,805,080'}, 'Intermediate Revenues': {'Intermediate Revenues': '$42,921,041'}, 'State Revenues': {'State Revenues': '$5,187,635,561'}, 'Federal Revenues': {'Direct Grants': '$13,677,835', 'Through State': '$1,475,899,987', 'Through Intermediate Agencies': '$372,888', 'Other Sources': '$20,422,208', 'Federal Revenues Subtotal': '$1,510,372,918'}, 'Other Sources': {'Other Sources': '$767,550,925'}, 'Total Revenues': {'Total Revenues': '$9,634,734,600'}}, 'Expenditu

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm
import json

def insert_data(session, data, batch_size=1000):
    # Calculate total number of batches for tqdm
    total_batches = len(data) // batch_size + (1 if len(data) % batch_size else 0)

    for i in tqdm(range(0, len(data), batch_size), total=total_batches, desc="Inserting batches"):
        batch = data[i:i + batch_size]
        # Serialize complex data structures into JSON strings
        for entry in batch:
            entry['Revenues'] = json.dumps(entry['Revenues'])
            entry['Expenditures'] = json.dumps(entry['Expenditures'])
            entry['COVID-19 Revenues'] = json.dumps(entry['COVID-19 Revenues'])
            entry['COVID-19 Expenditures'] = json.dumps(entry['COVID-19 Expenditures'])

        cypher_query = """
        UNWIND $data AS fiscalData
        MERGE (juris:Jurisdiction {name: fiscalData.State})
        SET juris.year = 2021,
            juris.membership = fiscalData.Membership,
            juris.revenues = fiscalData.Revenues,
            juris.expenditures = fiscalData.Expenditures,
            juris.covidRevenues = fiscalData['COVID-19 Revenues'],
            juris.covidExpenditures = fiscalData['COVID-19 Expenditures'],
            juris.perPupilExpenditures = fiscalData['perPupilExpenditures']

        MERGE (ada:Outcome {name: 'Average Daily Attendance'})
        MERGE (juris)-[r:REPORTS]->(ada)
        SET r.percentage = round(100 * toFloat(fiscalData.ADA) / toFloat(fiscalData.Membership) * 100) / 100,            r.year = 2021
        """
        session.run(cypher_query, {"data": batch})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
with driver.session() as session:
    insert_data(session, transformed_state_fiscal_data, batch_size=1000)

driver.close()

Inserting batches: 100%|██████████| 1/1 [00:02<00:00,  2.11s/it]


## CCD LEA Fiscal

### Extract

In [ ]:
import pandas as pd

# Load the LEA fiscal data
lea_fiscal_path = './sdf21_1a.csv'

# Try reading the tab-delimited file with a common encoding
try:
    lea_fiscal_df = pd.read_csv(lea_fiscal_path, sep='\t', encoding='latin1', dtype= dtype_dict)
except UnicodeDecodeError:
    try:
        lea_fiscal_df = pd.read_csv(lea_fiscal_path, sep='\t', encoding='cp1252', dtype= dtype_dict)
    except UnicodeDecodeError:
        lea_fiscal_df = pd.read_csv(lea_fiscal_path, sep='\t', encoding='utf-16', dtype= dtype_dict)

# Check the DataFrame
print(lea_fiscal_df.head())

     LEAID        CENSUSID  FIPST  CONUM  CSA   CBSA                    NAME  \
0  0100002               N      1   1101  388  33860  Alabama Youth Services   
1  0100005  01504840100000      1   1095    N  10700        Albertville City   
2  0100006  01504800100000      1   1095    N  10700         Marshall County   
3  0100007  01503740100000      1   1073  142  13820             Hoover City   
4  0100008  01504530100000      1   1089  290  26620            Madison City   

    STNAME STABBR SCHLEV  ... FL_AR6  FL_AR6A  FL_AE1  FL_AE2 FL_AE3 FL_AE4  \
0  Alabama     AL      N  ...      M        M       M       M      M      M   
1  Alabama     AL     03  ...      R        M       R       R      R      R   
2  Alabama     AL     03  ...      R        M       R       R      R      R   
3  Alabama     AL     03  ...      R        M       R       R      R      R   
4  Alabama     AL     03  ...      R        M       R       R      R      R   

   FL_AE5  FL_AE6  FL_AE7  FL_AE8  
0       

In [ ]:
import pandas as pd

def check_valid(value):
    """ Helper function to check if the value should be included. """
    return not pd.isnull(value) and value not in {0, -1, -2, -9}

def format_value(value):
    """ Format the financial value with commas """
    return f"${value:,.0f}"

def calculate_per_pupil_expenditures(total_exp, membership):
    try:
        if membership > 0:
            return round(total_exp / membership)
        else:
            return None  # Return None or some other indicator for invalid or missing data
    except TypeError:  # Handles cases where inputs are not numbers or are None
        return None

def process_category(row, mappings):
    """ Recursive function to process each category and its subcategories if any. """
    category_data = {}
    for description, column_key_or_subdict in mappings.items():
        if isinstance(column_key_or_subdict, dict):  # It's a subcategory with further details
            subcategory_data = process_category(row, column_key_or_subdict)
            if subcategory_data:  # Only add the subcategory if it's not empty
                category_data[description] = subcategory_data
        else:
            column_key = column_key_or_subdict
            if column_key in row and check_valid(row[column_key]):
                category_data[description] = format_value(row[column_key])
    return category_data

def transform_data(df):
    transformed_data = []

    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        if row['MEMBERSCH'] > 0:  # Check if Membership is positive
            lea_data = {
                'jurisdiction': row['STNAME'],
                'lea_id': row['LEAID'],
                'lea_name': row['NAME'],
                'perPupilExpenditures': calculate_per_pupil_expenditures(row['TOTALEXP'], row['MEMBERSCH']),
                'Membership': row['MEMBERSCH'],
                'Revenues': {},
                'Expenditures': {},
                'COVID-19 Revenues': {},
                'COVID-19 Expenditures': {}
            }

            # Process each revenue and expenditure category
            for main_key, main_value in [
                ('Revenues', revenue_mappings),
                ('Expenditures', expenditure_mappings),
                ('COVID-19 Revenues', covid_revenue_mappings),
                ('COVID-19 Expenditures', covid_expenditure_mappings)
            ]:
                lea_data[main_key] = process_category(row, main_value)

            transformed_data.append(lea_data)

    return transformed_data

# Define mappings for Revenue subcategories
revenue_mappings = {
    'Total Revenue': 'TOTALREV',
    'Federal Revenue': {
        'Total Federal Revenue': 'TFEDREV',
        'Through State': {
            'Title I': 'C14',
            'IDEA': 'C15',
            'Vocational and Tech Education': 'C19',
            'Supporting Effective Instruction': 'C22',
            'Student Support and Academic Enrichment': 'C23',
            '21st Century Learning Centers': 'C26',
            'Rural and Low-Income School Program': 'C27',
            'Bilingual Education': 'B11',
            'Other': 'C20',
            'Child Nutrition Act': 'C25',
            'Nonspecified': 'C36'
        },
        'Direct': {
            'Impact Aid': 'B10',
            'Indian Education': 'B12',
            'Small, Rural School Achievement Program': 'B14',
            'Other': 'B13'
        }
    },
    'State Revenue': {
        'Total State Revenue': 'TSTREV',
        'General Formula Assistance': 'C01',
        'Staff Improvement Programs': 'C04',
        'Special Education Programs': 'C05',
        'Compensatory and Basic Skills Programs': 'C06',
        'Bilingual Education Programs': 'C07',
        'Gifted and Talented Programs': 'C08',
        'Vocational Education Programs': 'C09',
        'School Lunch Programs': 'C10',
        'Capital Outlay and Debt Services Programs': 'C11',
        'Transportation Programs': 'C12',
        'Other Programs': 'C13',
        'Nonspecified': 'C35',
        'On Behalf Employee Benefits': 'C38',
        'On Behalf Non-Employee Benefits': 'C39'
    },
    'Local Revenue': {
        'Total Local Revenue': 'TLOCREV',
        'Parent Government Contributions': 'T02',
        'Property Taxes': 'T06',
        'General Sales Taxes': 'T09',
        'Public Utility Taxes': 'T15',
        'Individual and Corporate Income Taxes': 'T40',
        'All Other Taxes': 'T99',
        'From Other School Systems': 'D11',
        'From Cities and Counties': 'D23',
        'Tuition Fees from Pupils and Parents': 'A07',
        'Transportation Fees from Pupils and Parents': 'A08',
        'School Lunch': 'A09',
        'Textbook Sales and Rentals': 'A11',
        'District Activity Receipts': 'A13',
        'Students Fees, Nonspecified': 'A15',
        'Other Sales and Services': 'A20',
        'Rents and Royalties': 'A40',
        'Sale of Property': 'U11',
        'Interest Earnings': 'U22',
        'Fines and Forfeits': 'U30',
        'Private Contributions': 'U50',
        'Miscellaneous': 'U97',
        'NCES Local Revenue, Census Bureau State Revenue': 'C24'
    }
}

expenditure_mappings = {
    'Total Expenditures': 'TOTALEXP',
    'Total Current Expenditures for Elementary/Secondary Education': {
        'Total Instructional Expenditures': 'TCURINST',
        'Total Support Services Expenditures': 'TCURSSVC',
        'Total Other Elementary/Secondary Current Expenditures': 'TCUROTH'
    },
    'Non-Elementary/Secondary Expenditures': {
        'Community Services': 'V70',
        'Adult Education': 'V75',
        'Other Non-Elementary/Secondary': 'V80'
    },
    'Capital Outlay': {
        'Construction': 'F12',
        'Land and Existing Structures': 'G15',
        'Instructional Equipment': 'K09',
        'Other Equipment': 'K10',
        'Nonspecified Equipment': 'K11'
    },
    'Miscellaneous Expenditures': {
        'Payments to State Governments': 'L12',
        'Payments to Local Governments': 'M12',
        'Payments to Other School Systems': 'Q11',
        'Interest on Debt': 'I86'
    },
    'Salaries and Employee Benefits': {
        'Total Salaries': 'Z32',
        'Instruction': 'Z33',
        'Regular Education Programs': 'Z35',
        'Special Education Programs': 'Z36',
        'Vocational Education Programs': 'Z37',
        'Other Education Programs': 'Z38',
        'Support Services': {
            'Pupils': 'V11',
            'Instructional Staff': 'V13',
            'General Administration': 'V15',
            'School Administration': 'V17',
            'Operation and Maintenance of Plant': 'V21',
            'Student Transportation': 'V23',
            'Business/Central/Other': 'V37',
            'Food Services': 'V29'
        },
        'Total Employee Benefits': 'Z34',
        'Benefits for Instruction': 'V10',
        'Support Services Benefits': {
            'Pupils': 'V12',
            'Instructional Staff': 'V14',
            'General Administration': 'V16',
            'School Administration': 'V18',
            'Operation and Maintenance of Plant': 'V22',
            'Student Transportation': 'V24',
            'Business/Central/Other': 'V38',
            'Food Services': 'V30',
            'Enterprise Operations': 'V32'
        }
    }
}

covid_revenue_mappings = {
    'CARES Act': 'AR1',
    'CRRSA Act ESSER II Fund': 'AR1A',
    'ARP Act ESSER Fund': 'AR1B',
    'GEER I Fund': 'AR2',
    'GEER II Fund': 'AR2A',
    'ESF-RWP Grant': 'AR3',
    'Coronavirus Relief Fund': 'AR6',
    'SLFR Funds': 'AR6A'
}

covid_expenditure_mappings = {
    'Current': 'AE1',
    'Instructional': 'AE2',
    'Support Services': 'AE3',
    'Capital Outlay': 'AE4',
    'Technology Supplies and Services': 'AE5',
    'Technology Equipment': 'AE6',
    'Operations and Maintenance': 'AE7',
    'Food Services': 'AE8'
}

# Transform the data
lea_fiscal_df_sub = lea_fiscal_df #.head(200)
transformed_lea_fiscal_data = transform_data(lea_fiscal_df_sub)

# Print the first few entries to verify
for entry in transformed_lea_fiscal_data[:3]:
    print(entry)

{'jurisdiction': 'Alabama', 'lea_id': '0100005', 'lea_name': 'Albertville City', 'perPupilExpenditures': 10135, 'Membership': 5842, 'Revenues': {'Total Revenue': '$74,420,000', 'Federal Revenue': {'Total Federal Revenue': '$10,327,000', 'Through State': {'Title I': '$2,502,000', 'IDEA': '$1,255,000', 'Vocational and Tech Education': '$83,000', 'Supporting Effective Instruction': '$28,000', 'Bilingual Education': '$177,000', 'Other': '$2,374,000', 'Child Nutrition Act': '$3,904,000'}, 'Direct': {'Other': '$4,000'}}, 'State Revenue': {'Total State Revenue': '$47,611,000', 'General Formula Assistance': '$32,259,000', 'Staff Improvement Programs': '$73,000', 'Compensatory and Basic Skills Programs': '$465,000', 'Bilingual Education Programs': '$691,000', 'Capital Outlay and Debt Services Programs': '$1,640,000', 'Transportation Programs': '$2,466,000', 'Other Programs': '$10,017,000'}, 'Local Revenue': {'Total Local Revenue': '$16,482,000', 'Property Taxes': '$6,318,000', 'All Other Taxes'

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm
import json

def insert_data(session, data, batch_size=1000):
    # Calculate total number of batches for tqdm
    total_batches = len(data) // batch_size + (1 if len(data) % batch_size else 0)

    for i in tqdm(range(0, len(data), batch_size), total=total_batches, desc="Inserting batches"):
        batch = data[i:i + batch_size]
        # Serialize complex data structures into JSON strings
        for entry in batch:
            entry['Revenues'] = json.dumps(entry['Revenues'])
            entry['Expenditures'] = json.dumps(entry['Expenditures'])
            entry['COVID-19 Revenues'] = json.dumps(entry['COVID-19 Revenues'])
            entry['COVID-19 Expenditures'] = json.dumps(entry['COVID-19 Expenditures'])

        cypher_query = """
        UNWIND $data AS fiscalData
        MERGE (lea:LEA {ncesID: fiscalData.lea_id})
        SET lea.year = 2021,
            lea.membership = fiscalData.Membership,
            lea.revenues = fiscalData.Revenues,
            lea.expenditures = fiscalData.Expenditures,
            lea.covidRevenues = fiscalData['COVID-19 Revenues'],
            lea.covidExpenditures = fiscalData['COVID-19 Expenditures'],
            lea.perPupilExpenditures = fiscalData['perPupilExpenditures']
        """
        session.run(cypher_query, {"data": batch})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
with driver.session() as session:
    insert_data(session, transformed_lea_fiscal_data, batch_size=1000)

driver.close()

Inserting batches: 100%|██████████| 18/18 [00:09<00:00,  1.81it/s]


## CCD School Geography

### Extract

In [ ]:
import pandas as pd

column_names = [
    'NCESSCH', 'LEAID', 'NAME', 'OPSTFIPS', 'STREET', 'CITY', 'STATE', 'ZIP',
    'STFIP', 'CNTY', 'NMCNTY', 'LOCALE', 'LAT', 'LON', 'CBSA', 'NMCBSA',
    'CBSATYPE', 'CSA', 'NMCSA', 'NECTA', 'NMNECTA', 'CD', 'SLDL', 'SLDU', 'SCHOOLYEAR'
]

# Load the LEA fiscal data
school_geo_path = './EDGE_GEOCODE_PUBLICSCH_2021.TXT'

school_geo_df = pd.read_csv(school_geo_path, sep='|', encoding='latin1',  names=column_names, header=None, dtype= dtype_dict, index_col=False)

pd.set_option('display.max_columns', None)  # or specify a number of columns to display

# Check the DataFrame
print(school_geo_df.head())

        NCESSCH    LEAID                               NAME  OPSTFIPS  \
0  010000500870  0100005          Albertville Middle School         1   
1  010000500871  0100005            Albertville High School         1   
2  010000500879  0100005    Albertville Intermediate School         1   
3  010000500889  0100005      Albertville Elementary School         1   
4  010000501616  0100005  Albertville Kindergarten and PreK         1   

                STREET         CITY STATE    ZIP  STFIP  CNTY  \
0    600 E Alabama Ave  Albertville    AL  35950      1  1095   
1     402 E McCord Ave  Albertville    AL  35950      1  1095   
2   901 W McKinney Ave  Albertville    AL  35950      1  1095   
3   145 West End Drive  Albertville    AL  35950      1  1095   
4  257 Country Club Rd  Albertville    AL  35951      1  1095   

            NMCNTY  LOCALE      LAT        LON   CBSA           NMCBSA  \
0  Marshall County      32  34.2602 -86.206200  10700  Albertville, AL   
1  Marshall County    

<ipython-input-147-669a7eb8c9ef>:12: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  school_geo_df = pd.read_csv(school_geo_path, sep='|', encoding='latin1',  names=column_names, header=None, dtype= dtype_dict, index_col=False)


In [ ]:
import pandas as pd
from tqdm import tqdm

# Locale mapping based on the provided codes
locale_map = {
    '11': "City, Large",
    '12': "City, Midsize",
    '13': "City, Small",
    '21': "Suburban, Large",
    '22': "Suburban, Midsize",
    '23': "Suburban, Small",
    '31': "Town, Fringe",
    '32': "Town, Distant",
    '33': "Town, Remote",
    '41': "Rural, Fringe",
    '42': "Rural, Distant",
    '43': "Rural, Remote"
}

def transform_school_data(df):
    transformed_data = []
    # Iterate over the rows of the DataFrame
    for _, row in tqdm(df.iterrows(), total=df.shape[0]):
        # Assemble address
        address = f"{row['STREET']}, {row['CITY']}, {row['STATE']}, {row['ZIP']}"

        # Locale description
        locale_description = locale_map.get(str(row['LOCALE']), "Unknown Locale")

        # Coordinates
        coordinates = f"{row['LAT']}, {row['LON']}"

        # Year from the school year
        year = row['SCHOOLYEAR'][-4:]

        # Collect transformed data
        school_info = {
            'ncesID': row['NCESSCH'],
            'name': row['NAME'],
            'address': address,
            'locale': locale_description,
            'coordinates': coordinates,
            'year': year
        }
        transformed_data.append(school_info)

    return transformed_data

school_geo_df_sub = school_geo_df #.head(10)

transformed_school_geo_data = transform_school_data(school_geo_df_sub)
print(transformed_school_geo_data[:5])  # Print first 5 entries to verify

100%|██████████| 101662/101662 [00:26<00:00, 3787.36it/s]

[{'ncesID': '010000500870', 'name': 'Albertville Middle School', 'address': '600 E Alabama Ave, Albertville, AL, 35950', 'locale': 'Town, Distant', 'coordinates': '34.2602, -86.2062', 'year': '2021'}, {'ncesID': '010000500871', 'name': 'Albertville High School', 'address': '402 E McCord Ave, Albertville, AL, 35950', 'locale': 'Town, Distant', 'coordinates': '34.2622, -86.2049', 'year': '2021'}, {'ncesID': '010000500879', 'name': 'Albertville Intermediate School', 'address': '901 W McKinney Ave, Albertville, AL, 35950', 'locale': 'Town, Distant', 'coordinates': '34.2733, -86.2201', 'year': '2021'}, {'ncesID': '010000500889', 'name': 'Albertville Elementary School', 'address': '145 West End Drive, Albertville, AL, 35950', 'locale': 'Town, Distant', 'coordinates': '34.2527, -86.221806', 'year': '2021'}, {'ncesID': '010000501616', 'name': 'Albertville Kindergarten and PreK', 'address': '257 Country Club Rd, Albertville, AL, 35951', 'locale': 'Town, Distant', 'coordinates': '34.2898, -86.19

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm
import json

def insert_data(session, data, batch_size=1000):
    # Calculate total number of batches for tqdm
    total_batches = len(data) // batch_size + (1 if len(data) % batch_size else 0)

    for i in tqdm(range(0, len(data), batch_size), total=total_batches, desc="Inserting batches"):
        batch = data[i:i + batch_size]
        cypher_query = """
        UNWIND $data AS geoData
        MERGE (i:Institution {ncesID: geoData.ncesID})
        SET
            i.address = geoData.address,
            i.locale = geoData.locale,
            i.coordinates = geoData.coordinates
        """
        session.run(cypher_query, {"data": batch})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
with driver.session() as session:
    insert_data(session, transformed_school_geo_data, batch_size=1000)

driver.close()

Inserting batches: 100%|██████████| 102/102 [00:22<00:00,  4.51it/s]


## CCD School Directory

### Extract

In [ ]:
import pandas as pd

# Load the data
school_directory_path = './ccd_sch_029_2021_w_1a_080621.csv'

school_directory_df = pd.read_csv(school_directory_path, encoding='latin1', dtype= dtype_dict)

# Check the DataFrame
print(school_directory_df.head())

<ipython-input-160-d275c2cc713e>:6: DtypeWarning: Columns (14,15,21,22,39,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  school_characteristics_df = pd.read_csv(school_characteristics_path, encoding='latin1', dtype= dtype_dict)


  SCHOOL_YEAR  FIPST STATENAME  ST                           SCH_NAME  \
0   2020-2021      1   ALABAMA  AL          Albertville Middle School   
1   2020-2021      1   ALABAMA  AL            Albertville High School   
2   2020-2021      1   ALABAMA  AL    Albertville Intermediate School   
3   2020-2021      1   ALABAMA  AL      Albertville Elementary School   
4   2020-2021      1   ALABAMA  AL  Albertville Kindergarten and PreK   

           LEA_NAME  STATE_AGENCY_NO  UNION ST_LEAID    LEAID     ST_SCHID  \
0  Albertville City                1    NaN   AL-101  0100005  AL-101-0010   
1  Albertville City                1    NaN   AL-101  0100005  AL-101-0020   
2  Albertville City                1    NaN   AL-101  0100005  AL-101-0110   
3  Albertville City                1    NaN   AL-101  0100005  AL-101-0200   
4  Albertville City                1    NaN   AL-101  0100005  AL-101-0035   

        NCESSCH   SCHID             MSTREET1 MSTREET2 MSTREET3        MCITY  \
0  0100005008

In [ ]:
import pandas as pd
from tqdm import tqdm

def transform_school_data(df):
    transformed_data = []
    # Iterate over the rows of the DataFrame
    for _, row in tqdm(df.iterrows(), total=df.shape[0]):
        # Year from the school year
        year = row['SCHOOL_YEAR'][-4:]

        # Format grades available if present
        available_grades = f"{row['GSLO']} - {row['GSHI']}" if pd.notnull(row['GSLO']) and pd.notnull(row['GSHI']) else "Grades Not Available"

        # Charter school information
        charter = "Yes" if row['CHARTER_TEXT'] == "Yes" else None
        charter_authorizer = row['CHARTAUTHN1'] if pd.notnull(row['CHARTAUTHN1']) else None

        # Collect transformed data
        school_info = {
            'ncesID': row['NCESSCH'],
            'name': row['SCH_NAME'],
            'type': row['SCH_TYPE_TEXT'],
            'year': year,
        }

        # Optional fields
        if pd.notnull(row['WEBSITE']):
            school_info['website'] = row['WEBSITE']

        if pd.notnull(row['GSLO']) and pd.notnull(row['GSHI']):
            school_info['availableGrades'] = f"{row['GSLO']} - {row['GSHI']}"

        if row['CHARTER_TEXT'] == "Yes":
            school_info['charter'] = "Yes"
            if pd.notnull(row['CHARTAUTHN1']):
                school_info['charterAuthorizer'] = row['CHARTAUTHN1']

        transformed_data.append(school_info)


    return transformed_data

school_directory_df_sub = school_directory_df #.head(10)

transformed_school_directory_data = transform_school_data(school_directory_df_sub)
print(transformed_school_directory_data[:5])  # Print first 5 entries to verify

100%|██████████| 101662/101662 [00:20<00:00, 5001.64it/s]

[{'ncesID': '010000500870', 'name': 'Albertville Middle School', 'type': 'Regular School', 'year': '2021', 'website': 'http://www.albertk12.org', 'availableGrades': '07 - 08'}, {'ncesID': '010000500871', 'name': 'Albertville High School', 'type': 'Regular School', 'year': '2021', 'website': 'http://www.albertk12.org', 'availableGrades': '09 - 12'}, {'ncesID': '010000500879', 'name': 'Albertville Intermediate School', 'type': 'Regular School', 'year': '2021', 'website': 'http://www.albertk12.org', 'availableGrades': '05 - 06'}, {'ncesID': '010000500889', 'name': 'Albertville Elementary School', 'type': 'Regular School', 'year': '2021', 'website': 'http://www.albertk12.org', 'availableGrades': '03 - 04'}, {'ncesID': '010000501616', 'name': 'Albertville Kindergarten and PreK', 'type': 'Regular School', 'year': '2021', 'website': 'http://www.albertk12.org', 'availableGrades': 'PK - KG'}]


### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

def insert_data(session, data, batch_size=1000):
    # Calculate total number of batches for tqdm
    total_batches = len(data) // batch_size + (1 if len(data) % batch_size else 0)

    for i in tqdm(range(0, len(data), batch_size), total=total_batches, desc="Inserting batches"):
        batch = data[i:i + batch_size]
        cypher_query = """
        UNWIND $data AS schoolData
        MERGE (i:Institution {ncesID: schoolData.ncesID})
        SET
            i.website = schoolData.website,
            i.type = schoolData.type,
            i.availableGrades = schoolData.availableGrades,
            i.charter = schoolData.charter,
            i.charterAuthorizer = schoolData.charterAuthorizer
        """
        session.run(cypher_query, {"data": batch})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
with driver.session() as session:
    insert_data(session, transformed_school_directory_data, batch_size=1000)

driver.close()

Inserting batches: 100%|██████████| 102/102 [00:21<00:00,  4.73it/s]


## CCD School Characteristics

### Extract

In [ ]:
import pandas as pd

# Load the data
school_characteristics_path = './ccd_sch_129_2021_w_1a_080621.csv'

school_characteristics_df = pd.read_csv(school_characteristics_path, encoding='latin1', dtype= dtype_dict)

# Check the DataFrame
print(school_characteristics_df.head())

  SCHOOL_YEAR  FIPST STATENAME  ST                           SCH_NAME  \
0   2020-2021      1   ALABAMA  AL          Albertville Middle School   
1   2020-2021      1   ALABAMA  AL            Albertville High School   
2   2020-2021      1   ALABAMA  AL    Albertville Intermediate School   
3   2020-2021      1   ALABAMA  AL      Albertville Elementary School   
4   2020-2021      1   ALABAMA  AL  Albertville Kindergarten and PreK   

   STATE_AGENCY_NO UNION ST_LEAID    LEAID     ST_SCHID       NCESSCH   SCHID  \
0                1   NaN   AL-101  0100005  AL-101-0010  010000500870  100870   
1                1   NaN   AL-101  0100005  AL-101-0020  010000500871  100871   
2                1   NaN   AL-101  0100005  AL-101-0110  010000500879  100879   
3                1   NaN   AL-101  0100005  AL-101-0200  010000500889  100889   
4                1   NaN   AL-101  0100005  AL-101-0035  010000501616  101616   

  SHARED_TIME TITLEI_STATUS  \
0          No  SWELIGTGPROG   
1          N

In [ ]:
import pandas as pd
from tqdm import tqdm

def transform_school_data(df):
    transformed_data = []
    # Iterate over the rows of the DataFrame
    for _, row in tqdm(df.iterrows(), total=df.shape[0]):
        # Year from the school year
        year = row['SCHOOL_YEAR'][-4:]

        # Collect transformed data
        school_info = {
            'ncesID': row['NCESSCH'],
            'name': row['SCH_NAME'],
            'year': year,
        }

        # Optional fields
        if pd.notnull(row['UNION']):
            school_info['union'] = row['UNION']

        if (row['TITLEI_STATUS']) not in['MISSING', 'Not reported', 'NOTTITLE1ELIG']:
            school_info['titleI'] = row['TITLEI_STATUS_TEXT']

        if row['MAGNET_TEXT'] == "Yes":
            school_info['magnet'] = "Yes"

        if row['NSLP_STATUS'] not in [ 'MISSING', 'Not reported', 'NSLPNO']:
            school_info['nationalSchoolLunchProgram'] = row['NSLP_STATUS_TEXT']

        if row['SHARED_TIME'] == "Yes":
            school_info['sharedTime'] = "Yes"

        if row['VIRTUAL'] not in [ 'MISSING', 'Not reported', 'NOTVIRTUAL']:
            school_info['virtual'] = row['VIRTUAL_TEXT']

        transformed_data.append(school_info)


    return transformed_data

school_characteristics_df_sub = school_characteristics_df #.head(1000)

transformed_school_characteristics_data = transform_school_data(school_characteristics_df_sub)
print(transformed_school_characteristics_data[:5])  # Print first 5 entries to verify

100%|██████████| 99763/99763 [00:08<00:00, 11137.24it/s]

[{'ncesID': '010000500870', 'name': 'Albertville Middle School', 'year': '2021', 'titleI': 'Title I schoolwide eligible-Title I targeted assistance program', 'nationalSchoolLunchProgram': 'Yes participating without using any Provision or the CEO'}, {'ncesID': '010000500871', 'name': 'Albertville High School', 'year': '2021', 'titleI': 'Title I schoolwide eligible-Title I targeted assistance program', 'nationalSchoolLunchProgram': 'Yes participating without using any Provision or the CEO'}, {'ncesID': '010000500879', 'name': 'Albertville Intermediate School', 'year': '2021', 'titleI': 'Title I schoolwide eligible-Title I targeted assistance program', 'nationalSchoolLunchProgram': 'Yes participating without using any Provision or the CEO'}, {'ncesID': '010000500889', 'name': 'Albertville Elementary School', 'year': '2021', 'titleI': 'Title I schoolwide eligible-Title I targeted assistance program', 'nationalSchoolLunchProgram': 'Yes participating without using any Provision or the CEO'},

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

def insert_data(session, data, batch_size=1000):
    # Calculate total number of batches for tqdm
    total_batches = len(data) // batch_size + (1 if len(data) % batch_size else 0)

    for i in tqdm(range(0, len(data), batch_size), total=total_batches, desc="Inserting batches"):
        batch = data[i:i + batch_size]
        cypher_query = """
        UNWIND $data AS schoolData
        MERGE (i:Institution {ncesID: schoolData.ncesID})
        SET
            i.union = schoolData.union,
            i.titleI = schoolData.titleI,
            i.magnet = schoolData.magnet,
            i.nationalSchoolLunchProgram = schoolData.nationalSchoolLunchProgram,
            i.sharedTime = schoolData.sharedTime,
            i.virtual = schoolData.virtual
        """
        session.run(cypher_query, {"data": batch})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
with driver.session() as session:
    insert_data(session, transformed_school_characteristics_data, batch_size=1000)

driver.close()

Inserting batches: 100%|██████████| 100/100 [00:15<00:00,  6.55it/s]


## CCD School Lunch

### Extract

In [ ]:
import pandas as pd

# Load the data
school_lunch_path = './ccd_sch_033_2021_l_1a_080621.csv'

school_lunch_df = pd.read_csv(school_lunch_path, encoding='latin1', dtype= dtype_dict)

# Check the DataFrame
print(school_lunch_df.head())

  SCHOOL_YEAR  FIPST STATENAME  ST                   SCH_NAME  \
0   2020-2021      1   ALABAMA  AL  Albertville Middle School   
1   2020-2021      1   ALABAMA  AL  Albertville Middle School   
2   2020-2021      1   ALABAMA  AL  Albertville Middle School   
3   2020-2021      1   ALABAMA  AL  Albertville Middle School   
4   2020-2021      1   ALABAMA  AL  Albertville Middle School   

   STATE_AGENCY_NO  UNION ST_LEAID    LEAID     ST_SCHID       NCESSCH  \
0                1    NaN   AL-101  0100005  AL-101-0010  010000500870   
1                1    NaN   AL-101  0100005  AL-101-0010  010000500870   
2                1    NaN   AL-101  0100005  AL-101-0010  010000500870   
3                1    NaN   AL-101  0100005  AL-101-0010  010000500870   
4                1    NaN   AL-101  0100005  AL-101-0010  010000500870   

    SCHID                          DATA_GROUP                  LUNCH_PROGRAM  \
0  100870                Direct Certification                 Not Applicable   
1  1

In [ ]:
import pandas as pd
from tqdm import tqdm

def transform_school_lunch_data(df):
    transformed_data = []
    # Filter rows to include only those related to free and reduced-price lunch qualifications
    relevant_rows = df[df['LUNCH_PROGRAM'].isin(['Reduced-price lunch qualified', 'Free lunch qualified'])]

    # Group data by school ID to accumulate counts
    grouped_data = relevant_rows.groupby(['NCESSCH', 'SCH_NAME', 'SCHOOL_YEAR'])

    # Iterate over each group
    for (nces_id, school_name, school_year), group in tqdm(grouped_data, total=grouped_data.ngroups):
        free_lunch_data = group[group['LUNCH_PROGRAM'] == 'Free lunch qualified']['STUDENT_COUNT']
        reduced_lunch_data = group[group['LUNCH_PROGRAM'] == 'Reduced-price lunch qualified']['STUDENT_COUNT']

        # Check if data is not empty and convert to int safely
        free_lunch_count = int(pd.to_numeric(free_lunch_data, errors='coerce').dropna().sum())
        reduced_price_lunch_count = int(pd.to_numeric(reduced_lunch_data, errors='coerce').dropna().sum())

        # Only include the school if either count is greater than 0
        if free_lunch_count > 0 or reduced_price_lunch_count > 0:
            school_info = {
                'ncesID': nces_id,
                'name': school_name,
                'year': school_year[-4:],  # Extract last four digits for the year
            }
            if free_lunch_count > 0:
                school_info['freeLunch'] = free_lunch_count
            if reduced_price_lunch_count > 0:
                school_info['reducedPriceLunch'] = reduced_price_lunch_count

            transformed_data.append(school_info)

    return transformed_data

# Assuming school_lunch_df is your DataFrame loaded from a CSV file
school_lunch_df_sub = school_lunch_df #.head(100)  # Example subset for quick processing

transformed_school_lunch_data = transform_school_lunch_data(school_lunch_df_sub)
print(transformed_school_lunch_data[:20])  # Print first 5 entries to verify

100%|██████████| 99322/99322 [02:45<00:00, 599.34it/s]

[{'ncesID': '010000500870', 'name': 'Albertville Middle School', 'year': '2021', 'freeLunch': 332}, {'ncesID': '010000500871', 'name': 'Albertville High School', 'year': '2021', 'freeLunch': 456}, {'ncesID': '010000500879', 'name': 'Albertville Intermediate School', 'year': '2021', 'freeLunch': 330}, {'ncesID': '010000500889', 'name': 'Albertville Elementary School', 'year': '2021', 'freeLunch': 397}, {'ncesID': '010000501616', 'name': 'Albertville Kindergarten and PreK', 'year': '2021', 'freeLunch': 181}, {'ncesID': '010000502150', 'name': 'Albertville Primary School', 'year': '2021', 'freeLunch': 422}, {'ncesID': '010000600193', 'name': 'Kate Duncan Smith DAR Middle', 'year': '2021', 'freeLunch': 127, 'reducedPriceLunch': 29}, {'ncesID': '010000600872', 'name': 'Asbury High School', 'year': '2021', 'freeLunch': 473, 'reducedPriceLunch': 51}, {'ncesID': '010000600877', 'name': 'Douglas Elementary School', 'year': '2021', 'freeLunch': 221, 'reducedPriceLunch': 22}, {'ncesID': '01000060

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

def insert_data(session, data, batch_size=1000):
    # Calculate total number of batches for tqdm
    total_batches = len(data) // batch_size + (1 if len(data) % batch_size else 0)

    for i in tqdm(range(0, len(data), batch_size), total=total_batches, desc="Inserting batches"):
        batch = data[i:i + batch_size]
        cypher_query = """
        UNWIND $data AS schoolData
        MATCH (i:Institution {ncesID: schoolData.ncesID}) // Ensure Institution exists

        // Handle Free Lunch Intervention
        MERGE (fl:Intervention {name: 'Free Lunch'})
        WITH i, fl, schoolData
        FOREACH (_ IN CASE WHEN schoolData.freeLunch IS NOT NULL THEN [1] ELSE [] END |
            MERGE (i)-[r:ASSIGNS {year: schoolData.year}]->(fl)
            ON CREATE SET r.total = schoolData.freeLunch
            ON MATCH SET r.total = schoolData.freeLunch
        )

        // Carry forward institution, year, and reduced price lunch data
        WITH i, schoolData
        MERGE (rpl:Intervention {name: 'Reduced Price Lunch'})
        FOREACH (_ IN CASE WHEN schoolData.reducedPriceLunch IS NOT NULL THEN [1] ELSE [] END |
            MERGE (i)-[s:ASSIGNS {year: schoolData.year}]->(rpl)
            ON CREATE SET s.total = schoolData.reducedPriceLunch
            ON MATCH SET s.total = schoolData.reducedPriceLunch
        )
        """
        session.run(cypher_query, {"data": batch})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
with driver.session() as session:
    insert_data(session, transformed_school_lunch_data, batch_size=1000)

driver.close()

Inserting batches: 100%|██████████| 75/75 [00:49<00:00,  1.51it/s]


## CCD LEA Directory

### Extract

In [ ]:
import pandas as pd

# Load the data
lea_directory_path = './ccd_lea_029_2021_w_1a_080621.csv'

lea_directory_df = pd.read_csv(lea_directory_path, encoding='latin1', dtype= dtype_dict)

# Check the DataFrame
print(lea_directory_df.head())

  SCHOOL_YEAR  FIPST STATENAME  ST                LEA_NAME  STATE_AGENCY_NO  \
0   2020-2021      1   ALABAMA  AL  Alabama Youth Services                1   
1   2020-2021      1   ALABAMA  AL        Albertville City                1   
2   2020-2021      1   ALABAMA  AL         Marshall County                1   
3   2020-2021      1   ALABAMA  AL             Hoover City                1   
4   2020-2021      1   ALABAMA  AL            Madison City                1   

  UNION ST_LEAID    LEAID                MSTREET1  ... G_11_OFFERED  \
0   NaN   AL-210  0100002               PO Box 66  ...          Yes   
1   NaN   AL-101  0100005           107 W Main St  ...          Yes   
2   NaN   AL-048  0100006  12380 US Highway 431 S  ...          Yes   
3   NaN   AL-158  0100007   2810 Metropolitan Way  ...          Yes   
4   NaN   AL-169  0100008           211 Celtic Dr  ...          Yes   

  G_12_OFFERED G_13_OFFERED G_UG_OFFERED  G_AE_OFFERED  GSLO GSHI  LEVEL  \
0          Yes        

<ipython-input-69-ea27ddbc2075>:6: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  lea_directory_df = pd.read_csv(lea_directory_path, encoding='latin1', dtype= dtype_dict)


In [ ]:
import pandas as pd
from tqdm import tqdm

def format_available_grades(gslo, gshi):
    if pd.notnull(gslo) and pd.notnull(gshi) and gslo not in {'N', 'M', 'UG'} and gshi not in {'N', 'M', 'UG'}:
        return f"{gslo} - {gshi}"
    else:
        return

def transform_lea_data(df):
    transformed_data = []
    # Iterate over the rows of the DataFrame
    for _, row in tqdm(df.iterrows(), total=df.shape[0]):
        # Year from the school year
        year = row['SCHOOL_YEAR'][-4:]

        # Collect transformed data
        lea_info = {
            'ncesID': row['LEAID'],
            'name': row['LEA_NAME'],
            'type': row['LEA_TYPE_TEXT'],
            'year': year,
        }

        # Optional fields
        if pd.notnull(row['UNION']):
            lea_info['union'] = (row['UNION'])

        if pd.notnull(row['WEBSITE']):
            lea_info['website'] = row['WEBSITE']

        if pd.notnull(row['GSLO']) and pd.notnull(row['GSHI']):
            lea_info['availableGrades'] = format_available_grades(row['GSLO'], row['GSHI'])

        transformed_data.append(lea_info)


    return transformed_data

lea_directory_df_sub = lea_directory_df #.head(10)

transformed_lea_directory_data = transform_lea_data(lea_directory_df_sub)
print(transformed_lea_directory_data[:5])  # Print first 5 entries to verify

100%|██████████| 19690/19690 [00:05<00:00, 3642.95it/s]

[{'ncesID': '0100002', 'name': 'Alabama Youth Services', 'type': 'Regular public school district that is not a component of a supervisory union', 'year': '2021', 'website': 'http://www.dys.alabama.gov/school-district.html', 'availableGrades': 'KG - 12'}, {'ncesID': '0100005', 'name': 'Albertville City', 'type': 'Regular public school district that is not a component of a supervisory union', 'year': '2021', 'website': 'http://www.albertk12.org', 'availableGrades': 'PK - 12'}, {'ncesID': '0100006', 'name': 'Marshall County', 'type': 'Regular public school district that is not a component of a supervisory union', 'year': '2021', 'website': 'http://www.marshallk12.org', 'availableGrades': 'PK - 12'}, {'ncesID': '0100007', 'name': 'Hoover City', 'type': 'Regular public school district that is not a component of a supervisory union', 'year': '2021', 'website': 'http://www.hoovercityschools.net', 'availableGrades': 'PK - 12'}, {'ncesID': '0100008', 'name': 'Madison City', 'type': 'Regular pub

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm

def insert_data(session, data, batch_size=1000):
    # Calculate total number of batches for tqdm
    total_batches = len(data) // batch_size + (1 if len(data) % batch_size else 0)

    for i in tqdm(range(0, len(data), batch_size), total=total_batches, desc="Inserting batches"):
        batch = data[i:i + batch_size]
        cypher_query = """
        UNWIND $data AS leaData
        MERGE (lea:LEA {ncesID: leaData.ncesID})
        SET
            lea.union = leaData.union,
            lea.website = leaData.website,
            lea.type = leaData.type,
            lea.availableGrades = leaData.availableGrades
        """
        session.run(cypher_query, {"data": batch})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
with driver.session() as session:
    insert_data(session, transformed_lea_directory_data, batch_size=1000)

driver.close()

Inserting batches: 100%|██████████| 20/20 [00:08<00:00,  2.40it/s]


## CCD LEA Staff

### Extract

In [ ]:
import pandas as pd

# Load the data
lea_staff_path = './ccd_lea_059_2021_l_1a_080621.csv'

lea_staff_df = pd.read_csv(lea_staff_path, encoding='latin1', dtype= dtype_dict)

# Check the DataFrame
print(lea_staff_df.head())

  SCHOOL_YEAR  FIPST STATENAME  ST                LEA_NAME  STATE_AGENCY_NO  \
0   2020-2021      1   ALABAMA  AL  Alabama Youth Services                1   
1   2020-2021      1   ALABAMA  AL  Alabama Youth Services                1   
2   2020-2021      1   ALABAMA  AL  Alabama Youth Services                1   
3   2020-2021      1   ALABAMA  AL  Alabama Youth Services                1   
4   2020-2021      1   ALABAMA  AL  Alabama Youth Services                1   

  UNION ST_LEAID    LEAID                                              STAFF  \
0   NaN   AL-210  0100002                            All Other Support Staff   
1   NaN   AL-210  0100002                       Elementary School Counselors   
2   NaN   AL-210  0100002                                Elementary Teachers   
3   NaN   AL-210  0100002  Instructional Coordinators and Supervisors to ...   
4   NaN   AL-210  0100002                              Kindergarten Teachers   

   STAFF_COUNT TOTAL_INDICATOR  DMS_FLAG  
0

### Transform

In [ ]:
import pandas as pd
from tqdm import tqdm

def transform_data(df):
    transformed_data = []

    # Define the mapping structure for categories and their corresponding details
    staff_structure = {
        'Teachers': {
            'Total': 'Teachers',
            'Pre-kindergarten Teachers': 'Pre-kindergarten Teachers',
            'Kindergarten Teachers': 'Kindergarten Teachers',
            'Elementary Teachers': 'Elementary Teachers',
            'Secondary Teachers': 'Secondary Teachers',
            'Ungraded Teachers': 'Ungraded Teachers'
        },
        'LEA Staff': {
            'Total': 'LEA Staff',
            'LEA Administrative Support Staff': 'LEA Administrative Support Staff',
            'LEA Administrators': 'LEA Administrators',
            'Instructional Coordinators and Supervisors to the Staff': 'Instructional Coordinators and Supervisors to the Staff'
        },
        'School Staff': {
            'Total': 'School Staff',
            'Librarians/media specialists': 'Librarians/media specialists',
            'Library/Media Support Staff': 'Library/Media Support Staff',
            'Paraprofessionals/Instructional Aides': 'Paraprofessionals/Instructional Aides',
            'School Administrative Support Staff': 'School Administrative Support Staff',
            'School Administrators': 'School Administrators',
            'School Psychologists': 'School Psychologists',
            'Student Support Services Staff (w/o Psychology)': 'Student Support Services Staff (w/o Psychology)'
        },
        'Guidance Counselors': {
            'Total': 'Guidance Counselors',
            'School Counselors': 'School Counselors',
            'Elementary School Counselors': 'Elementary School Counselors',
            'Secondary School Counselors': 'Secondary School Counselors'
        },
        'All Other Staff': 'All Other Support Staff',
        'No Category Codes': 'No Category Codes'
    }

    # Group data by LEA ID and name
    grouped_data = df.groupby(['LEAID', 'LEA_NAME'])
    for (lea_id, lea_name), group in tqdm(grouped_data, total=grouped_data.ngroups):
        lea_info = {
            'lea_id': lea_id,
            'lea_name': lea_name,
            'staff': {}  # Initialize the staff dictionary
        }

        # Iterate through each category and subcategory to populate data
        for main_category, details in staff_structure.items():
            if isinstance(details, dict):  # Categories with subcategories
                for subcategory_label, staff_type in details.items():
                    # Filter and sum positive values for the given category
                    staff_count_series = group[group['STAFF'] == staff_type]['STAFF_COUNT'].apply(pd.to_numeric, errors='coerce').dropna()
                    staff_count = staff_count_series[staff_count_series > 0].sum()
                    if staff_count > 0:
                        lea_info['staff'].setdefault(main_category, {})[subcategory_label] = float("{:.2f}".format(staff_count))
            else:  # Single categories
                staff_count_series = group[group['STAFF'] == details]['STAFF_COUNT'].apply(pd.to_numeric, errors='coerce').dropna()
                staff_count = staff_count_series[staff_count_series > 0].sum()
                if staff_count > 0:
                    lea_info['staff'][main_category] = float("{:.2f}".format(staff_count))

        # Only append if there's meaningful data
        if len(lea_info['staff']) > 0:  # Ensure there is at least one staff entry
            transformed_data.append(lea_info)

    return transformed_data

# Assuming 'staff_df' is your DataFrame containing the staff data
lea_staff_df_sub = lea_staff_df #.head(200)

transformed_lea_staff_data = transform_data(lea_staff_df_sub)
print(transformed_lea_staff_data[:5])  # Print first 5 entries to verify

100%|██████████| 19468/19468 [09:29<00:00, 34.18it/s]

[{'lea_id': '0100005', 'lea_name': 'Albertville City', 'staff': {'Teachers': {'Total': 301.5, 'Pre-kindergarten Teachers': 3.0, 'Kindergarten Teachers': 27.0, 'Elementary Teachers': 147.5, 'Secondary Teachers': 124.0}, 'LEA Staff': {'Total': 29.0, 'LEA Administrative Support Staff': 22.0, 'LEA Administrators': 7.0}, 'School Staff': {'Total': 75.42, 'Librarians/media specialists': 6.0, 'Library/Media Support Staff': 3.0, 'Paraprofessionals/Instructional Aides': 15.42, 'School Administrative Support Staff': 13.0, 'Student Support Services Staff (w/o Psychology)': 9.0}, 'Guidance Counselors': {'Total': 13.0, 'Elementary School Counselors': 7.0, 'Secondary School Counselors': 6.0}, 'All Other Staff': 47.0, 'No Category Codes': 465.92}}, {'lea_id': '0100006', 'lea_name': 'Marshall County', 'staff': {'Teachers': {'Total': 298.16, 'Pre-kindergarten Teachers': 9.65, 'Kindergarten Teachers': 33.65, 'Elementary Teachers': 107.26, 'Secondary Teachers': 147.6}, 'LEA Staff': {'Total': 34.0, 'LEA Ad

### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm
import json

def insert_data(session, data, batch_size=1000):
    # Calculate total number of batches for tqdm
    total_batches = len(data) // batch_size + (1 if len(data) % batch_size else 0)

    for i in tqdm(range(0, len(data), batch_size), total=total_batches, desc="Inserting batches"):
        batch = data[i:i + batch_size]
        # Serialize complex data structures into JSON strings
        for entry in batch:
            entry['staff'] = json.dumps(entry['staff'])

        cypher_query = """
        UNWIND $data AS leaData
        MERGE (lea:LEA {ncesID: leaData.lea_id})
        SET lea.staff = leaData.staff
        """
        session.run(cypher_query, {"data": batch})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
with driver.session() as session:
    insert_data(session, transformed_lea_staff_data, batch_size=1000)

driver.close()

Inserting batches: 100%|██████████| 18/18 [00:04<00:00,  4.19it/s]


## Urban Institute School level poverty (MEPS)

### Extract and Transform

In [ ]:
from urllib.request import urlopen
from json import loads
from tqdm import tqdm

def fetch_all_data(url, total_records, records_per_page=10000):
    results = []
    # Estimate total pages needed
    total_pages = (total_records // records_per_page) + (1 if total_records % records_per_page else 0)
    pbar = tqdm(total=total_pages, desc='Fetching data')

    while url:
        response = urlopen(url)
        data = loads(response.read())
        results.extend(data['results'])
        url = data['next']  # Move to the next page if available
        pbar.update(1)  # Update progress bar for each page fetched
    pbar.close()
    return results

# Base URL for the MEPS data for the year 2020
base_url = "https://educationdata.urban.org/api/v1/schools/meps/2020/"
total_records = 94685  # Total number of records as provided

# Fetch all MEPS data
meps_data = fetch_all_data(base_url, total_records)

# Transform data to extract necessary details and round the poverty percentage, excluding None values
meps_list = [
    {'ncessh': entry['ncessch'], 'meps_poverty_pct': round(entry['meps_poverty_pct'], 2)}
    for entry in meps_data if entry['meps_poverty_pct'] is not None
]

# Output the length and the first 10 records
print("Total records:", len(meps_list))
print("First 10 records:", meps_list[:10])

Fetching data:   5%|▌         | 10/190 [00:03<01:08,  2.64it/s]

Total records: 91456
First 10 records: [{'ncessh': '010000500870', 'meps_poverty_pct': 20.61}, {'ncessh': '010000500871', 'meps_poverty_pct': 16.53}, {'ncessh': '010000500879', 'meps_poverty_pct': 22.57}, {'ncessh': '010000500889', 'meps_poverty_pct': 22.44}, {'ncessh': '010000501616', 'meps_poverty_pct': 17.83}, {'ncessh': '010000502150', 'meps_poverty_pct': 22.41}, {'ncessh': '010000600193', 'meps_poverty_pct': 17.53}, {'ncessh': '010000600872', 'meps_poverty_pct': 22.77}, {'ncessh': '010000600877', 'meps_poverty_pct': 21.9}, {'ncessh': '010000600878', 'meps_poverty_pct': 19.46}]


### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm
import json

def insert_data(session, data, batch_size=1000):
    # Calculate total number of batches for tqdm
    total_batches = len(data) // batch_size + (1 if len(data) % batch_size else 0)

    for i in tqdm(range(0, len(data), batch_size), total=total_batches, desc="Inserting batches"):
        batch = data[i:i + batch_size]

        cypher_query = """
        UNWIND $data AS schoolData
        MERGE (i:Institution {ncesID: schoolData.ncessh})
        SET i.pctStudentsInPovertyMEPS = schoolData.meps_poverty_pct
        """
        session.run(cypher_query, {"data": batch})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
with driver.session() as session:
    insert_data(session, meps_list, batch_size=1000)

driver.close()

Inserting batches: 100%|██████████| 92/92 [00:13<00:00,  6.72it/s]


## Census Bureau School Distric SAIPE poverty estimate

### Extract and Transform

In [ ]:
from urllib.request import urlopen
from json import loads
from tqdm import tqdm

def fetch_all_data(url, total_records, records_per_page=10000):
    results = []
    # Estimate total pages needed
    total_pages = (total_records // records_per_page) + (1 if total_records % records_per_page else 0)
    pbar = tqdm(total=total_pages, desc='Fetching data')

    while url:
        response = urlopen(url)
        data = loads(response.read())
        results.extend(data['results'])
        url = data['next']  # Move to the next page if available
        pbar.update(1)  # Update progress bar for each page fetched
    pbar.close()
    return results

# Base URL for the SAIPE data for the year 2020-2021
base_url = "https://educationdata.urban.org/api/v1/school-districts/saipe/2020/"
total_records = 13164  # Total number of records as provided

# Fetch all SAIPE data
saipe_data = fetch_all_data(base_url, total_records)

# Transform data to extract necessary details and round the poverty percentage, excluding None values
saipe_list = [{
    'lea_id': entry['leaid'],
    'pct_youth_population': round(entry['est_population_5_17_pct'], 2) if entry['est_population_5_17_pct'] is not None else None,
    'pct_youth_poverty': round(entry['est_population_5_17_poverty_pct'], 2) if entry['est_population_5_17_poverty_pct'] is not None else None
} for entry in saipe_data if entry['leaid'] is not None and entry['est_population_5_17_pct'] is not None and entry['est_population_5_17_poverty_pct'] is not None]

# Output the length and the first 10 records
print("Total records:", len(saipe_list))
print("First 10 records:", saipe_list[:10])


Fetching data: 100%|██████████| 2/2 [00:00<00:00,  2.46it/s]

Total records: 13164
First 10 records: [{'lea_id': '0100190', 'pct_youth_population': 0.19, 'pct_youth_poverty': 0.1}, {'lea_id': '0100005', 'pct_youth_population': 0.19, 'pct_youth_poverty': 0.22}, {'lea_id': '0100030', 'pct_youth_population': 0.15, 'pct_youth_poverty': 0.27}, {'lea_id': '0100060', 'pct_youth_population': 0.17, 'pct_youth_poverty': 0.24}, {'lea_id': '0100090', 'pct_youth_population': 0.14, 'pct_youth_poverty': 0.24}, {'lea_id': '0100100', 'pct_youth_population': 0.18, 'pct_youth_poverty': 0.14}, {'lea_id': '0100120', 'pct_youth_population': 0.15, 'pct_youth_poverty': 0.14}, {'lea_id': '0100180', 'pct_youth_population': 0.16, 'pct_youth_poverty': 0.24}, {'lea_id': '0100210', 'pct_youth_population': 0.12, 'pct_youth_poverty': 0.09}, {'lea_id': '0100240', 'pct_youth_population': 0.17, 'pct_youth_poverty': 0.14}]


### Load

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm
import json

def insert_data(session, data, batch_size=1000):
    # Calculate total number of batches for tqdm
    total_batches = len(data) // batch_size + (1 if len(data) % batch_size else 0)

    for i in tqdm(range(0, len(data), batch_size), total=total_batches, desc="Inserting batches"):
        batch = data[i:i + batch_size]

        cypher_query = """
        UNWIND $data AS leaData
        MERGE (lea:LEA {ncesID: leaData.lea_id})
        SET
            lea.pctStudentsInPovertySAIPE = leaData.pct_youth_poverty,
            lea.pctYouthPopulation = leaData.pct_youth_population
        """
        session.run(cypher_query, {"data": batch})

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
with driver.session() as session:
    insert_data(session, saipe_list, batch_size=1000)

driver.close()

Inserting batches: 100%|██████████| 14/14 [00:03<00:00,  3.78it/s]
